In [0]:
%cd /dbfs/FileStore/tables/riskreport/FinalCode
# /FileStore/tables/riskreport/FinalCode/Input_Files/377958_AIG-1.PDF

/dbfs/FileStore/tables/riskreport/FinalCode

In [0]:
%pwd

Out[109]: '/dbfs/FileStore/tables/riskreport/FinalCode'

In [0]:
dbutils.widgets.text("FileName","")

In [0]:
fileName= dbutils.widgets.get("FileName")

In [0]:
!pip install pdf.tocgen
!pip install subprocess.run

In [0]:
#dbutils.library.installPyPI('pdf.tocgen')
#dbutils.library.installPyPI('subprocess.run')
dbutils.library.installPyPI('opencv-python')
dbutils.library.installPyPI('azure.ai.formrecognizer')
dbutils.library.installPyPI('pdf2image')
dbutils.library.installPyPI('shutils')
dbutils.library.installPyPI('nltk')
dbutils.library.installPyPI('spacy')
dbutils.library.installPyPI('fuzzywuzzy')
dbutils.library.installPyPI('toml')
dbutils.library.installPyPI('pymupdf')

Out[1]: True

In [0]:
dbutils.library.list()

Out[2]: ['pdf2image', 'toml', 'opencv-python', 'shutils', 'pymupdf', 'fuzzywuzzy', 'nltk', 'azure.ai.formrecognizer', 'spacy']

In [0]:
import toml
import os
import pandas as pd
import subprocess
import json
import shutil
import traceback
from nltk import word_tokenize
import pandas as pd
from pandas import DataFrame
import itertools
import re
import nltk
import json
import string
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import spacy
import warnings
import cv2
import glob
from PIL import Image
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from pdf2image import convert_from_path, convert_from_bytes
import tempfile
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
from matplotlib import pyplot as plt
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient
from shutil import copyfile
from fuzzywuzzy import fuzz,process
from requests import get, post
import numpy as np
from datetime import datetime
import time
import csv
import io
#*******************import libraries for flask***************************************
#import os
#import urllib.request
#from app import app
#from flask import Flask, request, redirect, jsonify
#from werkzeug.utils import secure_filename
#from datetime import datetime
#from dateutil.relativedelta import relativedelta
import fitz
import re
import os
from collections import OrderedDict
warnings.filterwarnings('ignore')

nltk.download('punkt') # ---- Add this

/databricks/python/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
 warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
Out[6]: True

In [0]:
#os.mkdir("/dbfs/FileStore/tables/riskreport/FinalCode/TOC")
#import os
#os.remove("/dbfs/FileStore/tables/riskreport/FinalCode/heading.json")


In [0]:
import string
import random
N=10
res = ''.join(random.choices(string.ascii_uppercase +string.digits, k = N)) 
print(res)

toc_name = "/dbfs/FileStore/tables/riskreport/FinalCode/TOC/"+res

In [0]:
#with open('/dbfs/FileStore/tables/riskreport/FinalCode/heading.json') as f:
#  Construction_Headings = json.load(f)['Headings']
#  print(Construction_Headings)
  
#start_time = datetime.datetime.now()  
  
Construction_Headings = ['PROTECTION','WATER SUPPLY','SUPERVISION','Protection & Hazards','Public Protection','Protection Features']
#Construction_Headings = ['SUPERVISION']
print(Construction_Headings)

['PROTECTION', 'WATER SUPPLY', 'SUPERVISION', 'Protection & Hazards', 'Public Protection', 'Protection Features']

In [0]:
def check_page(result, heading, toc_page):
  start_page, end_page = 0,0
  for toc_index,toc_heading in enumerate(result):
    tmp = toc_heading.split(' ')
    headers,pageno = " ".join(tmp[:-1]), tmp[-1]
    tmp_header = headers
    headers = headers.lstrip()

    if (('table of content' in headers.lower() or 'T ABLE OF C ONTENT' in headers 
         or 'contents' in headers.lower() or 'index' in headers.lower()) 
        and toc_page == 0 ):
      toc_page = int(pageno)

    headers = headers.split('  ')
    if start_page == 0:
      for ind,header in enumerate(headers):
        header = header.replace('"','').strip()
        limit_cond = True if (len(header.split(' ')) <=7) else False
        if ((heading.lower() in header.lower()) and limit_cond 
            and int(pageno) != toc_page):

          if heading == "CONSTRUCTION":
            if not (any(1 for val in header.split(' ') 
              if heading.lower() in val.lower() and not val.islower())):
              continue

          try:
            # print(header)
            start_page = int(header.split(' ')[-1])
            if (ind+1)<len(headers):
              end_page = int(headers[ind+1].split(' ')[-1])
              return start_page, end_page, toc_page

            else:
              next_page = result[toc_index+1].split('"')
              end_page = int(next_page[1].split(' ')[-1])
              return start_page, end_page, toc_page

          except:
            start_page = int(pageno)
            break
    else:

      try:
        end_page = int(pageno)  
        return start_page, end_page, toc_page
      except:
        return 0, 0, toc_page
  if start_page != 0 and end_page == 0:
    end_page = -1

  return start_page, end_page, toc_page

def getpage(pdffile, toc_page, headingg, check_value = None):
  result = subprocess.run(["pdftocgen", "-r", toc_name+"final_risk_protection.toml", 
                           pdffile],stdout=subprocess.PIPE,text=True)
  result = result.stdout.split("\n")
  if len(result)<=2:
    return 0,0, ' ', toc_page
  result = result[:-1]

  if not check_value:
    check_value = Construction_Headings

  start_page, end_page = 0,0
  for con_heading in check_value:
    con_heading = headingg
    start_page, end_page, toc_page = check_page(result,con_heading,toc_page)
    if start_page !=0 and end_page !=0:
      return start_page, end_page, con_heading, toc_page

  return 0, 0, ' ', toc_page

def create_template(template,prev_font_size, headingg):
  prev = 0
  level = 0
  prev_font_name = []
  final = {'heading':[]}
  file = toml.loads(template)
  # if not file:
  #   return prev_font_size
  sorted_script = sorted(list(file.values())[0],
                         key = lambda x:x['font']['size'],reverse=True)

  for i in sorted_script:
    if i['font']['size'] < 10.0 and 'Bold' not in i['font']['name']:
        continue
    if i['font']['size'] != prev and (9.5 < i['font']['size'] < prev_font_size):
      level += 1
      if level > 1:
        break
      i['level'] = level
      prev = i['font']['size']
      prev_font_name = [i['font']['name']]
      final['heading'].append(i)

    elif i['font']['size'] == prev and i['font']['name'] not in prev_font_name:
      i['level'] = level
      final['heading'].append(i)
      prev_font_name.append(i['font']['name'])
      continue

  with open(toc_name+"final_risk_protection.toml","w") as f:
    toml.dump(final,f)
  return prev

def Extract_page(pdf_file_path, headingg):
  prev_font = 26.0
  toc_pageno = 0
  prev_result = []
  
  template = subprocess.run(["pdfxmeta", "-a 1", "-i", pdf_file_path],stdout=subprocess.PIPE,text=True)
  template_out = template.stdout

  tocpage = subprocess.run(["pdfxmeta", "-a 1", "-i", pdf_file_path, 
                          "TABLE OF CONTENT"],stdout=subprocess.PIPE,text=True)
  tocpageout = tocpage.stdout
  if tocpageout:
    create_template(tocpageout,prev_font, headingg)
    tocresult = subprocess.run(["pdftocgen", "-r", toc_name+"final_risk_protection.toml", 
                                pdf_file_path],stdout=subprocess.PIPE,text=True)
    tocresult = tocresult.stdout.split('\n')
    for tocpage in tocresult[:-1]:
      toc_tmp = tocpage.split(' ')
      tocheaders,tocpageno = " ".join(toc_tmp[:-1]), toc_tmp[-1]
      if 'table of content' in tocheaders.lower():
        toc_pageno = int(tocpageno)

  while prev_font > 0 and template_out:
    prev_font = create_template(template_out,prev_font, headingg)
    pages = getpage(pdf_file_path,toc_pageno, headingg)
    if toc_pageno == 0:
      toc_pageno = pages[3] 

    if pages[0] !=0 and pages[1] !=0:
      return pages[0], pages[1], pages[2]
      
  return 0, 0, ''


In [0]:
first_file=''
if '.' in fileName:
  first_file=fileName.split('.')[0]
print(first_file)


In [0]:
import string
import random
N=10
res = ''.join(random.choices(string.ascii_uppercase +string.digits, k = N)) 

In [0]:

# shutil.rmtree('/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/') 
# os.mkdir("/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/")

# shutil.rmtree('/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/') 
# os.mkdir("/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/")
try:
    pdf2image_path_protection = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/" + "protection" + "_" + first_file + res + '/'
    final_protection_images_path=pdf2image_path_protection
    os.mkdir(pdf2image_path_protection)
    logo_detection_path="/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/"+"Logo"+"_"+first_file+res+'/'
    final_logo_detection=logo_detection_path
    print(logo_detection_path)
    os.mkdir(logo_detection_path)
except FileExistsError:
    pdf2image_path_protection = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/" + "protection" + "_" + first_file + res + '/'
    shutil.rmtree(pdf2image_path_protection)
    logo_detection_path = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/" + "Logo" + "_" + first_file + res + '/'
    shutil.rmtree(logo_detection_path)
    dbutils.notebook.exit(json.dumps({
        "isToast": "True",
        "Error_message": "Please upload and run the file again!!!!!",
        "filename": fileName
    }))

/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/Logo_377955 AIG_redactedEET2VR6H7Y/

/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/Logo_377955 AIG_redactedWRXH6IE5Y7/

In [0]:
# for i in dict_list:
#   if start_chubb is not empty:
#     pdftoimage(filepath,IMAGE_PATH,first_page,last_page)

In [0]:
# PDF To Images
#filepath='/dbfs/FileStore/tables/riskreport/FinalCode/Input_Files/413456_AIG_3CRHPUBLICLIMITEDCOMPANY_AGC_Neosho_KS_Neosho_United_States_COPE_Report_09_Oct_19_772140_.pdf' AIG_Input/4663887881116584_377955 AIG_redacted.pdf
#AIG_Input/408487 Global Risk Consultants_redacted.pdf
#AIG_Input/392417 Chubb RES.pdf
#basepath ="/dbfs/mnt/aria/Batch_Input/Chubb Documents/"
basepath ="/dbfs/mnt/aria/AIG_Input/"

#fileName = "408487 Global Risk Consultants_redacted.pdf" #xzxz
#fileName = "377955 AIG_redacted.pdf" #xzxz
fileName=fileName
path = os.path.join(basepath,fileName)
print(path)
IMAGE_PATH=pdf2image_path_protection
filepath = path

def pdftoimage(filepath,IMAGE_PATH,start,end):
  with tempfile.TemporaryDirectory() as path:
    #images_from_path = convert_from_path(filepath,grayscale=False ,dpi=200, output_folder=IMAGE_PATH,fmt='png')
    images_from_path = convert_from_path(filepath,grayscale=False ,dpi=200, first_page=start, last_page=end, output_folder=IMAGE_PATH,fmt='png')
#pdftoimage(path,IMAGE_PATH)
print("IMAGE_PATH : ", IMAGE_PATH)

/dbfs/mnt/aria/AIG_Input/377955 AIG_redacted.pdf
IMAGE_PATH : /dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/

In [0]:
# basepath ="/dbfs/mnt/aria/AIG_Input/"
# filename = "404847.pdf" #xzxz
# path = os.path.join(basepath,filename)
# print(path)
start = {}
try:
  for my_input in Construction_Headings:
    Start_page, End_Page, Matched_Section = Extract_page(path, my_input)
    print(Start_page,End_Page,Matched_Section)

    if Matched_Section == "PROTECTION":
      start['start_AIG'] = Start_page
    if Matched_Section == "SUPERVISION":
      start['end_AIG'] = End_Page
    if Matched_Section == "Protection & Hazards":
      start['start_Chubb'] = Start_page
      start['end_Chubb'] = End_Page
    if Matched_Section == "Protection Features":
      start['start_grc1'] = Start_page
      start['end_grc1'] = End_Page
    if Matched_Section == "Public Protection":
      start['start_grc2'] = Start_page
      start['end_grc2'] = End_Page

  print(start)    

  if 'start_AIG' in start and 'end_AIG' in start:
    print("AIG Section Encountered")
    start_Page = start['start_AIG']
    end_Page = start['end_AIG']
    print(start_Page,end_Page)
    pdftoimage(filepath,IMAGE_PATH,start_Page,end_Page)
  if 'start_Chubb' in start and 'end_Chubb' in start:
    print("Chubb Section Encountered")
    start_Page = start['start_Chubb']
    end_Page = start['end_Chubb']
    print(start_Page,end_Page)
    pdftoimage(filepath,IMAGE_PATH,start_Page,end_Page)
  if 'start_grc1' in start and 'end_grc1' in start:
    print("GRC First Section Encountered")
    start_Page = start['start_grc1']
    end_Page = start['end_grc1']
    print(start_Page,end_Page)
    pdftoimage(filepath,IMAGE_PATH,start_Page,end_Page)
  if 'start_grc2' in start and 'end_grc2' in start:
    print("GRC Second Section Encountered")
    start_Page = start['start_grc2']
    end_Page = start['end_grc2']
    print(start_Page,end_Page)
    pdftoimage(filepath,IMAGE_PATH,start_Page,end_Page)

except FileNotFoundError:
  dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Please check if the file is uploaded in the right path.If you still continue to get this error please contact the administrator!!!!!"
  }))

  
#print(start,end)


17 19 PROTECTION
19 21 WATER SUPPLY
21 22 SUPERVISION
0 0 
0 0 
0 0 
{'start_AIG': 17, 'end_AIG': 22}
AIG Section Encountered
17 22

In [0]:
import os

Images_list = []
#Images_list_=os.listdir('/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection')
Images_list_=os.listdir(pdf2image_path_protection)

#Base_Path = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/"
Base_Path = pdf2image_path_protection
for i in Images_list_:
  i = Base_Path+i
  Images_list.append(i)
print(Images_list)

if len(Images_list) == 0 :
  print("Please check the format of the file")
  dbutils.notebook.exit(json.dumps({
        "isToast": "True",
        "Error_message": "Protection - The file format or the carrier is not handled. Please check the file !!!!!",
        "filename": fileName
  }))  

['/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/14f72845-6cce-4b63-88c5-2f1244fb90c1-17.png', '/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/14f72845-6cce-4b63-88c5-2f1244fb90c1-18.png', '/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/14f72845-6cce-4b63-88c5-2f1244fb90c1-19.png', '/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/14f72845-6cce-4b63-88c5-2f1244fb90c1-20.png', '/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/14f72845-6cce-4b63-88c5-2f1244fb90c1-21.png', '/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/protection_377955 AIG_redactedEET2VR6H7Y/14f72845-6cce-4b63-88c5-2f1244fb90c1-22.png']

In [0]:
#IMAGE_PATH='/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/'
IMAGE_PATH=logo_detection_path
filepath = path
pdftoimage(filepath,IMAGE_PATH,1,2)

import os

Images_list_logo = []
#Images_list_logo_list = os.listdir('/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection')
Images_list_logo_list = os.listdir(logo_detection_path)
#Base_Path = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/"
Base_Path=logo_detection_path
for i in Images_list_logo_list:
  i = Base_Path+i
  Images_list_logo.append(i)
print(Images_list_logo)

['/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/Logo_377955 AIG_redactedEET2VR6H7Y/c3cf2f2c-6753-4cf0-ad0d-bcbd9a2e9293-01.png', '/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/Logo_377955 AIG_redactedEET2VR6H7Y/c3cf2f2c-6753-4cf0-ad0d-bcbd9a2e9293-02.png']

In [0]:
def ratio(word, compare):
	ratio_result = fuzz.ratio(word, compare)
	return ratio_result

def regex_preprocessing(data):
	return re.sub('\W+\d+','', data)

In [0]:
def text_carrier_identification(file):
	line_data = [] 
	word_data = []
	data = {}
	path_to_sample_forms = file

	endpoint = "https://na-aria-formrecognizer.cognitiveservices.azure.com/"
	key = "581924cde7974bd68052fdd925dcdd1f"

	form_recognizer_client = FormRecognizerClient(endpoint=endpoint, credential=AzureKeyCredential(key))
	with open(path_to_sample_forms, "rb") as f:
		poller = form_recognizer_client.begin_recognize_content(form=f)
	form_pages = poller.result()

	for idx, content in enumerate(form_pages):
		for line_idx, line in enumerate(content.lines):
			if line.text not in line_data:
				line_data.append(line.text)
			for word in line.words:
				if word not in word_data:
					word_data.append(word.text)

	data['word'] = word_data
	data['line'] = line_data
	return data

In [0]:
carrier_flag = False
def analyse(wordCorpus, fullInput):
  global carrier_flag
  for line in corpus_LAREC:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              carrier_flag = True
              return 'LAREC'

  for line in corpus_CGU:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              carrier_flag = True
              return 'GCU'
            
  for line in corpus_Glaxo:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90:
              return 'GSK'

  for line in corpus_WTW:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              carrier_flag = True
              return 'WTW'

  for line in corpus_Marsh:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              carrier_flag = True
              return 'Marsh'

  for line in corpus_Bangkok_Insurance:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              return 'Bangkok Insurance'

  for line in corpus_Axa:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              return 'AXA'

  for line in corpus_GRC:
      for lines in fullInput['line']:
          lines = regex_preprocessing(lines)
          ratio_result = ratio(line, lines)
          if ratio_result > 90 and carrier_flag == False:
              carrier_flag = True
              return 'GRC'
          else:
              pass

  for word in wordCorpus:
      for words in fullInput['word']:
          words = regex_preprocessing(words)
          word = word.upper()
          words = words.upper()
          ratio_result = ratio(word, words)
          if ratio_result > 90 and carrier_flag == False:
              if word == "PARAGON" and carrier_flag == False:
                  carrier_flag = True
                  return "ParagonSG"
              #elif word == "BANGKOK" and carrier_flag == False:
              #	carrier_flag = True
              #	return "Bangkok Insurance"
              elif word == "WILLIS" and carrier_flag == False:
                  carrier_flag = True
                  return "WTW"
              else:
                  return word
          else:
              pass

In [0]:
def run(image, word):
	try:
		label = image.split('.')[:-1]
		try:
			result = text_carrier_identification(image)  # ------ Http Change
		except:
			dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file !!!!"
  }))
		l = analyse(word,result)
		if l != None:
			return l
	except:
		traceback.print_exc()


corpus = ['AIG', 'AXA', 'AVA', 'AON', 'CGU', 'CHUBB', 'MARSH', 'PARAGON', 'SAMSUNG', 'WILLIS', 'ZURICH', 'CATLIN', 'GSK','GRC']
corpus_GRC = ["Global Risk Consultants","Global Risk Consultants'"]
corpus_LAREC = ["Latin America Risk Engineering Consultants"]
corpus_CGU = ["CGU SEE IT THROUGH.", "CGU Risk Engineering"]
corpus_WTW = ["Willis Towers Watson Il I'll"]
corpus_Marsh = ["MARSH RISK CONSULTING", "MARSH & MCLENNAN COMPANIES", "MARSH & MCLENNAN"]
corpus_Axa = ["XL CATLIN", "AXA XL Associate Company"]
corpus_Bangkok_Insurance = ["Bangkok Insurance"]
corpus_Glaxo = ["GlaxoSmithKline Plc"]

In [0]:
Carrier_Name = ''
final_result = []
try:
  #for j,i in enumerate(os.listdir("/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/")):
   for j,i in enumerate(os.listdir(logo_detection_path)):
    if j < 2:
      print("File : ",i)
      #start = datetime.datetime.now()
      #full = run("/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/"+i, corpus)
      full = run(logo_detection_path+i, corpus)
      #print("Result : ",full)
      if (full is not None) and (full not in final_result):
          final_result.append(full)
      if full == None:
          print("Unknown Carrier")
      #end = datetime.datetime.now()
      #print("Timetaken : ", end-start)
#      print()
except:
  traceback.print_exc()
  
if len(final_result) == 1:
  Carrier_Name = final_result[0]
elif len(final_result) == 0:
  Carrier_Name = "No_carriers_Found"
  print("No Carriers Found in the Document")
else:
  Carrier_Name = "Multiple_carriers_Found"
  print("Multiple Carriers Found in same Document")

print(final_result)
print(Carrier_Name)
carrier_name = Carrier_Name

File : c3cf2f2c-6753-4cf0-ad0d-bcbd9a2e9293-01.png
File : c3cf2f2c-6753-4cf0-ad0d-bcbd9a2e9293-02.png
['AIG']
AIG

In [0]:
import numpy as np
import cv2

import os

def top_patch(IMAGE_PATH):
	print("Top Patch")
	image=cv2.imread(IMAGE_PATH)
	image_patch=cv2.imread("/dbfs/mnt/aria/image/patch_image.png")
	height,width,channels=image.shape
	resized_image = cv2.resize(image_patch, (int(height*0.6), int(width/8)))
	h1, w1 = resized_image.shape[:2]
	#set top left position of the resized image
	pip_h = 20
	pip_w = 120
	image[pip_h:pip_h+h1,pip_w:pip_w+w1] = resized_image
	cv2.imwrite(IMAGE_PATH,image)
  
def patch_work(image):  
	# Capturing video through webcam
	imageFrame = cv2.imread(image)
   
	# Convert the imageFrame in 
	# BGR(RGB color space) to 
	# HSV(hue-saturation-value)
	# color space
	hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)
  
	# Set range for red color and 
	# define mask (hMin = 179 , sMin = 166, vMin = 180), (hMax = 179 , sMax = 255, vMax = 255)
	red_lower = np.array([179, 166, 180], np.uint8)
	red_upper = np.array([179, 255, 255], np.uint8)
	red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)
  
	# Set range for green color and 
	# define mask (hMin = 54 , sMin = 170, vMin = 172), (hMax = 96 , sMax = 255, vMax = 200)
	green_lower = np.array([54, 170, 172], np.uint8)
	green_upper = np.array([96, 255, 200], np.uint8)
	green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)
  
	# Set range for blue color and
	# define mask (hMin = 82 , sMin = 150, vMin = 162), (hMax = 170 , sMax = 255, vMax = 255) 
	blue_lower = np.array([82, 150, 162], np.uint8) # 100, 255, 172
	blue_upper = np.array([170, 255, 255], np.uint8) # 112, 255, 194
	blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)

	# Set range for yellow color and
	# define mask (hMin = 24 , sMin = 96, vMin = 156), (hMax = 48 , sMax = 255, vMax = 255)
	yellow_lower = np.array([24, 96, 156], np.uint8) # 100, 255, 172
	yellow_upper = np.array([48, 255, 255], np.uint8) # 112, 255, 194
	yellow_mask = cv2.inRange(hsvFrame, yellow_lower, yellow_upper)

	# Set range for light_yellow color and
	# define mask (hMin = 6 , sMin = 255, vMin = 255), (hMax = 10 , sMax = 255, vMax = 255)
	light_yellow_lower = np.array([6, 255, 255], np.uint8) # 100, 255, 172
	light_yellow_upper = np.array([10, 255, 255], np.uint8) # 112, 255, 194
	light_yellow_mask = cv2.inRange(hsvFrame, light_yellow_lower, light_yellow_upper)

	# Set range for pista color and
	# define mask (hMin = 10 , sMin = 190, vMin = 255), (hMax = 24 , sMax = 255, vMax = 255)
	pista_lower = np.array([10, 190, 255], np.uint8) # 100, 255, 172
	pista_upper = np.array([24, 255, 255], np.uint8) # 112, 255, 194
	pista_mask = cv2.inRange(hsvFrame, pista_lower, pista_upper)


  
	# Morphological Transform, Dilation
	# for each color and bitwise_and operator
	# between imageFrame and mask determines
	# to detect only that particular color
	kernal = np.ones((2, 2), "uint8")
  
	# For red color
	red_mask = cv2.dilate(red_mask, kernal)
	res_red = cv2.bitwise_and(imageFrame, imageFrame,mask = red_mask)
  
	# For green color
	green_mask = cv2.dilate(green_mask, kernal)
	res_green = cv2.bitwise_and(imageFrame, imageFrame,mask = green_mask)
  
	# For blue color
	blue_mask = cv2.dilate(blue_mask, kernal)
	res_blue = cv2.bitwise_and(imageFrame, imageFrame,mask = blue_mask)

	# For yellow color
	yellow_mask = cv2.dilate(yellow_mask, kernal)
	res_yellow = cv2.bitwise_and(imageFrame, imageFrame,mask = yellow_mask)

	# For light_yellow color
	light_yellow_mask = cv2.dilate(light_yellow_mask, kernal)
	res_light_yellow = cv2.bitwise_and(imageFrame, imageFrame,mask = light_yellow_mask)

	# For pista color
	pista_mask = cv2.dilate(pista_mask, kernal)
	res_pista = cv2.bitwise_and(imageFrame, imageFrame,mask = pista_mask)
   

	# Creating contour to track red color
	contours, hierarchy = cv2.findContours(red_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  
	for pic, contour in enumerate(contours):
		area = cv2.contourArea(contour)
		if(area > 400):
			x, y, w, h = cv2.boundingRect(contour)
			imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0,0,0))
			cv2.putText(imageFrame, "Inadequate", (x, y+28),cv2.FONT_HERSHEY_DUPLEX, 1.0,(0,0,0))

  
	# Creating contour to track green color 
	contours, hierarchy = cv2.findContours(green_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  
	for pic, contour in enumerate(contours):
		area = cv2.contourArea(contour)
		if(area > 400):
			x, y, w, h = cv2.boundingRect(contour)
			imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0,0,0))  
			cv2.putText(imageFrame, "Good", (x, y+28),cv2.FONT_HERSHEY_DUPLEX,1.0, (0,0,0))
  

	# Creating contour to track blue color
	contours, hierarchy = cv2.findContours(blue_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

	for pic, contour in enumerate(contours):
		area = cv2.contourArea(contour)
		if(area > 4000):
			x, y, w, h = cv2.boundingRect(contour)
			imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0,0,0))  
			cv2.putText(imageFrame, "Excellent", (x, y+28),cv2.FONT_HERSHEY_DUPLEX,1.0, (0,0,0))


	# Creating contour to track yellow color
	contours, hierarchy = cv2.findContours(yellow_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

	for pic, contour in enumerate(contours):
		area = cv2.contourArea(contour)
		if(area > 400):
			x, y, w, h = cv2.boundingRect(contour)
			imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0,0,0))  
			cv2.putText(imageFrame, "Adequate", (x, y+28),cv2.FONT_HERSHEY_DUPLEX,1.0, (0,0,0))


	# Creating contour to track light_yellow color
	contours, hierarchy = cv2.findContours(light_yellow_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

	for pic, contour in enumerate(contours):
		area = cv2.contourArea(contour)
		if(area > 400):
			x, y, w, h = cv2.boundingRect(contour)
			imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0,0,0))  
			cv2.putText(imageFrame, "Adequate", (x, y+28),cv2.FONT_HERSHEY_DUPLEX,1.0, (0,0,0))


	# Creating contour to track pista color
	contours, hierarchy = cv2.findContours(pista_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  
	for pic, contour in enumerate(contours):
		area = cv2.contourArea(contour)
		if(area > 400):
			x, y, w, h = cv2.boundingRect(contour)
			imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0,0,0))
			cv2.putText(imageFrame, "N_Adeq", (x, y+28),cv2.FONT_HERSHEY_DUPLEX, 1.0,(0,0,0))

  
	# Program Termination
	cv2.imwrite(image, imageFrame)
	#cv2.imwrite("/dbfs/mnt/aria/notebook_images_protection_section/1.png", imageFrame)

In [0]:
storage_account_name = "publicdo"
storage_account_key = "tbmN7Q1Kcd5ePx72c1rI4lRi90RNjSuD2i7Fjkm46Ec6dAllOQ3j5U2JoaLpd12pcqTan5acnDITKaILMkvBvQ=="
container = "aria"

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

try:
  dbutils.fs.mount(
  source = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net",
  mount_point = "/mnt/Risk Report UK CI Project",
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
)
except:
  print("Already Mounted")
#""/dbfs/mnt/aria/Risk/output/408196_AIG_Bilkent_Hospital_Ankara__Turkey_Cankaya_District_Uni_11Oct19_COPE_RFS_111243_1_redacted-3.csv""

Already Mounted

In [0]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# from matplotlib import rcParams

# %matplotlib inline

# #figure size in inches optional
# rcParams['figure.figsize'] = 200, 200

# #read images
# converted_local = mpimg.imread(Images_list[1])
# converted_databricks = mpimg.imread(Images_list[2])

# #display images
# fig, ax = plt.subplots(1,2)
# ax[0].title.set_text("Hi")
# ax[0].imshow(converted_local)
# ax[1].title.set_text("Hi")
# ax[1].imshow(converted_databricks)

In [0]:
import json
import pandas as pd
import csv
import re
print("----------------------- IMAGE_PATH : ", IMAGE_PATH)
# Reading the data received from the JSON file 
def frame_creation(name):
    with open(IMAGE_PATH+'data_protection.txt') as json_file:
        data = json.load(json_file)

    # Position of the data inside the dicsnory for table 
    content = data['analyzeResult']['pageResults'][0]['tables']
    
    tables=[]
    df_protection = []

    for table_idx, table in enumerate(content):
       cells = []
       tmp_cells = [] 
       prev_index = 0
       # Getting total number of columns
       columns = table['columns']
       #  Getting total number of rows 
       rows = table['rows']
       # Creating a list with same length as total column number with empty string  
       tmp_cells = [''] *  (columns)
       for cell in table['cells']:
           if prev_index == cell['rowIndex']:
               tmp_cells[cell['columnIndex']] = cell['text']
           else:
               cells.append(tmp_cells)
               # Creating a list with same length as total row number with empty string
               tmp_cells = [''] *  (columns)
               tmp_cells[cell['columnIndex']] = cell['text']
               prev_index = cell['rowIndex']
       cells.append(tmp_cells)
       tables.append(cells)
       df = pd.DataFrame(cells)
       df_protection.append(df)
       #print("df : ", df)
       #print("df_protection : ", df_protection)
    return df_protection

----------------------- IMAGE_PATH : /dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/Logo_377955 AIG_redactedEET2VR6H7Y/

In [0]:
import re


# from fuzzywuzzy import fuzz


def convert_To_String(l):
    s = ""
    for ll in l:
        s = s + ''.join(ll) + " "
    return s


def organization_finder(text, public_keyword, private_keyword, volunteer_keyword, paid_unpaid_keyword, full_keyword):
    text = text.lower()
    print("text : ", text)

    result = {}
    result_org = []

    public_flag = False
    private_flag = False
    volunteer_flag = False
    paid_flag = False
    full_flag = False
    fire_only_flag = False
    municipal_flag = False

    # Public fire department All scenarios
    if any(public_key in text for public_key in public_keyword):
        public_flag = True
        result["public_flag"] = public_flag

    # volunteer all scenarios
    if any(volunteer_key in text for volunteer_key in volunteer_keyword):
        volunteer_flag = True
        result["volunteer_flag"] = volunteer_flag

    # full-time all scenarios
    if any(full_key in text for full_key in full_keyword):
        full_flag = True
        result["full_flag"] = full_flag

    # paid all scenarios
    if any(paid_key in text for paid_key in paid_keyword):
        paid_flag = True
        result["paid_flag"] = paid_flag

    # municipal all scenarios
    if any(municipal_key in text for municipal_key in municipal_keyword):
        municipal_flag = True
        result["municipal_flag"] = municipal_flag

    # fire only all scenarios
    if any(fire_only_key in text for fire_only_key in fire_only_keyword):
        fire_only_flag = True
        result["fire_only_flag"] = fire_only_flag

    if public_flag is True:
        if full_flag is True:
            result_org.append("Public fire brigade Full-time")
            return result, result_org
        if volunteer_flag is True:
            result_org.append("Public fire brigade Volunteer")
            return result, result_org
        if paid_flag is True:
            result_org.append("Public fire brigade Paid")
            return result, result_org
        if fire_only_flag is True:
            result_org.append("Public fire brigade")
            return result, result_org
    elif private_flag is True:
        result_org.append()
        return result, result_org
    else:
        if municipal_flag is True:
            if full_flag is True:
                result_org.append("Public fire brigade Full-time")
                return result, result_org
            if volunteer_flag is True:
                result_org.append("Public fire brigade Volunteer")
                return result, result_org
            if paid_flag is True:
                result_org.append("Public fire brigade Paid")
                return result, result_org
            else:
                result_org.append("Public fire brigade")
                return result, result_org
        elif fire_only_flag is True:
            print("fire_only_flag : ", fire_only_flag)
            if full_flag is True:
                result_org.append("Public fire brigade Full-time")
                return result, result_org
            if volunteer_flag is True:
                result_org.append("Public fire brigade Volunteer")
                return result, result_org
            if paid_flag is True:
                result_org.append("Public fire brigade Paid")
                return result, result_org
            else:
                result_org.append("Public fire brigade")
                print("result, result_org :", result, result_org)
                return result, result_org
        else:
            if full_flag is True:
                result_org.append("Public fire brigade Full-time")
                return result, result_org
            if paid_flag is True:
                result_org.append("Public fire brigade Paid")
                return result, result_org
            print()
    # print("result, result_org :", result, result_org)
    return result, result_org


# kilometres
def score_calculation_protection_fire(input_data):
    pattern_distance_1 = "\(*([0-9]+\.?\/?[0-9]*)\)*\-*\s*(km|kms|Km|kms|kilometer|kilometers|Kilometer|kilometre|kilometres|Kilometers|miles|mile|meters|meter|m\.?\s+|m\.?$)"
    pattern_responce_1 = "\(*([0-9]+\.?[0-9]*)\)*\-*\s*(min|minute|minutes|hours|hour|second|seconds)"

    near_close_keyword = ["nearest", "closest"]
    ignore_near_close_keyword = ["next closest", "other closest", "next nearest", "other nearest", "second nearest",
                                 "second closest", "several within", "second station"]

    rex = r'|'.join(["one", "two", "\d+", "three", "four", "five", "six", "seven", "eight", "nine","ten"])
    dics_word_distance = {"one": '1.0', "two": '2.0', "three": '3.0', "four": '4.0', "five": '5.0', "six": '6.0',
                          "seven": '7.0', "eight": '8.0', "nine": '9.0',"ten" : '10.0'}

    distance = []
    distance_near = []
    responce = []
    temp_distance = []
    temp_responce = []

    return_responce = ""
    return_distance = ""

    km_flag = 0
    mile_flag = 0

    input = input_data.lower()
    input_specific = input.split('. ')
    # print(input_specific)
    input_data = input_data.split(' ')
    if len(input_data) == 0:
        return distance, responce
    # print(input_data)

    for input in input_specific:

        distance_re = re.compile('\s({})\.?(\d*)\s*(miles|km|m|kilometre|kilometres)(?:\s|\.|\n)'.format(rex))
        distances = distance_re.findall(input)
        # print("-----------distances----------", distances)

        if any(near_close_key in input for near_close_key in near_close_keyword):
            if any(ignore_near_close_keyword_key in input for ignore_near_close_keyword_key in
                   ignore_near_close_keyword):
                pass
            else:
                distance.append("nearest")

        for result_d in distances:
            result_d = list(result_d)
            if "one" in result_d:
                distance.append(dics_word_distance["one"])
            elif "two" in result_d:
                distance.append(dics_word_distance["two"])
            elif "three" in result_d:
                distance.append(dics_word_distance["three"])
            elif "four" in result_d:
                distance.append(dics_word_distance["four"])
            elif "five" in result_d:
                distance.append(dics_word_distance["five"])
            elif "six" in result_d:
                distance.append(dics_word_distance["six"])
            elif "seven" in result_d:
                distance.append(dics_word_distance["seven"])
            elif "eight" in result_d:
                distance.append(dics_word_distance["eight"])
            elif "nine" in result_d:
                distance.append(dics_word_distance["nine"])
            elif "ten" in result_d:
                distance.append(dics_word_distance["ten"])
            else:
                pass  # print("No dics number found")

            if (("mile" in result_d)) or (("miles" in result_d)):
                mile_flag = mile_flag + 1
            # distance.append(result_d[0])#kilometres
            if (("km" in result_d)) or (("kilometer" in result_d)) or (("kms" in result_d)) or (
                    ("kilometers" in result_d) or ("kilometres" in result_d) or (("kilometre" in result_d))):
                km_flag = km_flag + 1

        # print("--------Distance----------", distance)

        result_distance = re.findall(pattern_distance_1, input)
        result_responce = re.findall(pattern_responce_1, input)
        # print("result_distance : ", result_distance)
        for result_d in result_distance:
            result_d = list(result_d)
            # print("result_d : ", result_d)

            if (("mile" in result_d)) or (("miles" in result_d)):
                mile_flag = mile_flag + 1
                distance.append(result_d[0])

            if (("km" in result_d)) or (("kilometer" in result_d)) or (("kms" in result_d)) or (
                    ("kilometers" in result_d) or ("kilometre" in result_d) or ("kilometres" in result_d)):
                km_flag = km_flag + 1
                distance.append(result_d[0])

            result_d[1] = re.sub('\s*', '', result_d[1])

            if (("m" in result_d)) or (("meter" in result_d)):
                km_flag = km_flag + 1
                m_to_km = str(int(result_d[0]) * 0.001)
                distance.append(m_to_km)

        for result_r in result_responce:
            result_r = list(result_r)
            if (("hour" in result_r)) or (("hours" in result_r)):
                hour_to_minute = str(int(result_r[0]) * 60)
                responce.append(hour_to_minute)
            elif (("second" in result_r)) or (("seconds" in result_r)):
                second_to_minute = str(int(result_r[0]) / 60)
                responce.append(second_to_minute)
            else:
                responce.append(result_r[0])

    if mile_flag > km_flag:
        if "Mile" not in distance:
            distance.append("Mile")
    if mile_flag < km_flag:
        if "Km" not in distance:
            distance.append("Km")

        # print("mile_flag : ",mile_flag)
    # print("km_flag : ",km_flag)
    # print("distance : ",distance)
    # print("responce : ",responce)

    if len(responce) == 0 and len(distance) == 0:
        return responce, distance

    try:
        if "nearest" in distance:
            pos = distance.index("nearest")
            temp_distance.append(float(distance[pos + 1]))
            if "Km" in distance:
                temp_distance.append("Km")
            if "Mile" in distance:
                temp_distance.append("Mile")
        else:
            for d in distance:
                if "Mile" not in d and "Km" not in d and "nearest" not in d and "closest" not in d:
                    if d == "1/2":
                        d = "0.5"
                    temp_distance.append(float(d))
                if "Mile" in d:
                    temp_distance.append("Mile")
                if "Km" in d:
                    temp_distance.append("Km")
    except:
        pass

    for r in responce:
        if "Minute" not in r:
            temp_responce.append(float(r))
        if "Minute" in r:
            temp_responce.append("Minute")

    if len(temp_distance) > 0:
        print("temp_distance : ", temp_distance, "--------------", min(temp_distance[:-1]))
        print("temp_responce : ", temp_responce)

    if len(responce) == 0:
        min_distance = min(temp_distance[:-1])
        responce_time_calculated = ""
        if "Mile" in distance:
            # print("Here 1")
            # temp_responce.append(str(round(min_distance)) * 3)
            responce_time_calculated = round(min_distance) * 3
            responce_time_calculated = str(responce_time_calculated) + " Minute"
            min_distance = str(min_distance) + " Mile"
            return responce_time_calculated, min_distance
        else:
            # print("Here 2")
            # temp_responce.append(str(round(min_distance)) * 2)
            responce_time_calculated = round(min_distance) * 2
            responce_time_calculated = str(responce_time_calculated) + " Minute"
            min_distance = str(min_distance) + " Km"
            return responce_time_calculated, min_distance

    # if "nearest" in distance or "closest" in distance:
    #	return responce[1], distance[1]
    # print("temp_distance : ",temp_distance)
    temp_responce.append('Minute')
    # print("temp_responce : ",temp_responce)

    if len(temp_responce) > 0:
        t_unit = temp_responce[len(temp_responce) - 1]
        return_responce = str(min(temp_responce[:-1])) + " " + t_unit

    if len(temp_distance) > 0:
        d_unit = temp_distance[len(temp_distance) - 1]
        return_distance = str(min(temp_distance[:-1])) + " " + d_unit

    return return_responce, return_distance

    # public_keyword = ["public fire department", "public fire brigade", "public fire-brigade", "public fire-department", "local authority"]
    # private_keyword = []
    # municipal_keyword = ["municipal"]
    # volunteer_keyword = ["volunteer", "low resources", "volunteer fire department", "volunteer fire service"]
    # paid_keyword = ["paid"]
    # full_keyword = ["manned 24/7", "full time", "full-time", "24/7", "7/24", "permanently manned"]
    # fire_only_keyword = ["fire brigade", "fire-brigade", "fire fighters", "fire fighter", "fire station", "fire department", "fire-department", "station", "fire service"]

    """
input_text = ""the fully attended and paid fire service is located approximately 5 minutes from the industrial estate. the fire service is indicated to undertake routine walk throughs of the estate and associated properties.""

input_text = input_text.lower()

result_dics, result_org = organization_finder(input_text, public_keyword, private_keyword, volunteer_keyword, paid_keyword, full_keyword)
print("Result : ", result_dics)
print()
print("Result Final: ", result_org)

print("---------------------Distance-----------------------")
p = score_calculation_protection_fire(input_text)
print("Responce result final : ",p[0])
print("Distance result final : ",p[1])
"""


In [0]:
import pandas as pd 

# Function returns a dataframe in client specific format
def intermidiet_table_fire_brigade(name, data, filename, discription):
	#print(data)
	# Defining the structure of the table
	Description_from_Report = [discription['org'],discription['responce'],discription['distance'],""] 
	Components = ['Organisation Type', 'Response Time', 'Distance from Risk', 'Rating (Present in report)']
	Title = ["", "Fire", "Department", ""]
	
	temp = []

	temp.append(data['Public Fire Department'])
	temp.append(data['fire_chubb_score'])
	temp.append(data['distance'])
	if data['responce'] == "Minute":
		temp.append("")
	else:
		temp.append(data['responce'])
	temp.append(data['organization_type'])

	Output = [temp[4],temp[3],str(temp[2]),temp[0]]
	#print(temp)
	Chubb_Fire_Brigade_Rating = [temp[1],"","",""]

	# Creating a dataframe after zipping all the list we defined for table structure with the corresponding column names 
	df= pd.DataFrame(list(zip(Title, Components, Description_from_Report, Output, Chubb_Fire_Brigade_Rating)), columns =['','Components', 'Description_from_Report', 'Output', 'Chubb_Fire_Brigade_Rating'])
	#df.to_csv(filename+".csv")
	#print(df)
	return df

"""
color_detection_result :  {'Detection': 'Good', 'Detection Installed': 'Yes', 'Coverage Provided': '100%', 'Extra coverage required': '', 'Coverage is adequate': '', 'Detection Rating': ''}
color_fire_result :  {'Public Fire Department': 'Good', 'fire_chubb_score': '', 'distance': '5.2 Mile', 'responce': '15 Minute', 'organization_type': 'Public fire brigade'}
color_sprinkler_result :  {'Sprinklers': 'Good', 'Sprinklers Installed': 'Yes', 'Coverage Provided': '97.8 %', 'Extra coverage required': '2.2 %', 'Coverage is adequate': 'Not Adequate', 'Sprinkler Rating': 'AA'}
color_water_result :  {'Water Supply': 'N_Adeq', 'Class for NAICS': '', 'Water Supplied': 'Yes', 'Water Supply Flow': '11340.0 lpm', 'Water Supply Rating': ''}
"""

#color_water_result :  {'Water Supply': 'N_Adeq', 'Class for NAICS': '', 'Water Supplied': 'Yes', 'Water Supply Flow': '11340.0 lpm', 'Water Supply Rating': ''}
# Function returns a dataframe in client specific format
def intermidiet_table_water_supply(name, data, filename, discription):
	#print(data)
	# Defining the structure of the table
	Description_from_Report = ["","",discription['flow'],""]
	Components = ['Class for NAICS', 'Water Supplied', 'Water Supply Flow', 'Water Supply']
	Title = ["", "Water", "Supply", ""]
	
	temp = []

	temp.append(data['Class for NAICS'])
	temp.append(data['Water Supplied'])
	temp.append(data['Water Supply Flow'])
	temp.append(data['Water Supply'])
	temp.append(data['Water Supply Rating'])

	Output = [temp[0],temp[1],str(temp[2]),temp[3]]
	#print(temp)
	Water_Supply_Rating = [temp[4],"","",""]

	# Creating a dataframe after zipping all the list we defined for table structure with the corresponding column names 
	df= pd.DataFrame(list(zip(Title, Components, Description_from_Report, Output, Water_Supply_Rating)), columns =['','Components', 'Description_from_Report', 'Output', 'Water Supply Rating'])
	#df.to_csv(filename+".csv")
	#print(df)
	return df

#data = {'Water Supply': 'N_Adeq', 'Class for NAICS': '', 'Water Supplied': 'Yes', 'Water Supply Flow': '11340.0 lpm', 'Water Supply Rating': ''}
#df_water = intermidiet_table_water_supply("name", data, "filename")
#print(df_water)

def df_to_json_Conversion_water_supply(df):
  Outer_dict={}
  internal_list=[]
  Internal_dict={}
  Internal_dict['Output']=list(df.iloc[:,3])
  Internal_dict['Water Supply Rating']=list(df.iloc[:,4])
  internal_list.append(Internal_dict)
  Outer_dict['Water_Supply']=internal_list
  #print(Outer_dict)
  return Outer_dict

#df_json_water = df_to_json_Conversion_water_supply(df_water)
#print(df_json_water)

#"result": "{\"protection\": [{\"Output\": [\"\", \"\", \"\", \"\"], \"Chubb_Fire_Brigade_Rating\": [\"Public fire brigade Paid\", \"10 Minute\", \"4 Km\", \"Good\"]}]}"



import pandas as pd 

#color_sprinkler_result :  {'Sprinklers': 'Good', 'Sprinklers Installed': 'Yes', 'Coverage Provided': '97.8 %', 'Extra coverage required': '2.2 %', 'Coverage is adequate': 'Not Adequate', 'Sprinkler Rating': 'AA'}
# Function returns a dataframe in client specific format
def intermidiet_table_sprinkler(name, data, filename, discription):
	#print(data)
	# Defining the structure of the table
	Description_from_Report = [discription['install'],"","","",""]
	Components = ['Sprinklers Installed', 'Coverage Provided', 'Extra coverage required', 'Coverage is adequate', 'Sprinklers']
	Title = ["", "Sprinkler", "System", "",""]
	
	temp = []

	temp.append(data['Sprinklers Installed'])
	temp.append(data['Coverage Provided'])
	temp.append(data['Extra coverage required'])
	temp.append(data['Coverage is adequate'])
	temp.append(data['Sprinklers'])
	temp.append(data['Sprinkler Rating'])

	Output = [temp[0],temp[1],str(temp[2]),temp[3],temp[4]]
	Sprinkler_Rating = [temp[5],"","","",""]
	#print(temp)

	# Creating a dataframe after zipping all the list we defined for table structure with the corresponding column names 
	df= pd.DataFrame(list(zip(Title, Components, Description_from_Report, Output, Sprinkler_Rating)), columns =['','Components', 'Description_from_Report', 'Output', 'Sprinkler Rating'])
	#df.to_csv(filename+".csv")
	#print(df)
	return df



#data_sprinkler = {'Sprinklers': 'Good', 'Sprinklers Installed': 'Yes', 'Coverage Provided': '97.8 %', 'Extra coverage required': '2.2 %', 'Coverage is adequate': 'Not Adequate', 'Sprinkler Rating': 'AA'}
#df_sprinkler = intermidiet_table_sprinkler("name", data_sprinkler, "filename")
#print(df_sprinkler)



def df_to_json_Conversion_sprinkler(df):
  Outer_dict={}
  internal_list=[]
  Internal_dict={}
  Internal_dict['Output']=list(df.iloc[:,3])
  Internal_dict['Sprinkler Rating']=list(df.iloc[:,4])
  internal_list.append(Internal_dict)
  Outer_dict['Sprinkler']=internal_list
  #print(Outer_dict)
  return Outer_dict



#df_json_sprinkler = df_to_json_Conversion_sprinkler(df_sprinkler)
#print(df_json_sprinkler)



import pandas as pd 

#color_detection_result :  {'Detection': 'Good', 'Detection Installed': 'Yes', 'Coverage Provided': '100%', 'Extra coverage required': '', 'Coverage is adequate': '', 'Detection Rating': ''}
# Function returns a dataframe in client specific format
def intermidiet_table_detection(name, data, filename, discription):
	#print(data)
	# Defining the structure of the table
	Description_from_Report = [discription['install'],"","","",""]
	Components = ['Detection Installed', 'Coverage Provided', 'Extra coverage required', 'Coverage is adequate', 'Detection']
	Title = ["", "Fire", "Detection", "",""]
	
	temp = []

	temp.append(data['Detection Installed'])
	temp.append(data['Coverage Provided'])
	temp.append(data['Extra coverage required'])
	temp.append(data['Coverage is adequate'])
	temp.append(data['Detection'])
	temp.append(data['Detection Rating'])

	Output = [temp[0],temp[1],str(temp[2]),temp[3],temp[4]]
	Chubb_Detection_Rating = [temp[5],"","","",""]
	#print(temp)
	# Creating a dataframe after zipping all the list we defined for table structure with the corresponding column names 
	df= pd.DataFrame(list(zip(Title, Components, Description_from_Report, Output, Chubb_Detection_Rating)), columns =['','Components', 'Description_from_Report', 'Output', 'Chubb_Detection_Rating'])
	#df.to_csv(filename+".csv")
	#print(df)
	return df


#data = {'Detection': 'Good', 'Detection Installed': 'Yes', 'Coverage Provided': '100%', 'Extra coverage required': '', 'Coverage is adequate': '', 'Detection Rating': ''}
#df = intermidiet_table_detection("name", data, "filename")
#print(df)


def df_to_json_Conversion_detection(df):
  Outer_dict={}
  internal_list=[]
  Internal_dict={}
  Internal_dict['Output']=list(df.iloc[:,3])
  Internal_dict['Detection Rating']=list(df.iloc[:,4])
  internal_list.append(Internal_dict)
  Outer_dict['Fire Detection']=internal_list
  #print(Outer_dict)
  return Outer_dict



#df_json = df_to_json_Conversion_detection(df)
#print(df_json)





In [0]:
import cv2
import json
from requests import get, post
import time
import os

def frame_creation_mask(image):
	# Endpoint URL
	endpoint = "https://na-aria-formrecognizer.cognitiveservices.azure.com/"
	apim_key = "581924cde7974bd68052fdd925dcdd1f"
	#model_id = "<model_id>"
	post_url = endpoint + "/formrecognizer/v2.1-preview.3/layout/analyze?%s" 
	source = image
	params = {
	    "includeTextDetails": True
	}

	headers = {
	    # Request headers
	   'Content-Type': 'image/png',
	    'Ocp-Apim-Subscription-Key': apim_key,
	}
	with open(source, "rb") as f:
		data_bytes = f.read()

	try:
		resp = post(url = post_url, data = data_bytes, headers = headers, params = params)
		if resp.status_code != 202:
			pass#print("POST analyze failed:\n%s" % json.dumps(resp.json()))
			#quit()
		#print("POST analyze succeeded:\n%s" % resp.headers)
		get_url = resp.headers["operation-location"]
		request_id = resp.headers["apim-request-id"]
    
	except Exception as e:
		print("POST analyze failed:\n%s" % str(e))
		#quit() 
    
    
	n_tries = 15
	n_try = 0
	wait_sec = 5
	max_wait_sec = 60
	while n_try < n_tries:
		try:
			resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
			resp_json = resp.json()
			if resp.status_code != 200:
				#print("GET analyze results failed:\n%s" % json.dumps(resp_json))
				#quit()
				pass
			status = resp_json["status"]
			if status == "succeeded":
				#print("Here 1")
				#print("Analysis succeeded:\n%s" % json.dumps(resp_json))
				with open('data.txt', 'w') as outfile:
					json.dump(resp_json, outfile)
					break
					#quit()
			if status == "failed":
				pass#print("Analysis failed:\n%s" % json.dumps(resp_json))
				#quit()
				# Analysis still running. Wait and retry.
			time.sleep(wait_sec)
			n_try += 1
			wait_sec = min(2*wait_sec, max_wait_sec)     
		except Exception as e:
			msg = "GET analyze results failed:\n%s" % str(e)
			print(msg)
			#quit()
			print("Analyze operation did not complete within the allocated time.")



# Reading the data received from the JSON file 
def maskImage(image_File):
    #print("Here 2")
    with open(IMAGE_PATH+'data_protection.txt') as json_file:
        data = json.load(json_file)
   
    # Position of the data inside the dicsnory for table 
    content = data['analyzeResult']['pageResults'][0]['tables']

    tables=[]

    for table_idx, table in enumerate(content):
       #print(table['boundingBox'])
       #print(len(content))
       read_image = cv2.imread(image_File)
       cv2.rectangle(read_image, (table['boundingBox'][0],table['boundingBox'][1]), (table['boundingBox'][4],table['boundingBox'][5]), (255, 255, 255), -1)
       cv2.imwrite(image_File,read_image)

In [0]:
import re

def text_protection(data):

	# Best logic and considered as final
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Public Fire Department' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				#if ('Hosereels' in data[count]) or ('Special Extinguising' in data[count]) or ('Sprinklers' in data[count]):
				if (fire_flag == True):
					#print(data[count])
					if (('Public Fire Department' in data[count]) or ('Page' in data[count]) or ('Version' in data[count])):
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					#print(data[count])                  #Special Extinguishing Protection
					if (('Hosereels' in data[count]) or ('Special Extinguishing' in data[count]) or ('Sprinklers' in data[count])) and (fire_flag == True):
						print("No Append")
						return result, result_list
					#print(data[count])
					if ('WATER SUPPLY' in data[count]) or ('Water Supply' in data[count]) or ('SUPPLY' in data[count]) : 
						water_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

In [0]:
import traceback
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import datetime
import os

import re

#from text_processing_fire_protection import text_protection

def text_identification(file):
	line_data = [] 
	word_data = []
	data = {}
	path_to_sample_forms = file

	endpoint = "https://na-aria-formrecognizer.cognitiveservices.azure.com/"
	key = "581924cde7974bd68052fdd925dcdd1f"

	form_recognizer_client = FormRecognizerClient(endpoint=endpoint, credential=AzureKeyCredential(key))
	with open(path_to_sample_forms, "rb") as f:
		poller = form_recognizer_client.begin_recognize_content(form=f)
	form_pages = poller.result()

	for idx, content in enumerate(form_pages):
		for line_idx, line in enumerate(content.lines):
			#if line.text not in line_data:
			line_data.append(line.text)
				#print("Line Data : ", line.text)
			#for word in line.words:
				#if word not in word_data:
				#	word_data.append(word.text)
			#	print("Word Data : ", word.text)

		for table_idx, table in enumerate(content.tables):
			#print(type(table))

			for cell in table.cells:
				#print("Table-Cell : ",cell.text)
				#print("cell.bounding_box : ,", cell.bounding_box)
				word_data.append(cell.text)
				#if cell.text not in temp:
				#	temp.append(cell.text)


	data['word'] = word_data
	data['line'] = line_data
	return data

In [0]:
# ----------------------------------- Below is Version 0 code -----------------------------------------------------------------
# import re

# from fuzzywuzzy import fuzz

# # color_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
# def final_rating_calculation(color_fire_result):
#     # print(color_fire_result)
#     if "-" in color_fire_result['responce']:
#         color_fire_result['responce'] = color_fire_result['responce'].split('-')[0] + " Minute"
#     print(" ------------------------- ", color_fire_result)
#     # Calculation on the distance
#     if color_fire_result['fire_chubb_score'] == '':
#         if (color_fire_result['distance'] != '' and color_fire_result['responce'] != ''):
#             if 'Mile' in color_fire_result['distance']:
#                 distance = re.sub(' Mile', '', color_fire_result['distance'])
#             else:
#                 distance = re.sub(' Km', '', color_fire_result['distance'])

#             if (float(distance) >= 0.1) and (float(distance) <= 0.5):
#                 distance = 1.0
#             distance = round(float(distance))

#             print("Distance ----------- 1 ", distance, color_fire_result['organization_type'])
#             if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
#                 if ((color_fire_result['organization_type'] == 'Public fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 1) and (distance <= 3)):
#                             color_fire_result['fire_chubb_score'] = 'AAA'
#                     else:
#                         if ((distance >= 1) and (distance <= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AAA'

#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 4) and (distance <= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                         if ((distance >= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         if ((distance >= 6) and (distance <= 7)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                         elif ((distance >= 8) and (distance <= 10)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                         else:
#                             print("Rating can't be done as per the current rules")
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")

#     if color_fire_result['fire_chubb_score'] == '':
#         if (color_fire_result['distance'] != '' and color_fire_result['responce'] != ''):
#             if 'Mile' in color_fire_result['distance']:
#                 distance = re.sub(' Mile', '', color_fire_result['distance'])
#             else:
#                 distance = re.sub(' Km', '', color_fire_result['distance'])

#             if (float(distance) >= 0.1) and (float(distance) <= 0.5):
#                 distance = 1.0
#             distance = round(float(distance))

#             print("Distance ---------------- 2 ", distance, color_fire_result['organization_type'])
#             if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
#                 if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
#                     if 'Mile' in color_fire_result['distance']:
#                         if (distance <= 5):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         if (distance <= 5):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 6) and (distance <= 7)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     else:
#                         if ((distance >= 8) and (distance <= 10)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 8)):
#                             color_fire_result['fire_chubb_score'] = 'SD'
#                     else:
#                         if ((distance >= 11)):
#                             color_fire_result['fire_chubb_score'] = 'SD'
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")

#             if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
#                 if ((color_fire_result[
#                          'organization_type'] == 'Private fire brigade Paid')):  # or (color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 1) and (distance <= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         if ((distance >= 1) and (distance <= 7)):
#                             color_fire_result['fire_chubb_score'] = 'AA'

#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 6)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     else:
#                         if ((distance >= 8)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")
#             """
#             if (color_fire_result['organization_type'] != ''): #Private fire brigade Full-time
#                 if ((color_fire_result['organization_type'] == 'Public fire brigade')):
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 1) and (distance <= 3)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                         else:
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     else:
#                         if ((distance >= 1) and (distance <= 5)):
#                                 color_fire_result['fire_chubb_score'] = 'AA'
#                         else:
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")
#             """

#         # Calculation on responce time
#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if ((color_fire_result['organization_type'] == 'Public fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if ((responce >= 1) and (responce <= 10)):
#                         color_fire_result['fire_chubb_score'] = 'AAA'
#                     if ((responce >= 11) and (responce <= 15)):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     print("Here 2")
#             else:
#                 print("No data for organization_type !")

#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
#                     if (responce <= 15):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     if ((responce >= 16) and (responce <= 20)):
#                         color_fire_result['fire_chubb_score'] = 'A'
#                     if (responce >= 21):
#                         color_fire_result['fire_chubb_score'] = 'SD'
#                     print("Here 2")
#             else:
#                 print("No data for organization_type !")

#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if ((color_fire_result['organization_type'] == 'Private fire brigade Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if ((responce >= 1) and (responce <= 10)):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     print("Here 2")
#             else:
#                 print("No data for organization_type !")
#         """
#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if ((color_fire_result['organization_type'] == 'Public fire brigade')):
#                     if ((responce >= 1) and (responce <= 15)):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         color_fire_result['fire_chubb_score'] = 'A'
#                     print("Here 2")
#             else:
#                 print("No data for organization_type !")
#         """
#         print("color_fire_result['responce'] ", color_fire_result['responce'])
#         print("color_fire_result['distance'] ", color_fire_result['distance'])
#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] == ''):
#             if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer'):
#                 color_fire_result['fire_chubb_score'] = "AA"
#             else:
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Excellent') >= 80:
#                     color_fire_result['fire_chubb_score'] = "AAA"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Good') >= 80:
#                     color_fire_result['fire_chubb_score'] = "AA"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Adequate') >= 80:
#                     color_fire_result['fire_chubb_score'] = "A"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'N_Adeq') >= 80:
#                     color_fire_result['fire_chubb_score'] = "A"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Inadequate') >= 80:
#                     color_fire_result['fire_chubb_score'] = "SD"

#                 # print(color_fire_result)
#     return color_fire_result

# #color_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'5.0 Mile', 'responce':'20.0 Minute', 'organization_type':'Public fire brigade'}
# #result = final_rating_calculation(color_fire_result)
# #print("result : ",result)


#Below is the new logic to calculate rating based on response time

# ----------------------------------- Below is Version 1 code -----------------------------------------------------------------
import re

from fuzzywuzzy import fuzz

# color_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}

# def final_rating_calculation(color_fire_result):
#     # print(color_fire_result)
#     if "-" in color_fire_result['responce']:
#         color_fire_result['responce'] = color_fire_result['responce'].split('-')[0] + " Minute"
#     print(" ------------------------- ", color_fire_result)
#     # Calculation on the distance
#     if color_fire_result['fire_chubb_score'] == '':
#         """
#         if (color_fire_result['distance'] != '' and color_fire_result['responce'] != ''):
#             if 'Mile' in color_fire_result['distance']:
#                 distance = re.sub(' Mile', '', color_fire_result['distance'])
#             else:
#                 distance = re.sub(' Km', '', color_fire_result['distance'])

#             if (float(distance) >= 0.1) and (float(distance) <= 0.5):
#                 distance = 1.0
#             distance = round(float(distance))

#             print("Distance ----------- 1 ", distance, color_fire_result['organization_type'])
#             if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
#                 if ((color_fire_result['organization_type'] == 'Public fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 1) and (distance <= 3)):
#                             color_fire_result['fire_chubb_score'] = 'AAA'
#                     else:
#                         if ((distance >= 1) and (distance <= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AAA'

#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 4) and (distance <= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                         if ((distance >= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         if ((distance >= 6) and (distance <= 7)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                         elif ((distance >= 8) and (distance <= 10)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                         else:
#                             print("Rating can't be done as per the current rules")
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")

#     if color_fire_result['fire_chubb_score'] == '':
#         if (color_fire_result['distance'] != '' and color_fire_result['responce'] != ''):
#             if 'Mile' in color_fire_result['distance']:
#                 distance = re.sub(' Mile', '', color_fire_result['distance'])
#             else:
#                 distance = re.sub(' Km', '', color_fire_result['distance'])

#             if (float(distance) >= 0.1) and (float(distance) <= 0.5):
#                 distance = 1.0
#             distance = round(float(distance))

#             print("Distance ---------------- 2 ", distance, color_fire_result['organization_type'])
#             if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
#                 if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
#                     if 'Mile' in color_fire_result['distance']:
#                         if (distance <= 5):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         if (distance <= 5):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 6) and (distance <= 7)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     else:
#                         if ((distance >= 8) and (distance <= 10)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 8)):
#                             color_fire_result['fire_chubb_score'] = 'SD'
#                     else:
#                         if ((distance >= 11)):
#                             color_fire_result['fire_chubb_score'] = 'SD'
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")

#             if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
#                 if ((color_fire_result[
#                          'organization_type'] == 'Private fire brigade Paid')):  # or (color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 1) and (distance <= 5)):
#                             color_fire_result['fire_chubb_score'] = 'AA'
#                     else:
#                         if ((distance >= 1) and (distance <= 7)):
#                             color_fire_result['fire_chubb_score'] = 'AA'

#                     if 'Mile' in color_fire_result['distance']:
#                         if ((distance >= 6)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     else:
#                         if ((distance >= 8)):
#                             color_fire_result['fire_chubb_score'] = 'A'
#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")
#         """
#         # Calculation on responce time
#         if (color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if ((color_fire_result['organization_type'] == 'Public fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade') or (
#                         color_fire_result['organization_type'] == 'Public fire brigade Paid')):
#                     if ((responce >= 1) and (responce <= 10)):
#                         color_fire_result['fire_chubb_score'] = 'AAA'
#                     if ((responce >= 11) and (responce <= 15)):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     if ((responce >= 16) and (responce <= 20)):
#                         color_fire_result['fire_chubb_score'] = 'A'
#                     if ((responce >= 21)):
#                         color_fire_result['fire_chubb_score'] = 'SD'

#                     print("Here 1")
#             else:
#                 print("No data for organization_type !")

#         if (color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer') or (
#                         color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
#                     if (responce <= 15) and (color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     if (responce <= 15) and (color_fire_result['organization_type'] == 'Public fire brigade Volunteer'):
#                         color_fire_result['fire_chubb_score'] = 'A'
#                     if ((responce >= 16) and (responce <= 20)):
#                         color_fire_result['fire_chubb_score'] = 'A'
#                     if (responce >= 21):
#                         color_fire_result['fire_chubb_score'] = 'SD'
#                     print("Here 2")
#             else:
#                 print("No data for organization_type !")

#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] != ''):
#             if 'Minute' in color_fire_result['responce']:
#                 responce = re.sub(' Minute', '', color_fire_result['responce'])
#             responce = round(float(responce))
#             print("Responce ", responce)
#             if (color_fire_result['organization_type'] != ''):
#                 if ((color_fire_result['organization_type'] == 'Private fire brigade Paid')):
#                     if ((responce >= 11) and (responce <= 15)):
#                         color_fire_result['fire_chubb_score'] = 'AA'
#                     if ((responce >= 16)):
#                         color_fire_result['fire_chubb_score'] = 'A'
#                     print("Here 2")
#             else:
#                 print("No data for organization_type !")

#         print("color_fire_result['responce'] ", color_fire_result['responce'])
#         print("color_fire_result['distance'] ", color_fire_result['distance'])
#         if (color_fire_result['distance'] == '' and color_fire_result['responce'] == ''):
#             if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer'):
#                 color_fire_result['fire_chubb_score'] = "AA"
#             else:
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Excellent') >= 80:
#                     color_fire_result['fire_chubb_score'] = "AAA"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Good') >= 80:
#                     color_fire_result['fire_chubb_score'] = "AA"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Adequate') >= 80:
#                     color_fire_result['fire_chubb_score'] = "A"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'N_Adeq') >= 80:
#                     color_fire_result['fire_chubb_score'] = "A"
#                 if fuzz.ratio(color_fire_result['Public Fire Department'], 'Inadequate') >= 80:
#                     color_fire_result['fire_chubb_score'] = "SD"

#                 # print(color_fire_result)
#     return color_fire_result

# ----------------------------------- Below is Version 2 code -----------------------------------------------------------------
def final_rating_calculation(color_fire_result):
    # print(color_fire_result)
    if "-" in color_fire_result['responce']:
        color_fire_result['responce'] = color_fire_result['responce'].split('-')[0] + " Minute"
    print(" ------------------------- ", color_fire_result)
    # Calculation on the distance
    if color_fire_result['fire_chubb_score'] == '':
        """
        if (color_fire_result['distance'] != '' and color_fire_result['responce'] != ''):
            if 'Mile' in color_fire_result['distance']:
                distance = re.sub(' Mile', '', color_fire_result['distance'])
            else:
                distance = re.sub(' Km', '', color_fire_result['distance'])

            if (float(distance) >= 0.1) and (float(distance) <= 0.5):
                distance = 1.0
            distance = round(float(distance))

            print("Distance ----------- 1 ", distance, color_fire_result['organization_type'])
            if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
                if ((color_fire_result['organization_type'] == 'Public fire brigade Full-time Paid') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
                        color_fire_result['organization_type'] == 'Public fire brigade Full-time') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Full-time') or (
                        color_fire_result['organization_type'] == 'Public fire brigade') or (
                        color_fire_result['organization_type'] == 'Public fire brigade Paid')):
                    if 'Mile' in color_fire_result['distance']:
                        if ((distance >= 1) and (distance <= 3)):
                            color_fire_result['fire_chubb_score'] = 'AAA'
                    else:
                        if ((distance >= 1) and (distance <= 5)):
                            color_fire_result['fire_chubb_score'] = 'AAA'

                    if 'Mile' in color_fire_result['distance']:
                        if ((distance >= 4) and (distance <= 5)):
                            color_fire_result['fire_chubb_score'] = 'AA'
                        if ((distance >= 5)):
                            color_fire_result['fire_chubb_score'] = 'AA'
                    else:
                        if ((distance >= 6) and (distance <= 7)):
                            color_fire_result['fire_chubb_score'] = 'AA'
                        elif ((distance >= 8) and (distance <= 10)):
                            color_fire_result['fire_chubb_score'] = 'A'
                        else:
                            print("Rating can't be done as per the current rules")
                    print("Here 1")
            else:
                print("No data for organization_type !")

    if color_fire_result['fire_chubb_score'] == '':
        if (color_fire_result['distance'] != '' and color_fire_result['responce'] != ''):
            if 'Mile' in color_fire_result['distance']:
                distance = re.sub(' Mile', '', color_fire_result['distance'])
            else:
                distance = re.sub(' Km', '', color_fire_result['distance'])

            if (float(distance) >= 0.1) and (float(distance) <= 0.5):
                distance = 1.0
            distance = round(float(distance))

            print("Distance ---------------- 2 ", distance, color_fire_result['organization_type'])
            if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
                if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
                    if 'Mile' in color_fire_result['distance']:
                        if (distance <= 5):
                            color_fire_result['fire_chubb_score'] = 'AA'
                    else:
                        if (distance <= 5):
                            color_fire_result['fire_chubb_score'] = 'AA'
                    if 'Mile' in color_fire_result['distance']:
                        if ((distance >= 6) and (distance <= 7)):
                            color_fire_result['fire_chubb_score'] = 'A'
                    else:
                        if ((distance >= 8) and (distance <= 10)):
                            color_fire_result['fire_chubb_score'] = 'A'
                    if 'Mile' in color_fire_result['distance']:
                        if ((distance >= 8)):
                            color_fire_result['fire_chubb_score'] = 'SD'
                    else:
                        if ((distance >= 11)):
                            color_fire_result['fire_chubb_score'] = 'SD'
                    print("Here 1")
            else:
                print("No data for organization_type !")

            if (color_fire_result['organization_type'] != ''):  # Private fire brigade Full-time
                if ((color_fire_result[
                         'organization_type'] == 'Private fire brigade Paid')):  # or (color_fire_result['organization_type'] == 'Public fire brigade Paid')):
                    if 'Mile' in color_fire_result['distance']:
                        if ((distance >= 1) and (distance <= 5)):
                            color_fire_result['fire_chubb_score'] = 'AA'
                    else:
                        if ((distance >= 1) and (distance <= 7)):
                            color_fire_result['fire_chubb_score'] = 'AA'

                    if 'Mile' in color_fire_result['distance']:
                        if ((distance >= 6)):
                            color_fire_result['fire_chubb_score'] = 'A'
                    else:
                        if ((distance >= 8)):
                            color_fire_result['fire_chubb_score'] = 'A'
                    print("Here 1")
            else:
                print("No data for organization_type !")
        """
        # Calculation on responce time
        if (color_fire_result['responce'] != ''):
            if 'Minute' in color_fire_result['responce']:
                responce = re.sub(' Minute', '', color_fire_result['responce'])
            responce = round(float(responce))
            print("Responce ", responce)
            if (color_fire_result['organization_type'] != ''):
                if ((color_fire_result['organization_type'] == 'Public fire brigade Full-time Paid') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
                        color_fire_result['organization_type'] == 'Public fire brigade Full-time') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Full-time') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Full-time Paid') or (
                        color_fire_result['organization_type'] == 'Public fire brigade') or (
                        color_fire_result['organization_type'] == 'Public fire brigade Paid')):
                    if ((responce >= 1) and (responce <= 10)):
                        color_fire_result['fire_chubb_score'] = 'AAA'
                    if ((responce >= 11) and (responce <= 15)):
                        color_fire_result['fire_chubb_score'] = 'AA'
                    if ((responce >= 16) and (responce <= 20)):
                        color_fire_result['fire_chubb_score'] = 'A'
                    if ((responce >= 21)):
                        color_fire_result['fire_chubb_score'] = 'SD'

                    print("Here 1")
            else:
                print("No data for organization_type !")

        if (color_fire_result['responce'] != ''):
            if 'Minute' in color_fire_result['responce']:
                responce = re.sub(' Minute', '', color_fire_result['responce'])
            responce = round(float(responce))
            print("Responce ", responce)
            if (color_fire_result['organization_type'] != ''):
                if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer') or (
                        color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
                    if (responce <= 15) and (color_fire_result['organization_type'] == 'Private fire brigade Volunteer'):
                        color_fire_result['fire_chubb_score'] = 'AA'
                    if (responce <= 15) and (color_fire_result['organization_type'] == 'Public fire brigade Volunteer'):
                        color_fire_result['fire_chubb_score'] = 'AA'    # Today Change
                    if ((responce >= 16) and (responce <= 20)):
                        color_fire_result['fire_chubb_score'] = 'A'
                    if (responce >= 21):
                        color_fire_result['fire_chubb_score'] = 'SD'
                    print("Here 2")
            else:
                print("No data for organization_type !")

        if (color_fire_result['distance'] == '' and color_fire_result['responce'] != ''):
            if 'Minute' in color_fire_result['responce']:
                responce = re.sub(' Minute', '', color_fire_result['responce'])
            responce = round(float(responce))
            print("Responce ", responce)
            if (color_fire_result['organization_type'] != ''):
                if ((color_fire_result['organization_type'] == 'Private fire brigade Paid')):
                    if ((responce >= 11) and (responce <= 15)):
                        color_fire_result['fire_chubb_score'] = 'AA'
                    if ((responce >= 16)):
                        color_fire_result['fire_chubb_score'] = 'A'
                    print("Here 2")
            else:
                print("No data for organization_type !")

        print("color_fire_result['responce'] ", color_fire_result['responce'])
        print("color_fire_result['distance'] ", color_fire_result['distance'])
        if (color_fire_result['distance'] == '' and color_fire_result['responce'] == ''):
            if (color_fire_result['organization_type'] == 'Public fire brigade Volunteer'):
                color_fire_result['fire_chubb_score'] = "AA"
            else:
                if fuzz.ratio(color_fire_result['Public Fire Department'], 'Excellent') >= 80:
                    color_fire_result['fire_chubb_score'] = "AAA"
                if fuzz.ratio(color_fire_result['Public Fire Department'], 'Good') >= 80:
                    color_fire_result['fire_chubb_score'] = "AA"
                if fuzz.ratio(color_fire_result['Public Fire Department'], 'Adequate') >= 80:
                    color_fire_result['fire_chubb_score'] = "A"
                if fuzz.ratio(color_fire_result['Public Fire Department'], 'N_Adeq') >= 80:
                    color_fire_result['fire_chubb_score'] = "A"
                if fuzz.ratio(color_fire_result['Public Fire Department'], 'Inadequate') >= 80:
                    color_fire_result['fire_chubb_score'] = "SD"

                # print(color_fire_result)
    return color_fire_result


#color_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'10.0 Minute', 'organization_type':'Public fire brigade Paid'}
#result = final_rating_calculation(color_fire_result)
#print("result : ",result)


/databricks/python/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
 warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
 ------------------------- {'Public Fire Department': '', 'fire_chubb_score': '', 'distance': '', 'responce': '10.0 Minute', 'organization_type': 'Public fire brigade Paid'}
Responce 10
Here 1
Responce 10
Responce 10
color_fire_result['responce'] 10.0 Minute
color_fire_result['distance'] 
result : {'Public Fire Department': '', 'fire_chubb_score': 'AAA', 'distance': '', 'responce': '10.0 Minute', 'organization_type': 'Public fire brigade Paid'}

In [0]:
def text_sprinkler_AIG(data):

	# Best logic and considered as final
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Automatic Sprinklers' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				#if ('Hosereels' in data[count]) or ('Special Extinguising' in data[count]) or ('Sprinklers' in data[count]):
				if (fire_flag == True):
					#print(data[count])
					if (('Automatic Sprinklers' in data[count]) or ('Page' in data[count]) or ('Version' in data[count])):
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					#print(data[count])                  #Special Extinguishing Protection
					if (('Hosereels' in data[count]) or ('Special Protection' in data[count]) or ('Public Fire Department' in data[count])) and (fire_flag == True):
						print("No Append")
						return result, result_list
					print(data[count])
					if ('WATER SUPPLY' in data[count]) or ('Water Supply' in data[count]) or ('SUPPLY' in data[count]) : 
						water_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

def text_water_AIG(data):

	# Best logic and considered as final
	result = ""	
	result_list = []
	water_flag = False
	supervision_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Water Supply' in data[line] or 'Water Supply, Water Mains, and Fire Hydrants' in data[line]):
			#print(data[line])
			water_flag = True
			count = line
			while count < len(data):
				if (water_flag == True):
					#print(data[count])
					if (('Water Supply' in data[count]) or ('Page' in data[count]) or ('Version' in data[count])):
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					if ('SUPERVISION' in data[count]): 
						supervision_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if supervision_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list


def text_detection_AIG(data):

	# Best logic and considered as final
	result = ""	
	result_list = []
	detection_flag = False
	other_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Alarm Systems' in data[line]):
			#print(data[line])
			detection_flag = True
			count = line
			while count < len(data):
				if (detection_flag == True):
					#print(data[count])
					if (('Alarm Systems' in data[count]) or ('Page' in data[count]) or ('Version' in data[count])):
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					if ('MANAGEMENT PROGRAMS' in data[count] or 'Security Measures' in data[count]): 
						other_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if other_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

In [0]:
import re
# ----------------------------------- Below is Version 1 code -----------------------------------------------------------------
# def final_rating_calculation_fire_sprinkler(color_sprinkler_result):
# 	#print("final_rating_calculation_fire_sprinkler : ", color_detection_result)
# 	try:
# 		temp_sprinkler_cover = float(re.sub("\s*%", '', color_sprinkler_result['Coverage Provided']))
# 	except :
# 		temp_sprinkler_cover = 0
# 	try:
# 		temp_sprinkler_required_cover = float(re.sub("\s*%", '', color_sprinkler_result['Extra coverage required']))
# 	except:
# 		temp_sprinkler_required_cover = 0
# 	print(temp_sprinkler_cover, " ---- ", temp_sprinkler_required_cover)
    
# 	if color_sprinkler_result['Coverage is adequate'] == ' ' or color_sprinkler_result['Coverage is adequate'] == '':
# 		if temp_sprinkler_cover < 74 and temp_sprinkler_cover >= 0:
# 			color_sprinkler_result['Coverage is adequate'] = 'Not Adequate'
# 		elif temp_sprinkler_cover <= 100 and temp_sprinkler_cover >= 75:
# 			color_sprinkler_result['Coverage is adequate'] = 'Adequate'
# 		else:
# 			print("Can't determined")
    
# 	if temp_sprinkler_cover == 100  or 'full' in color_sprinkler_result['Coverage Provided']:
# 		color_sprinkler_result['Sprinkler Rating'] = 'AAA'
# 	elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage Provided'] == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 		color_sprinkler_result['Sprinkler Rating'] = 'AA' 
# 	elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 		color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 	elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage Provided'] == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 		color_sprinkler_result['Sprinkler Rating'] = 'A'
# 	elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 		color_sprinkler_result['Sprinkler Rating'] = 'A' 
# 	elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_required_cover == 0:
# 		color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 	elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_required_cover == 0:
# 		color_sprinkler_result['Sprinkler Rating'] = 'AA'    
# 	elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_required_cover > 0:
# 		color_sprinkler_result['Sprinkler Rating'] = 'A'
# 	elif temp_sprinkler_cover < 100 and temp_sprinkler_cover > 74 and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 		#color_sprinkler_result['Coverage is adequate'] = 'Adequate'
# 		color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 	elif color_sprinkler_result['Sprinklers Installed'] == "Yes":
# 		color_sprinkler_result['Sprinkler Rating'] = 'A'
# 	else:
# 		print("Can't determined 2")
                    
# 	return color_sprinkler_result

# ----------------------------------- Below is Version 2 code -----------------------------------------------------------------

# def final_rating_calculation_fire_sprinkler(color_sprinkler_result, carrier_name):  # Today Change
# 	print("final_rating_calculation_fire_sprinkler 1 : ", color_sprinkler_result)
# 	temp_sprinkler_cover = ""
# 	temp_sprinkler_required_cover = ""
# 	if carrier_name == "AIG":   # Today Change
# 		try:
# 			temp_sprinkler_cover = float(re.sub("\s*%", '', color_sprinkler_result['Coverage Provided']))
# 		except :
# 			temp_sprinkler_cover = 0
# 		try:
# 			temp_sprinkler_required_cover = float(re.sub("\s*%", '', color_sprinkler_result['Extra coverage required']))
# 		except:
# 			temp_sprinkler_required_cover = 0
# 		print(temp_sprinkler_cover, " ---- ", temp_sprinkler_required_cover)
    

# 		if color_sprinkler_result['Coverage is adequate'] == ' ' or color_sprinkler_result['Coverage is adequate'] == '':  # Today Change
# 			if temp_sprinkler_cover < 74 and temp_sprinkler_cover >= 0:  # Today Change
# 				color_sprinkler_result['Coverage is adequate'] = 'Not Adequate'  # Today Change
# 			elif temp_sprinkler_cover <= 100 and temp_sprinkler_cover >= 75:  # Today Change
# 				color_sprinkler_result['Coverage is adequate'] = 'Adequate'  # Today Change
# 			else:  # Today Change
# 				print("Can't determined adequecy for AIG")  # Today Change
    
# 	try:    
# 		if temp_sprinkler_cover == 100  or 'full' in color_sprinkler_result['Coverage Provided']:
# 			color_sprinkler_result['Sprinkler Rating'] = 'AAA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'A' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""): # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""):
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'    
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_required_cover > 0:
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif temp_sprinkler_cover < 100 and temp_sprinkler_cover > 74 and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover <=74 and temp_sprinkler_cover > 0 :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif temp_sprinkler_cover == 0 and temp_sprinkler_required_cover > 0 :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change _ _
# 			color_sprinkler_result['Sprinklers Installed'] = "No"
# 		else:
# 			print("Sprinkler rating can't be dedtermined")
# 	except TypeError:
# 		if 'full' in color_sprinkler_result['Coverage Provided']:
# 			color_sprinkler_result['Sprinkler Rating'] = 'AAA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'A' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""): # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""):
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'    
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		else:
# 			print("Sprinkler rating can't be dedtermined") 
# 	return color_sprinkler_result

# ----------------------------------- Below is Version 3 code -----------------------------------------------------------------

# def final_rating_calculation_fire_sprinkler(color_sprinkler_result, carrier_name):  # Today Change
# 	print("final_rating_calculation_fire_sprinkler 1 : ", color_sprinkler_result)
# 	if color_sprinkler_result['Coverage Provided'] == "":
# 		temp_sprinkler_cover = ""
# 	else:
# 		temp_sprinkler_cover = color_sprinkler_result['Coverage Provided']
# 	if color_sprinkler_result['Extra coverage required'] == "":
# 		temp_sprinkler_required_cover = ""
# 	else:
# 		temp_sprinkler_required_cover = color_sprinkler_result['Extra coverage required']
# 	if carrier_name == "AIG":   # Today Change
# 		try:
# 			temp_sprinkler_cover = float(re.sub("\s*%", '', color_sprinkler_result['Coverage Provided']))
# 		except :
# 			temp_sprinkler_cover = 0
# 		try:
# 			temp_sprinkler_required_cover = float(re.sub("\s*%", '', color_sprinkler_result['Extra coverage required']))
# 		except:
# 			temp_sprinkler_required_cover = 0
# 		print(temp_sprinkler_cover, " ---- ", temp_sprinkler_required_cover)
    

# 		if color_sprinkler_result['Coverage is adequate'] == ' ' or color_sprinkler_result['Coverage is adequate'] == '':  # Today Change
# 			if temp_sprinkler_cover < 74 and temp_sprinkler_cover >= 0:  # Today Change
# 				color_sprinkler_result['Coverage is adequate'] = 'Not Adequate'  # Today Change
# 			elif temp_sprinkler_cover <= 100 and temp_sprinkler_cover >= 75:  # Today Change
# 				color_sprinkler_result['Coverage is adequate'] = 'Adequate'  # Today Change
# 			else:  # Today Change
# 				print("Can't determined adequecy for AIG")  # Today Change
# 	try:    
# 		if temp_sprinkler_cover == 100  or 'full' in color_sprinkler_result['Coverage Provided']:
# 			color_sprinkler_result['Sprinkler Rating'] = 'AAA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""): # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""):
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'    
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_required_cover > 0:
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif temp_sprinkler_cover < 100 and temp_sprinkler_cover > 74 and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover <=74 and temp_sprinkler_cover > 0 :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif temp_sprinkler_cover == 0 and temp_sprinkler_required_cover > 0 :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change _ _
# 			color_sprinkler_result['Sprinklers Installed'] = "No"
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		else:
# 			print("Sprinkler rating can't be dedtermined")
# 	except TypeError:
# 		if 'full' in color_sprinkler_result['Coverage Provided']:
# 			color_sprinkler_result['Sprinkler Rating'] = 'AAA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'A' 
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		#elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :  # Today Change
# 		#	color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""): # Today Change
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""):
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'    
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'AA'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :
# 			color_sprinkler_result['Sprinkler Rating'] = 'A'
# 		else:
# 			print("Sprinkler rating can't be dedtermined")    

# 	return color_sprinkler_result

# ----------------------------------- Below is Version 4 code -----------------------------------------------------------------

def final_rating_calculation_fire_sprinkler(color_sprinkler_result, carrier_name):  # Today Change
	print("final_rating_calculation_fire_sprinkler 1 : ", color_sprinkler_result)
	if color_sprinkler_result['Coverage Provided'] == "":
		temp_sprinkler_cover = ""
	else:
		temp_sprinkler_cover = color_sprinkler_result['Coverage Provided']
	if color_sprinkler_result['Extra coverage required'] == "":
		temp_sprinkler_required_cover = ""
	else:
		temp_sprinkler_required_cover = color_sprinkler_result['Extra coverage required']
	if carrier_name == "AIG":   # Today Change
		try:
			temp_sprinkler_cover = float(re.sub("\s*%", '', color_sprinkler_result['Coverage Provided']))
		except :
			temp_sprinkler_cover = 0
		try:
			temp_sprinkler_required_cover = float(re.sub("\s*%", '', color_sprinkler_result['Extra coverage required']))
		except:
			temp_sprinkler_required_cover = 0
		print(temp_sprinkler_cover, " ---- ", temp_sprinkler_required_cover)


		if color_sprinkler_result['Coverage is adequate'] == ' ' or color_sprinkler_result['Coverage is adequate'] == '':  # Today Change
			if temp_sprinkler_cover < 74 and temp_sprinkler_cover >= 0:  # Today Change
				color_sprinkler_result['Coverage is adequate'] = 'Not Adequate'  # Today Change
			elif temp_sprinkler_cover <= 100 and temp_sprinkler_cover >= 75:  # Today Change
				color_sprinkler_result['Coverage is adequate'] = 'Adequate'  # Today Change
			else:  # Today Change
				print("Can't determined adequecy for AIG")  # Today Change               
	else:    
		try:
			temp_sprinkler_cover = float(re.sub("\s*%", '', color_sprinkler_result['Coverage Provided']))
		except :
			temp_sprinkler_cover = "--"
		try:
			temp_sprinkler_required_cover = float(re.sub("\s*%", '', color_sprinkler_result['Extra coverage required']))
		except:
			temp_sprinkler_required_cover = "--"
            
	try:    
		if temp_sprinkler_cover == 100  or 'full' in color_sprinkler_result['Coverage Provided']:
			color_sprinkler_result['Sprinkler Rating'] = 'AAA'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'AA' 
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "No" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""): # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""):
			color_sprinkler_result['Sprinkler Rating'] = 'AA'    
		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_required_cover > 0:
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		elif temp_sprinkler_cover < 100 and temp_sprinkler_cover > 74 and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'AA'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover <=74 and temp_sprinkler_cover > 0 :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif temp_sprinkler_cover == 0 and temp_sprinkler_required_cover > 0 :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change _ _
			color_sprinkler_result['Sprinklers Installed'] = "No"
		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		else:
			print("Sprinkler rating can't be dedtermined")
	except TypeError:
		if 'full' in color_sprinkler_result['Coverage Provided']:
			color_sprinkler_result['Sprinkler Rating'] = 'AAA'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'AA'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'AA' 
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'A' 
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		#elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :  # Today Change
		#	color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "No" and temp_sprinkler_cover == '' and color_sprinkler_result['Coverage is adequate'] == '' :  # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "No" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""): # Today Change
			color_sprinkler_result['Sprinkler Rating'] = 'A'  # Today Change
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and (temp_sprinkler_required_cover == 0 or temp_sprinkler_required_cover == ""):
			color_sprinkler_result['Sprinkler Rating'] = 'AA'    
		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'AA'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" and color_sprinkler_result['Coverage is adequate'] == 'Not Adequate':
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		elif color_sprinkler_result['Sprinklers Installed'] == "No" :
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		elif color_sprinkler_result['Sprinklers Installed'] == "Yes" :
			color_sprinkler_result['Sprinkler Rating'] = 'A'
		else:
			print("Sprinkler rating can't be dedtermined")    

	return color_sprinkler_result  
  
#color_sprinkler_result =  {'Sprinklers': '', 'Sprinklers Installed': 'Yes', 'Coverage Provided': '', 'Extra coverage required': '', 'Coverage is adequate': 'Adequate', 'Sprinkler Rating': ''}
#result = final_rating_calculation_fire_sprinkler(color_sprinkler_result)
#print(result)

In [0]:
#import re
# ----------------------------------- Below is Version 1 code -----------------------------------------------------------------
# def final_rating_calculation_fire_detection(color_detection_result):
# 	print("final_rating_calculation_fire_detection : ", color_detection_result)
# 	try:
# 		temp_detection_cover = float(re.sub("\s*%", '', color_detection_result['Coverage Provided']))
# 	except :
# 		temp_detection_cover = 0
# 	try:
# 		temp_detection_required_cover = float(re.sub("\s*%", '', color_detection_result['Extra coverage required']))
# 	except:
# 		temp_detection_required_cover = 0
# 	print(temp_detection_cover, " ---- ", temp_detection_required_cover)
    
# 	if color_detection_result['Coverage is adequate'] == '' or color_detection_result['Coverage is adequate'] == ' ':
# 		if temp_detection_cover <= 74 and temp_detection_cover >= 0:
# 			color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 		elif temp_detection_cover <= 100 and temp_detection_cover >= 75:
# 			color_detection_result['Coverage is adequate'] = 'Adequate'
# 		else:
# 			print("Can't determined")
    
# 	if temp_detection_cover == 100 or 'full' in color_detection_result['Coverage Provided']:
# 		color_detection_result['Detection Rating'] = 'AAA'
# 	elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Adequate':
# 		color_detection_result['Detection Rating'] = 'AA' 
# 	elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage Provided'] == '' and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 		color_detection_result['Detection Rating'] = 'A' 
# 	elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 		color_detection_result['Detection Rating'] = 'A'
# 	elif color_detection_result['Detection Installed'] == "No" and temp_detection_required_cover == 0:
# 		color_detection_result['Detection Rating'] = 'AA'
# 	elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_required_cover == 0:
# 		color_detection_result['Detection Rating'] = 'AA'    
# 	elif color_detection_result['Detection Installed'] == "Yes":
# 		color_detection_result['Detection Rating'] = 'A'
# 	elif temp_detection_cover < 100 and temp_detection_cover >= 75:
# 		#color_detection_result['Coverage is adequate'] = 'Adequate'
# 		color_detection_result['Detection Rating'] = 'AA'
# 	elif temp_detection_cover <= 74 and temp_detection_cover > 0:
# 		#color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 		color_detection_result['Detection Rating'] = 'A'
# 	else:
# 		print("Else statement")
# 	return color_detection_result

# ----------------------------------- Below is Version 2 code -----------------------------------------------------------------

# def final_rating_calculation_fire_detection(color_detection_result, carrier_name):
# 	print("final_rating_calculation_fire_detection : ", color_detection_result)
# 	temp_detection_cover = ""
# 	temp_detection_required_cover = ""
# 	if carrier_name == "AIG":
# 		try:
# 			temp_detection_cover = float(re.sub("\s*%", '', color_detection_result['Coverage Provided']))
# 		except :
# 			temp_detection_cover = 0
# 		try:
# 			temp_detection_required_cover = float(re.sub("\s*%", '', color_detection_result['Extra coverage required']))
# 		except:
# 			temp_detection_required_cover = 0
# 		print(temp_detection_cover, " ---- ", temp_detection_required_cover)
    
# 		if color_detection_result['Coverage is adequate'] == '' or color_detection_result['Coverage is adequate'] == ' ':
# 			if temp_detection_cover <= 74 and temp_detection_cover >= 0:
# 				color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 			elif temp_detection_cover <= 100 and temp_detection_cover >= 75:
# 				color_detection_result['Coverage is adequate'] = 'Adequate'
# 			else:
# 				print("Can't determined adequecy for AIG")
# 	print("temp_detection_cover : ", temp_detection_cover)
    
# 	try:    
# 		if temp_detection_cover == 100 or 'full' in color_detection_result['Coverage Provided']:
# 			color_detection_result['Detection Rating'] = 'AAA'
# 		elif color_detection_result['Detection Installed'] == "Yes"  and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Adequate':
# 			color_detection_result['Detection Rating'] = 'AA' 
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 			color_detection_result['Detection Rating'] = 'A' 
# 		#elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 		#	color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "No" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'AA'    
# 		elif color_detection_result['Detection Installed'] == "Yes":
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif temp_detection_cover < 100 and temp_detection_cover >= 75:
# 			#color_detection_result['Coverage is adequate'] = 'Adequate'
# 			color_detection_result['Detection Rating'] = 'AA'
# 		elif temp_detection_cover <= 74 and temp_detection_cover > 0:
# 			#color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change
# 		elif color_detection_result['Detection Installed'] == "No" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change     
# 		else:
# 			print("Detection rating can't be determined")
# 	except TypeError:
# 		if 'full' in color_detection_result['Coverage Provided']:
# 			color_detection_result['Detection Rating'] = 'AAA'
# 		elif color_detection_result['Detection Installed'] == "Yes"  and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Adequate':
# 			color_detection_result['Detection Rating'] = 'AA' 
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 			color_detection_result['Detection Rating'] = 'A' 
# 		#elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 		#	color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "No" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'AA'    
# 		elif color_detection_result['Detection Installed'] == "Yes":
# 			color_detection_result['Detection Rating'] = 'A'
# 		#elif temp_detection_cover < 100 and temp_detection_cover >= 75:
# 			#color_detection_result['Coverage is adequate'] = 'Adequate'
# 			#color_detection_result['Detection Rating'] = 'AA'
# 		#elif temp_detection_cover <= 74 and temp_detection_cover > 0:
# 			#color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 			#color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change
# 		elif color_detection_result['Detection Installed'] == "No" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change     
# 		else:
# 			print("Detection rating can't be determined") 
# 	return color_detection_result

# def final_rating_calculation_fire_detection(color_detection_result, carrier_name):
# 	print("final_rating_calculation_fire_detection : ", color_detection_result)
# 	if color_detection_result['Coverage Provided'] == "":
# 		temp_detection_cover = ""
# 	else:
# 		temp_detection_cover = color_detection_result['Coverage Provided']
# 	if color_detection_result['Extra coverage required'] == "":    
# 		temp_detection_required_cover = ""
# 	else:
# 		temp_detection_required_cover = color_detection_result['Extra coverage required']
# 	if carrier_name == "AIG":
# 		try:
# 			temp_detection_cover = float(re.sub("\s*%", '', color_detection_result['Coverage Provided']))
# 		except :
# 			temp_detection_cover = 0
# 		try:
# 			temp_detection_required_cover = float(re.sub("\s*%", '', color_detection_result['Extra coverage required']))
# 		except:
# 			temp_detection_required_cover = 0
# 		print(temp_detection_cover, " ---- ", temp_detection_required_cover)
    
# 		if color_detection_result['Coverage is adequate'] == '' or color_detection_result['Coverage is adequate'] == ' ':
# 			if temp_detection_cover <= 74 and temp_detection_cover >= 0:
# 				color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 			elif temp_detection_cover <= 100 and temp_detection_cover >= 75:
# 				color_detection_result['Coverage is adequate'] = 'Adequate'
# 			else:
# 				print("Can't determined adequecy for AIG")

# ----------------------------------- Below is Version 3 code ----------------------------------------------------------------- 

# 	try:    
# 		if temp_detection_cover == 100 or 'full' in color_detection_result['Coverage Provided']:
# 			color_detection_result['Detection Rating'] = 'AAA'
# 		elif color_detection_result['Detection Installed'] == "Yes"  and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Adequate':
# 			color_detection_result['Detection Rating'] = 'AA' 
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 			color_detection_result['Detection Rating'] = 'A' 
# 		#elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 		#	color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change
# 		elif color_detection_result['Detection Installed'] == "No" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change 
# 		elif color_detection_result['Detection Installed'] == "No" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'AA'    
# 		elif color_detection_result['Detection Installed'] == "Yes":
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif temp_detection_cover < 100 and temp_detection_cover >= 75:
# 			#color_detection_result['Coverage is adequate'] = 'Adequate'
# 			color_detection_result['Detection Rating'] = 'AA'
# 		elif temp_detection_cover <= 74 and temp_detection_cover > 0:
# 			#color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 			color_detection_result['Detection Rating'] = 'A' 
# 		elif color_detection_result['Detection Installed'] == "No" :
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" :
# 			color_detection_result['Detection Rating'] = 'A'
# 		else:
# 			print("Detection rating can't be determined")
# 	except TypeError:
# 		if 'full' in color_detection_result['Coverage Provided']:
# 			color_detection_result['Detection Rating'] = 'AAA'
# 		elif color_detection_result['Detection Installed'] == "Yes"  and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Adequate':
# 			color_detection_result['Detection Rating'] = 'AA' 
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 			color_detection_result['Detection Rating'] = 'A' 
# 		#elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
# 		#	color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change
# 		elif color_detection_result['Detection Installed'] == "No" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
# 			color_detection_result['Detection Rating'] = 'A'  # Today Change 
# 		elif color_detection_result['Detection Installed'] == "No" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "Yes" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
# 			color_detection_result['Detection Rating'] = 'AA'    
# 		elif color_detection_result['Detection Installed'] == "Yes":
# 			color_detection_result['Detection Rating'] = 'A'
# 		elif color_detection_result['Detection Installed'] == "No":
# 			color_detection_result['Detection Rating'] = 'A'
# 		#elif temp_detection_cover < 100 and temp_detection_cover >= 75:
# 			#color_detection_result['Coverage is adequate'] = 'Adequate'
# 			#color_detection_result['Detection Rating'] = 'AA'
# 		#elif temp_detection_cover <= 74 and temp_detection_cover > 0:
# 			#color_detection_result['Coverage is adequate'] = 'Not Adequate'
# 			#color_detection_result['Detection Rating'] = 'A'    
# 		else:
# 			print("Detection rating can't be determined")
# 	return color_detection_result

# ----------------------------------- Below is Version 4 code -----------------------------------------------------------------

def final_rating_calculation_fire_detection(color_detection_result, carrier_name):
    print("final_rating_calculation_fire_detection : ", color_detection_result)
    if color_detection_result['Coverage Provided'] == "":
        temp_detection_cover = ""
    else:
        temp_detection_cover = color_detection_result['Coverage Provided']
    if color_detection_result['Extra coverage required'] == "":    
        temp_detection_required_cover = ""
    else:
        temp_detection_required_cover = color_detection_result['Extra coverage required']
    if carrier_name == "AIG":
        try:
            temp_detection_cover = float(re.sub("\s*%", '', color_detection_result['Coverage Provided']))
        except :
            temp_detection_cover = 0
        try:
            print("----")
            temp_detection_required_cover = float(re.sub("\s*%", '', color_detection_result['Extra coverage required']))
        except:
            temp_detection_required_cover = "--"
        print(temp_detection_cover, " ---- ", temp_detection_required_cover)
    
        if color_detection_result['Coverage is adequate'] == '' or color_detection_result['Coverage is adequate'] == ' ':
            if temp_detection_cover <= 74 and temp_detection_cover >= 0:
                color_detection_result['Coverage is adequate'] = 'Not Adequate'
            elif temp_detection_cover <= 100 and temp_detection_cover >= 75:
                color_detection_result['Coverage is adequate'] = 'Adequate'
            else:
                print("Can't determined adequecy for AIG")
    else:
        try:
            temp_detection_cover = float(re.sub("\s*%", '', color_detection_result['Coverage Provided']))
        except :
            temp_detection_cover = "--"
        try:
            print("----")
            temp_detection_required_cover = float(re.sub("\s*%", '', color_detection_result['Extra coverage required']))
        except:
            temp_detection_required_cover = "--"
            
    try:    
        if temp_detection_cover == 100 or 'full' in color_detection_result['Coverage Provided']:
            color_detection_result['Detection Rating'] = 'AAA'
        elif color_detection_result['Detection Installed'] == "Yes"  and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Adequate':
            color_detection_result['Detection Rating'] = 'AA' 
        elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
            color_detection_result['Detection Rating'] = 'A' 
        #elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
        #    color_detection_result['Detection Rating'] = 'A'
        elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
            color_detection_result['Detection Rating'] = 'A'  # Today Change
        elif color_detection_result['Detection Installed'] == "No" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
            color_detection_result['Detection Rating'] = 'A'  # Today Change 
        elif color_detection_result['Detection Installed'] == "No" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
            color_detection_result['Detection Rating'] = 'A'
        elif color_detection_result['Detection Installed'] == "Yes" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
            color_detection_result['Detection Rating'] = 'AA'    
        elif color_detection_result['Detection Installed'] == "Yes":
            color_detection_result['Detection Rating'] = 'A'
        elif temp_detection_cover < 100 and temp_detection_cover >= 75:
            #color_detection_result['Coverage is adequate'] = 'Adequate'
            color_detection_result['Detection Rating'] = 'AA'
        elif temp_detection_cover <= 74 and temp_detection_cover > 0:
            #color_detection_result['Coverage is adequate'] = 'Not Adequate'
            color_detection_result['Detection Rating'] = 'A' 
        elif color_detection_result['Detection Installed'] == "No" :
            color_detection_result['Detection Rating'] = 'A'
        elif color_detection_result['Detection Installed'] == "Yes" :
            color_detection_result['Detection Rating'] = 'A'
        else:
            print("Detection rating can't be determined")
    except TypeError:
        print("here")
        if 'full' in color_detection_result['Coverage Provided']:
            color_detection_result['Detection Rating'] = 'AAA'
        elif color_detection_result['Detection Installed'] == "Yes"  and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Adequate':
            color_detection_result['Detection Rating'] = 'AA' 
        elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
            color_detection_result['Detection Rating'] = 'A' 
        #elif color_detection_result['Detection Installed'] == "Yes" and color_detection_result['Coverage is adequate'] == 'Not Adequate':
        #    color_detection_result['Detection Rating'] = 'A'
        elif color_detection_result['Detection Installed'] == "Yes" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
            color_detection_result['Detection Rating'] = 'A'  # Today Change
        elif color_detection_result['Detection Installed'] == "No" and temp_detection_cover == "" and color_detection_result['Coverage is adequate'] == '' :  # Today Change
            color_detection_result['Detection Rating'] = 'A'  # Today Change 
        elif color_detection_result['Detection Installed'] == "No" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
            color_detection_result['Detection Rating'] = 'A'
        elif color_detection_result['Detection Installed'] == "Yes" and (temp_detection_required_cover == 0 or temp_detection_required_cover == ""):
            color_detection_result['Detection Rating'] = 'AA'    
        elif color_detection_result['Detection Installed'] == "Yes":
            color_detection_result['Detection Rating'] = 'A'
        elif color_detection_result['Detection Installed'] == "No":
            color_detection_result['Detection Rating'] = 'A'
        #elif temp_detection_cover < 100 and temp_detection_cover >= 75:
            #color_detection_result['Coverage is adequate'] = 'Adequate'
            #color_detection_result['Detection Rating'] = 'AA'
        #elif temp_detection_cover <= 74 and temp_detection_cover > 0:
            #color_detection_result['Coverage is adequate'] = 'Not Adequate'
            #color_detection_result['Detection Rating'] = 'A'    
        else:
            print("Detection rating can't be determined")
    return color_detection_result
  
#color_detection_result = {'Detection': 'Inadequate', 'Detection Installed': 'Yes', 'Coverage Provided': 'fully covered', 'Extra coverage required': '', 'Coverage is #adequate': '', 'Detection Rating': ''}
#result = final_rating_calculation_fire_detection(color_detection_result)
#print(result)

In [0]:
import re
import nltk
from nltk import sent_tokenize

# ----------------------------------- Below is Version 1 code -----------------------------------------------------------------

# #data = """the nearest public fire department is located at campana city, 1 km away from this mill. however this is a fire department with very low resources, since it is a very small city. the fire department in buenos aires, capital city of argentina, could also respond, but no earlier than one hour. tenaris siderca has its own fire brigade who would probably provide the initial response. please refer to "emergency organization narrative" section. additionally, there is a mutual and formal agreement in between most important industrial companies installed within this region in order to provide each other with emergency fire protection response in case one of them need. it is named pacei. to mention some of them: air liquide, cabot argentina, carboclor, celulosa argentina, ceveceria y malteria quilmes, clariant, esso, lanxess, dak americas argentina, merisant argentina, monsanto argentina, petrobras energia sa, praxair, among others. there are frequent meetings in between companies representatives and formal procedures."""


# def discription_fire_bridage_processing(data):
#     data = data.lower()
#     # print("Fire Discription : ", data)
#     discription_fire_bridage = {'distance': '', 'responce': '', 'org': ''}

#     pattern_distance = "\(*([0-9]+\.?\/?[0-9]*)\)*\-*\s*(km|kms|Km|kms|kilometer|kilometers|Kilometer|kilometre|kilometres|Kilometers|miles|mile|meters|meter|m\.?\s+|m\.?$)"
#     pattern_responce = "\(*([0-9]+\.?[0-9]*)\)*\-*\s*(min|minute|minutes|hours|hour|second|seconds)"
#     # keyword_pattern = '(paid|volunteer|service|full time|24|manned|fighter|fire|department|station)'
#     keyword_org = ["public fire department", "public fire brigade", "public fire-brigade", "public fire-department",
#                    "local authority", "municipal", "volunteer", "low resources", "volunteer fire department",
#                    "volunteer fire service", "paid", "manned 24/7", "full time", "full-time", "24/7", "7/24",
#                    "permanently manned", "fire brigade", "fire-brigade", "fire fighters", "fire fighter",
#                    "fire station", "fire department", "fire-department", "station"]
#     orgni = []
#     res = []
#     dis = []
#     sentences = nltk.sent_tokenize(data)

#     for sentence in sentences:
#         # print("sentence fire : ",sentence)
#         result_distance = re.search(pattern_distance, sentence)
#         if result_distance and (sentence not in discription_fire_bridage['distance']):
#             print("sentence fire : ", sentence)
#             discription_fire_bridage['distance'] += discription_fire_bridage['distance'] + sentence + " "
#         result_responce = re.search(pattern_responce, sentence)
#         if result_responce and (sentence not in discription_fire_bridage['responce']):
#             discription_fire_bridage['responce'] += discription_fire_bridage['responce'] + sentence + " "
#         # 		result_org = re.search(keyword_pattern, sentence)
#         # 		if result_org and (sentence not in discription_fire_bridage['org']):
#         # 			discription_fire_bridage['org'] += discription_fire_bridage['org']+sentence+" "
#         # 		#result_distance = re.search(pattern, sentence)
#         if any(org_key in sentence for org_key in keyword_org):
#             if (sentence not in orgni):
#                 orgni.append(sentence)
#         discription_fire_bridage['org'] = convert_To_String(orgni)

#     discription_fire_bridage['distance'] = re.sub("\n*\t*", '', discription_fire_bridage['distance'])
#     discription_fire_bridage['responce'] = re.sub("\n*\t*", '', discription_fire_bridage['responce'])
#     discription_fire_bridage['org'] = re.sub("\n*\t*", '', discription_fire_bridage['org'])

#     return discription_fire_bridage
  
# #discription_fire_bridage = discription_fire_bridage(data)
# #print("discription_fire_bridage : ", discription_fire_bridage)


# #data = """This facility is provided with two separate private water supplies, one located at Tamsa 2 and one located at Tamsa 3. These are two separated system that are not interconnected due to the distance between them. 
# #"""

# def discription_water_supply_processing(data):
#     data = data.lower()
#     # print("": ", data)
#     discription_water_supply = {'flow': '', 'install': ''}

#     pattern_water = '(gpm|lpm|l/min|g/min|l/s|l/sec)'
#     keyword_water = ["install", "installed"]

#     wat = []

#     sentences = nltk.sent_tokenize(data)


#     print("SENTENCES INSIDE FUNCTION", sentences)
#     for sentence in sentences:
#         # print("sentence water : ",sentence)
#         result_water = re.search(pattern_water, sentence)
#         if result_water and (sentence not in discription_water_supply['flow']):
#             discription_water_supply['flow'] += discription_water_supply['flow'] + sentence + " "
#         if any(water_key in sentence for water_key in keyword_water):
#             if sentence not in wat:
#                 wat.append(sentence)

#     discription_water_supply['install'] = convert_To_String(wat)

#     discription_water_supply['flow'] = re.sub("\n*\t*", '', discription_water_supply['flow'])
#     discription_water_supply['install'] = re.sub("\n*\t*", '', discription_water_supply['install'])

#     return discription_water_supply
  
# #discription_water_supply = discription_water_supply(data)
# #print("discription_water_supply : ", discription_water_supply)

# ----------------------------------- Below is Version 2 code -----------------------------------------------------------------

#data = """The site is served by the NSW Fire and Rescue Stations with full-time crews 5min. The nearest station is located in Maroubra, which is less than 1km away from the site. The next closest fire station is #located in Matraville which is located less than 3km away from the site. """


def discription_fire_bridage_processing(data):
    data = data.lower()
    # print("Fire Discription : ", data)
    discription_fire_bridage = {'distance': '', 'responce': '', 'org': ''}

    pattern_distance = "(km|kms|Km|kms|kilometer|kilometers|Kilometer|kilometre|kilometres|Kilometers|miles|mile|meters|meter|m\.?\s+|m\.?$)"
    pattern_responce = "\(*([0-9]+\.?[0-9]*)\)*\-*\s*(min|minute|minutes|hours|hour|second|seconds)"
    # keyword_pattern = '(paid|volunteer|service|full time|24|manned|fighter|fire|department|station)'
    keyword_org = ["public fire department", "public fire brigade", "public fire-brigade", "public fire-department",
                   "local authority", "municipal", "volunteer", "low resources", "volunteer fire department",
                   "volunteer fire service", "paid", "manned 24/7", "full time", "full-time", "24/7", "7/24",
                   "permanently manned", "fire brigade", "fire-brigade", "fire fighters", "fire fighter",
                   "fire station", "fire department", "fire-department", "station"]
    orgni = []
    res = []
    dis = []
    sentences = nltk.sent_tokenize(data)

    for sentence in sentences:
        # print("sentence fire : ",sentence)
        result_distance = re.search(pattern_distance, sentence)
        if result_distance:# and (sentence not in discription_fire_bridage['distance']):
            #print("sentence fire : ", sentence)
            if sentence not in dis:
              dis.append(sentence)
            #discription_fire_bridage['distance'] += discription_fire_bridage['distance'] + sentence #+ " "
        result_responce = re.search(pattern_responce, sentence)
        if result_responce:# and (sentence not in discription_fire_bridage['responce']):
            if sentence not in dis:#discription_fire_bridage['responce'] += discription_fire_bridage['responce'] + sentence + " "
              res.append(sentence)
        if any(org_key in sentence for org_key in keyword_org):
            if (sentence not in orgni):
                orgni.append(sentence)
        discription_fire_bridage['org'] = convert_To_String(orgni)

    discription_fire_bridage['distance'] = convert_To_String(dis)
    discription_fire_bridage['distance'] = re.sub("\n*\t*", '', discription_fire_bridage['distance'])
    discription_fire_bridage['responce'] = convert_To_String(res)
    discription_fire_bridage['responce'] = re.sub("\n*\t*", '', discription_fire_bridage['responce'])
    discription_fire_bridage['org'] = re.sub("\n*\t*", '', discription_fire_bridage['org'])

    return discription_fire_bridage
  
#discription_fire_bridage = discription_fire_bridage_processing(data)
#print("discription_fire_bridage : ", discription_fire_bridage)


#data = """This facility is provided with two separate private 12 gpm water supplies, one located at Tamsa 2 and one located at Tamsa 3. These are two separated system that are not interconnected due to the distance between them install. 
#"""

def discription_water_supply_processing(data):
    data = data.lower()
    # print("": ", data)
    discription_water_supply = {'flow': '', 'install': ''}

    pattern_water = '(gpm|lpm|l/min|g/min|l/s|l/sec)'
    keyword_water = ["install", "installed"]

    wat = []
    wat_flow = []
    sentences = nltk.sent_tokenize(data)


    print("SENTENCES INSIDE FUNCTION", sentences)
    for sentence in sentences:
        # print("sentence water : ",sentence)
        result_water = re.search(pattern_water, sentence)
        if result_water:# and (sentence not in discription_water_supply['flow']):
            if sentence not in wat_flow:
              wat_flow.append(sentence)
        if any(water_key in sentence for water_key in keyword_water):
            if sentence not in wat:
                wat.append(sentence)

    discription_water_supply['install'] = convert_To_String(wat)
    discription_water_supply['flow'] = convert_To_String(wat_flow)
    discription_water_supply['flow'] = re.sub("\n*\t*", '', discription_water_supply['flow'])
    discription_water_supply['install'] = re.sub("\n*\t*", '', discription_water_supply['install'])

    return discription_water_supply
  
#discription_water_supply = discription_water_supply_processing(data)
#print("discription_water_supply : ", discription_water_supply)



#data = """All alarms report to the constantly attended security gate house with fire alarms automatically retransmitted to a SAMFS monitoring station using an ASE.Security Measures
#"""
# ----------------------------------------------  Version 1 Code Start ----------------------------------------------------------------------
# def discription_fire_detection_processing(data):
# 	data = data.lower()
# 	#print("Fire Discription : ", data)
# 	discription_fire_detection = {'install':'', 'adequate':''}

# 	pattern_install = ["installed", "installations", "fire alarms are generally installed", "detection is also provided", "fire alarms are checked", "detectors located", "fire alarms and transmit", "sprinkler protection is provided", "coverage completely", 
#      "automatic deluge systems","automatic fire suppression system","sprinkler system is provided", "smoke and heat detectors", "smoke and heat detectors,", 
#     "sprinklers are installed", "there are automatic", "sprinklers installed", "system is installed", "sprinkler system installed",
#     "fire water distribution & hydrants average","coverage completely ", "areas are protected with automatic sprinkler",
#     "protected with sprinklers","protected by","protection is provided", "have sprinkler protection", "protection is installed",
#     "sprinkler systems have been designed", "system is located throughout", "adequate sprinkler", "active protection",
#     "sprinkler throughout", "system is through", "sprinkler protection is installed", "throughout","sprinkler system reported",
#     "been provided", "provided with automatic", "automatic Fire detection systems", "existing fire detection", "detection provided", 
#     "installed throughout", "located throughout", "fire detection throughout", "reported in all", "detectors are present", "detectors are installed", 
#     "detection installed", "provided with smoke detectors", "system installed", "systems installed", "systems are fully functional", 
#     "detection is installed", "following areas have", "transmit via", "protected by", "fire alarms are generally installed", 
#     "fire alarm panel", "approved alarm control panel", "protection provided", "detection is provided", "smoke detection is provided", 
#     "smoke detection common", "smoke detectors located", "scattered fire alarms", "detection for this site consists", "combined sprinkler and hydrant system installed",
# 	"not installed", "not reported", "sprinklers none", "no sprinkler system provided", 
#     "no sprinkler","no sprinklers","no sprinkler protection installed",
#     "no comprehensive sprinkler","no automatic","not sprinkler protected","protection initially installed",
#     "no automatic fire suppression", "sprinklers throughout","sprinkler schematics confirm","none", "fire detection none", "detection none",
#     "no detection system provided","no automatic","no automatic fire detection","not provided"]#'(install|installation|indicator|smoke|heat|alarms)'
# 	pattern_adequate = ['adequate', 'adequacy coverage', 'as adequate', 'satisfactory','not adequate','non adequate','below average','out of standard', 'recommendation to expand']#'(adequate|adequacy|below average|out of standard)'
        
# 	sentences = nltk.sent_tokenize(data)

# 	install_result = []
# 	adequate_result = []
    
# 	for sentence in sentences:
# 		#print("sentence water : ",sentence)
# 		#result_install_detection = re.search(pattern_install, sentence)
# 		#result_adedquate_detection = re.search(pattern_adequate, sentence)
# 		if any(result_install_detection in sentence for result_install_detection in pattern_install):
# 			if sentence not in install_result:
# 				install_result.append(sentence)
# 		if any(result_adedquate_detection in sentence for result_adedquate_detection in pattern_adequate):
# 			if sentence not in adequate_result:
# 				adequate_result.append(sentence)

# 	discription_fire_detection['install'] = convert_To_String(install_result)
# 	discription_fire_detection['adequate'] = convert_To_String(adequate_result)
                
# 	discription_fire_detection['install'] = re.sub("\n*\t*",'',discription_fire_detection['install'])
# 	discription_fire_detection['adequate'] = re.sub("\n*\t*",'',discription_fire_detection['adequate'])
    
# 	return discription_fire_detection
  
# #discription_fire_detection = discription_fire_detection(data)
# #print("discription_fire_detection : ", discription_fire_detection)


# #data = """The original installation date of sprinklers in the main building is unknown. However, the system was largely replaced and upgraded as part of the renovations in 2013, so has been classified as installed in this year.The system is designed and installed in accordance with AS2118 OH2 (5mm/min over 144sq.m) in all areas. NFPA requires a higher density in the car park, but it is considered based on the good water supply available. The other buildings are not sprinkler protected. As a result protection is rated Below average and a recommendation to provide in 83 Myer St has been made. Provision in 43 Little Myer St is not considered cost justified.Fire Extinguishing Apparatus 
# #"""

# def discription_fire_sprinkler_processing(data):
# 	data = data.lower()
# 	#print("Fire Discription : ", data)
# 	discription_fire_sprinkler = {'install':'', 'adequate':''}

# 	pattern_install = ["installed", "fire alarms are generally installed", "fully sprinkler protected", "detection is also provided", "fire alarms are checked", "detectors located", "fire alarms and transmit", "installations", "sprinkler protection is provided", "coverage completely", 
#      "automatic deluge systems","automatic fire suppression system","sprinkler system is provided",
#     "sprinklers are installed", "there are automatic", "sprinklers installed", "system is installed", "sprinkler system installed",
#     "fire water distribution & hydrants average","coverage completely ", "areas are protected with automatic sprinkler",
#     "protected with sprinklers","protected by","protection is provided", "have sprinkler protection", "protection is installed",
#     "sprinkler systems have been designed", "system is located throughout", "adequate sprinkler", "active protection",
#     "sprinkler throughout", "system is through", "sprinkler protection is installed", "throughout","sprinkler system reported",
#     "been provided", "provided with automatic", "automatic Fire detection systems", "existing fire detection", "detection provided", 
#     "installed throughout", "located throughout", "fire detection throughout", "reported in all", "detectors are present", "detectors are installed", 
#     "detection installed", "provided with smoke detectors", "system installed", "systems installed", "systems are fully functional", 
#     "detection is installed", "following areas have", "transmit via", "protected by", "fire alarms are generally installed", 
#     "fire alarm panel", "approved alarm control panel", "protection provided", "detection is provided", "smoke detection is provided", 
#     "smoke detection common", "smoke detectors located", "scattered fire alarms", "detection for this site consists", "combined sprinkler and hydrant system installed",
# 	"not installed", "not reported", "sprinklers none", "no sprinkler system provided", 
#     "no sprinkler","no sprinklers","no sprinkler protection installed",
#     "no comprehensive sprinkler","no automatic","not sprinkler protected","protection initially installed",
#     "no automatic fire suppression", "sprinklers throughout","sprinkler schematics confirm","none", "fire detection none", "detection none",
#     "no detection system provided","no automatic","no automatic fire detection","not provided"] # '(install|installation|sprinklers)'
# 	pattern_adequate = ['adequate', 'adequacy coverage', 'as adequate', 'satisfactory','not adequate','non adequate','below average','out of standard', 'recommendation to expand'] #'(adequate|adequacy|below average|out of standard)'
        
# 	sentences = nltk.sent_tokenize(data)

# 	install_result = []
# 	adequate_result = []
    
# 	for sentence in sentences:
# 		#print("sentence water : ",sentence)
# 		#result_install_sprinkler = re.search(pattern_install, sentence)
# 		#result_adequate_sprinkler = re.search(pattern_adequate, sentence)
# 		if any(result_install_sprinkler in sentence for result_install_sprinkler in pattern_install):
# 			if sentence not in install_result:
# 				install_result.append(sentence)
# 		if any(result_adequate_sprinkler in sentence for result_adequate_sprinkler in pattern_adequate):
# 			if sentence not in adequate_result:
# 				adequate_result.append(sentence)

# 	discription_fire_sprinkler['install'] = convert_To_String(install_result)
# 	discription_fire_sprinkler['adequate'] = convert_To_String(adequate_result)
                
# 	discription_fire_sprinkler['install'] = re.sub("\n*\t*",'',discription_fire_sprinkler['install'])
# 	discription_fire_sprinkler['adequate'] = re.sub("\n*\t*",'',discription_fire_sprinkler['adequate'])
    
# 	return discription_fire_sprinkler
# ----------------------------------------------  Version 1 Code End  ----------------------------------------------------------------------

# ----------------------------------------------  Version 2 Code Start  --------------------------------------------------------------------

def discription_fire_detection_processing(data):
	data = data.lower()
	#print("Fire Discription : ", data)
	discription_fire_detection = {'install':'', 'adequate':'', 'provided': '', 'required': ''}

	pattern_install = ["installed", "installations", "fire alarms are generally installed", "detection is also provided", "fire alarms are checked", "detectors located", "fire alarms and transmit", "sprinkler protection is provided", "coverage completely", "fire detection", "automatic smoke detection systems", "activation of the fire alarm",
     "automatic deluge systems","automatic fire suppression system","sprinkler system is provided", "smoke and heat detectors", "smoke and heat detectors", 
    "sprinklers are installed", "there are automatic", "sprinklers installed", "system is installed", "sprinkler system installed",
    "fire water distribution & hydrants average","coverage completely ", "areas are protected with automatic sprinkler",
    "protected with sprinklers","protected by","protection is provided", "have sprinkler protection", "protection is installed",
    "sprinkler systems have been designed", "system is located throughout", "adequate sprinkler", "active protection",
    "sprinkler throughout", "system is through", "sprinkler protection is installed", "throughout","sprinkler system reported",
    "been provided", "provided with automatic", "automatic Fire detection systems", "existing fire detection", "detection provided", 
    "installed throughout", "located throughout", "fire detection throughout", "reported in all", "detectors are present", "detectors are installed", 
    "detection installed", "provided with smoke detectors", "system installed", "systems installed", "systems are fully functional", 
    "detection is installed", "following areas have", "transmit via", "protected by", "fire alarms are generally installed", 
    "fire alarm panel", "approved alarm control panel", "protection provided", "detection is provided", "smoke detection is provided", 
    "smoke detection common", "smoke detectors located", "scattered fire alarms", "detection for this site consists", "combined sprinkler and hydrant system installed",
	"not installed", "not reported", "sprinklers none", "no sprinkler system provided", 
    "no sprinkler","no sprinklers","no sprinkler protection installed",
    "no comprehensive sprinkler","no automatic","not sprinkler protected","protection initially installed",
    "no automatic fire suppression", "sprinklers throughout","sprinkler schematics confirm","none", "fire detection none", "detection none",
    "no detection system provided","no automatic","no automatic fire detection","not provided"]#'(install|installation|indicator|smoke|heat|alarms)'
	pattern_adequate = ['adequate', 'adequacy coverage', 'as adequate', 'satisfactory','not adequate','non adequate','below average','out of standard', 'recommendation to expand']#'(adequate|adequacy|below average|out of standard)'
	provided_coverage_word = ["installed throughout", "not required in other areas", "fully sprinkler protected", 
                                         "fully covered", "full coverage provided", "partly covered",
                                         "coverage is partly coverage", "most areas are protected",
                                         "no extra coverage required", "no smoke detection", "system is through", "sprinkler supply", 
                                         "installed", "combined sprinkler and hydrant system installed", "no smoke detection", 
                                         "installed throughout the site", "installed", "not installed", "scattered fire alarms", 
                                         "fire detection throughout"]
	required_coverage_word = ["no extra coverage required", "not applicable"]
	full = ["installed throughout", "not required in other areas","fully covered", "full coverage provided","installed throughout the site", "fire detection throughout"]
	partial = ["partly covered", "coverage is partly coverage", "most areas are protected", "combined sprinkler and hydrant system installed",
    "scattered fire alarms", "installed", "not installed"]
        
	sentences = nltk.sent_tokenize(data)

	install_result = []
	adequate_result = []
	provided_result = []
	required_result = []
    
	for sentence in sentences:
		#print("sentence water : ",sentence)
		#result_install_detection = re.search(pattern_install, sentence)
		#result_adedquate_detection = re.search(pattern_adequate, sentence)
		if any(result_install_detection in sentence for result_install_detection in pattern_install):
			if sentence not in install_result:
				install_result.append(sentence)
		if any(result_adedquate_detection in sentence for result_adedquate_detection in pattern_adequate):
			if sentence not in adequate_result:
				adequate_result.append(sentence)
		if any(result_provided_detection in sentence for result_provided_detection in provided_coverage_word):
			if sentence not in provided_result:
				provided_result.append(sentence)
		if any(result_required_detection in sentence for result_required_detection in required_coverage_word):
			if sentence not in required_result:
				required_result.append(sentence)
                

	discription_fire_detection['install'] = convert_To_String(install_result)
	discription_fire_detection['adequate'] = convert_To_String(adequate_result)
	discription_fire_detection['provided'] = convert_To_String(provided_result)
	discription_fire_detection['required'] = convert_To_String(required_result)
                
	discription_fire_detection['install'] = re.sub("\n*\t*",'',discription_fire_detection['install'])
	discription_fire_detection['adequate'] = re.sub("\n*\t*",'',discription_fire_detection['adequate'])
	discription_fire_detection['provided'] = re.sub("\n*\t*",'',discription_fire_detection['provided'])
	discription_fire_detection['required'] = re.sub("\n*\t*",'',discription_fire_detection['required'])
	print("Fire Discription : ", data)    
	return discription_fire_detection
  
def discription_fire_sprinkler_processing(data):
	data = data.lower()
	#print("Fire Discription : ", data)
	discription_fire_sprinkler = {'install':'', 'adequate':'', 'provided': '', 'required': ''}

	pattern_install = ["installed", "fire alarms are generally installed", "fully sprinkler protected", "detection is also provided", "fire alarms are checked", "detectors located", "fire alarms and transmit", "installations", "sprinkler protection is provided", "coverage completely", 
     "automatic deluge systems","automatic fire suppression system","sprinkler system is provided",
    "sprinklers are installed", "there are automatic", "sprinklers installed", "system is installed", "sprinkler system installed",
    "fire water distribution & hydrants average","coverage completely ", "areas are protected with automatic sprinkler",
    "protected with sprinklers","protected by","protection is provided", "have sprinkler protection", "protection is installed",
    "sprinkler systems have been designed", "system is located throughout", "adequate sprinkler", "active protection",
    "sprinkler throughout", "system is through", "sprinkler protection is installed", "throughout","sprinkler system reported",
    "been provided", "provided with automatic", "automatic Fire detection systems", "existing fire detection", "detection provided", 
    "installed throughout", "located throughout", "fire detection throughout", "reported in all", "detectors are present", "detectors are installed", 
    "detection installed", "provided with smoke detectors", "system installed", "systems installed", "systems are fully functional", 
    "detection is installed", "following areas have", "transmit via", "protected by", "fire alarms are generally installed", 
    "fire alarm panel", "approved alarm control panel", "protection provided", "detection is provided", "smoke detection is provided", 
    "smoke detection common", "smoke detectors located", "scattered fire alarms", "detection for this site consists", "combined sprinkler and hydrant system installed",
	"not installed", "not reported", "sprinklers none", "no sprinkler system provided", 
    "no sprinkler","no sprinklers","no sprinkler protection installed",
    "no comprehensive sprinkler","no automatic","not sprinkler protected","protection initially installed",
    "no automatic fire suppression", "sprinklers throughout","sprinkler schematics confirm","none", "fire detection none", "detection none",
    "no detection system provided","no automatic","no automatic fire detection","not provided"] # '(install|installation|sprinklers)'
	pattern_adequate = ['adequate', 'adequacy coverage', 'as adequate', 'satisfactory','not adequate','non adequate','below average','out of standard', 'recommendation to expand'] #'(adequate|adequacy|below average|out of standard)'
	provided_coverage_word = ["installed throughout", "not required in other areas", "fully sprinkler protected", 
                                         "fully covered", "full coverage provided", "partly covered",
                                         "coverage is partly coverage", "most areas are protected",
                                         "no extra coverage required", "no smoke detection", "system is through", "sprinkler supply", 
                                         "installed", "combined sprinkler and hydrant system installed", "no smoke detection", 
                                         "installed throughout the site", "installed", "not installed", "scattered fire alarms", 
                                         "fire detection throughout"]
	required_coverage_word = ["no extra coverage required", "not applicable"]
        
	sentences = nltk.sent_tokenize(data)

	install_result = []
	adequate_result = []
	provided_result = []
	required_result = []
    
	for sentence in sentences:
		#print("sentence water : ",sentence)
		#result_install_sprinkler = re.search(pattern_install, sentence)
		#result_adequate_sprinkler = re.search(pattern_adequate, sentence)
		if any(result_install_sprinkler in sentence for result_install_sprinkler in pattern_install):
			if sentence not in install_result:
				install_result.append(sentence)
		if any(result_adequate_sprinkler in sentence for result_adequate_sprinkler in pattern_adequate):
			if sentence not in adequate_result:
				adequate_result.append(sentence)
		if any(result_provided_sprinkler in sentence for result_provided_sprinkler in provided_coverage_word):
			if sentence not in provided_result:
				provided_result.append(sentence)
		if any(result_required_sprinkler in sentence for result_required_sprinkler in required_coverage_word):
			if sentence not in required_result:
				required_result.append(sentence)

	discription_fire_sprinkler['install'] = convert_To_String(install_result)
	discription_fire_sprinkler['adequate'] = convert_To_String(adequate_result)
	discription_fire_sprinkler['provided'] = convert_To_String(provided_result)
	discription_fire_sprinkler['required'] = convert_To_String(required_result)
                
	discription_fire_sprinkler['install'] = re.sub("\n*\t*",'',discription_fire_sprinkler['install'])
	discription_fire_sprinkler['adequate'] = re.sub("\n*\t*",'',discription_fire_sprinkler['adequate'])
	discription_fire_sprinkler['provided'] = re.sub("\n*\t*",'',discription_fire_sprinkler['provided'])
	discription_fire_sprinkler['required'] = re.sub("\n*\t*",'',discription_fire_sprinkler['required'])
	print("Fire Sprinkler : ", data)    
	return discription_fire_sprinkler


# ----------------------------------------------  Version 2 Code End  ----------------------------------------------------------------------

#discription_fire_sprinkler = discription_fire_sprinkler_processing(data)
#print("discription_fire_sprinkler : ", discription_fire_sprinkler)

  
#discription_fire_detection = discription_fire_detection_processing(data)
#print("discription_fire_detection : ", discription_fire_detection)


In [0]:
########### Python Form Recognizer Async Analyze #############
import json
import time
from requests import get, post
import os
import traceback

import re
import gc

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import fitz
 
def data_extraction(nameImageWithPath):
	# Endpoint URL
	endpoint = "https://na-aria-formrecognizer.cognitiveservices.azure.com/"
	apim_key = "581924cde7974bd68052fdd925dcdd1f"
	#model_id = "<model_id>"
	post_url = endpoint + "/formrecognizer/v2.1-preview.3/layout/analyze?%s" 
	source = nameImageWithPath
	params = {
	    "includeTextDetails": True
	}

	headers = {
	    # Request headers
	   'Content-Type': 'image/png',
	    'Ocp-Apim-Subscription-Key': apim_key,
	}
	with open(source, "rb") as f:
		data_bytes = f.read()

	try:
		resp = post(url = post_url, data = data_bytes, headers = headers, params = params)
		if resp.status_code != 202:
			pass#print("POST analyze failed:\n%s" % json.dumps(resp.json()))
			#quit()
		#print("POST analyze succeeded:\n%s" % resp.headers)
		get_url = resp.headers["operation-location"]
		request_id = resp.headers["apim-request-id"]
    
	except Exception as e:
		print("POST analyze failed:\n%s" % str(e))
		#quit() 
    
    
	n_tries = 15
	n_try = 0
	wait_sec = 5
	max_wait_sec = 60
	while n_try < n_tries:
		try:
			resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
			resp_json = resp.json()
			if resp.status_code != 200:
				#print("GET analyze results failed:\n%s" % json.dumps(resp_json))
				#quit()
				pass
			status = resp_json["status"]
			if status == "succeeded":
				#print("Analysis succeeded:\n%s" % json.dumps(resp_json))
				with open(IMAGE_PATH+'data_protection.txt', 'w') as outfile:
					json.dump(resp_json, outfile)
					break
					#quit()
			if status == "failed":
				pass#print("Analysis failed:\n%s" % json.dumps(resp_json))
				#quit()
				# Analysis still running. Wait and retry.
			time.sleep(wait_sec)
			n_try += 1
			wait_sec = min(2*wait_sec, max_wait_sec)     
		except Exception as e:
			msg = "GET analyze results failed:\n%s" % str(e)
			print(msg)
			#quit()
			print("Analyze operation did not complete within the allocated time.")

In [0]:
def calculate_total(sprinkler_detection_confirmed):
    areas=[]
    area_sprinkler_covered=[]
    area_detection=[]
    area_sprinkler_required=[]
    for row in sprinkler_detection_confirmed:
      area= re.sub("," ,"", row[2]) #row[2]
      try:
        areas.append(float(area))
      except:
        areas.append(float(0))
      sprinkler_covered=row[3]
      try:
        area_sprinkler_covered.append(int(re.search(r'\d+', sprinkler_covered).group()))
      except:
        area_sprinkler_covered.append(float(0))


      sprinkler_required=row[5]
      try:
        area_sprinkler_required.append(int(re.search(r'\d+', sprinkler_required).group()))
      except:
        area_sprinkler_required.append(float(0))

      detection=row[6]
      try:
        area_detection.append(int(re.search(r'\d+', detection).group()))
      except:
        area_detection.append(float(0))
    #print(areas, area_sprinkler_covered, area_detection, area_sprinkler_required)
    perc_sprinkler=str(round(sum([a*b for a,b in zip(areas,area_sprinkler_covered)])/sum(areas*100)*100)) + "%"
    perc_detection=str(round(sum([a*b for a,b in zip(areas,area_detection)])/sum(areas*100)*100)) + "%"
    perc_sprinkler_required=str(round(sum([a*b for a,b in zip(areas,area_sprinkler_required)])/sum(areas*100)*100)) + "%"     

    return perc_sprinkler, perc_sprinkler_required, perc_detection 

In [0]:
def ratio(word, compare):
	ratio_result = fuzz.ratio(word, compare)
	return ratio_result


def find_relevent_AIG_FireBrigade(results):
	result = [[i for i in row] for row in results.itertuples()]
	#print("result : ", result)	
	if (ratio(result[0], [0, 'Element', 'Excellent', 'Good', 'Adequate', 'Nearly Adeq.', 'Inadequate']) > 90) or (ratio(result[0], [0, 'Element', 'Excellent', 'Good', 'Adequate', 'Nearly Adequate', 'Inadequate']) > 90):		
		return 0



def find_relevent_table_sprinkler(input_data, data, specific, sprinkler_spread_flag):      # -------------- Add this 
	pattern = '[0-9]+\.?[0-9]*'
	total = []
	result_index = {'% AS':"", '% ASN':"", '% Detection':""}
	input_data_result = [[i for i in row] for row in input_data.itertuples()]
	#print("input_data_result : ", input_data_result)
	#print("data_result : ", data)
	required_index = -1
	final_result = ""
	flag_count = 0
	sprinkler_detection = None
	#print("input_data_result[] : ", input_data_result[0], input_data_result[1])
	#print("data[] : ",data[0], data[1])
	#if (ratio(input_data_result[0], data[0]) > 90) or (ratio(input_data_result[1], data[1]) > 90) or (ratio(input_data_result[1], data[2]) > 90) or (ratio(input_data_result[1], data[3]) > 90):
	for match in data:
		#print("Match : ", match)
		#print("input_data_result[] : ", input_data_result[0])
		if (ratio(input_data_result[0], match) > 90) or (ratio(input_data_result[1], match) > 90):	
			#if True:
				#print("input_result : ", input_result)
			sprinkler_detection = input_data_result
			for special in specific:
				try:
					#print("special : ",special)
					try:
						required_index = input_data_result[0].index(special)
					except ValueError:
						required_index = input_data_result[1].index(special)
					if special == "% AS":
						flag_count = flag_count+1
						result_index["% AS"] = required_index
					elif special == "%AS":
						flag_count = flag_count+1
						result_index["% AS"] = required_index
					elif special == "% ASN":
						flag_count = flag_count+1
						result_index["% ASN"] = required_index
					elif special == "% Detection":
						flag_count = flag_count+1
						result_index["% Detection"] = required_index 
					else:
						pass
					#print("Sprinkler : ",input_data_result[0].index(special))
				except:
					#traceback.print_exc()
					#try:
					#	required_index = input_data_result[1].index(special)
					#except:
					#traceback.print_exc()
					pass
		if flag_count == 3:
			break
            
		if sprinkler_spread_flag == True:           # -------------- Add this 
			#print("sprinkler_spread_flag : ", sprinkler_spread_flag)         # -------------- Add this  
			for input_ in input_data_result:     	      # -------------- Add this 
				#l_ = re.findall(pattern, input_[required_index])           # -------------- Add this 
				#print("input_ : ", input_)            # -------------- Add this 
				if ("% Totals" in input_):            # -------------- Add this 
					result_index['% AS'] = input_[3]           # -------------- Add this  
					result_index['% ASN'] = input_[5]           # -------------- Add this 
					result_index['% Detection'] = input_[6]          # -------------- Add this 
			return result_index, sprinkler_detection       # -------------- Add this 
            
	#print("input_result[required_index] : ", required_index)
					#print(pattern)
	#print("result_index : ", result_index)
	for input_ in input_data_result:	
		#l_ = re.findall(pattern, input_[required_index])
		#print(l_)
		if ("% Totals" in input_):
			if (result_index['% AS'] != ''):
				result_index['% AS'] = input_[result_index['% AS']]
			if (result_index['% ASN'] != ''):
				result_index['% ASN'] = input_[result_index['% ASN']]
			if (result_index['% Detection'] != ''):
					result_index['% Detection'] = input_[result_index['% Detection']]
	#print("result_index : ", result_index)
	return result_index, sprinkler_detection



def find_relevent_table_water(input_data, data, specific):
	pattern = '[0-9]+\.?[0-9]*'
	total = []
	unit = ""
	input_data_result = [[i for i in row] for row in input_data.itertuples()]
	#print("input_data_result : ", input_data_result)
	#print("data_result : ", data)
	required_index = -1
	final_result = ""
	#print("input_data_result[] : ", input_data_result[0], input_data_result[1])
	#print("data[] : ",data[0], data[1])
	#if (ratio(input_data_result[0], data[0]) > 90) or (ratio(input_data_result[1], data[1]) > 90) or (ratio(input_data_result[1], data[2]) > 90) or (ratio(input_data_result[1], data[3]) > 90):
	for match in data:
		#print("Match : ", match)
		if (ratio(input_data_result[0], match) > 90) or (ratio(input_data_result[1], match) > 90):	
			#if True:
				#print("input_result : ", input_result)
			for special in specific:
				#print("special : ",special)
				try:
					required_index = input_data_result[2].index(special)
				except ValueError:
					#traceback.print_exc()
					try:
						required_index = input_data_result[1].index(special)
					except:
						pass
				if required_index >= 0:
					unit = special
					break
		if required_index >= 0:
			break
				#print("Rated Flow (gpm) : ",input_result.index(special))
				#traceback.print_exc()
	#print("input_result[required_index] : ", required_index)
					#print(pattern)
	
	for input_ in input_data_result:	
		l_ = re.findall(pattern, input_[required_index])
		#print(l_)
		if (required_index != -1 and len(l_) > 0):
			final_result = float(input_[required_index])
			#if final_result not in total:
			total.append(final_result)
	#print("total, unit : ", total, unit)
	return total, unit

# def gpm_lpm_conversion(l):      # -------------- Add this
# 	result = 0
# 	if ("gpm" in l[-1].lower() or "g/min" in l[-1].lower()):
# 		l[0] = re.sub(',+', '', l[0])
# 		result = result+(float(l[0]) * 3.78)
# 	if ("lpm" in l[-1].lower() or "l/min" in l[-1].lower()):
# 		l[0] = re.sub(',+', '', l[0])
# 		result = result+(float(l[0]))

# 	return result


# def water_supply_gpm_calculation(input_data):      # -------------- Add this

# 	pattern_water = "([0-9]+\.?,?[0-9]*)\s*(gpm|lpm|l/min|g/min|l/s)"
# 	ignore_water_word = ["test", "lpm / ft²", "demands"]
   
# 	input = input_data.lower()
# 	sentences = sent_tokenize(input)
# 	#print(sentences)
# 	total = 0

# 	for sentence in sentences:
# 		#print("sentence : ", sentence)
# 		if any(ignore in sentence for ignore in ignore_water_word):
# 			continue
# 		else:
# 			result_water = re.findall(pattern_water, sentence)
# 			#print("result_water : ", result_water)
# 			for result_d in result_water:
# 				result_d =  list(result_d)
# 				#print("result_d : ", result_d)
# 				total = total+gpm_lpm_conversion(result_d)
# 				#print("Each loop total : ", total)
# 	return total

# Below is the code which incorporat l/s logic.  If anything is going wrong uncomment the above code and comment the below code
def gpm_lpm_conversion(l):      # -------------- Add this
	result = 0
	if ("gpm" in l[-1].lower() or "g/min" in l[-1].lower()):
		l[0] = re.sub(',+', '', l[0])
		result = result+(float(l[0]) * 3.78)
	if ("lpm" in l[-1].lower() or "l/min" in l[-1].lower()):
		l[0] = re.sub(',+', '', l[0])
		result = result+(float(l[0]))
	if ("l/s" in l[-1].lower() or "l/sec" in l[-1].lower()):
		l[0] = re.sub(',+', '', l[0])
		result = result+(float(l[0]) * 60)
	return result


def water_supply_gpm_calculation(input_data):      # -------------- Add this

	pattern_water = "([0-9]+\.?,?[0-9]*)\s*(gpm|lpm|l/min|g/min|l/s|l/sec)"
	ignore_water_word = ["test", "lpm / ft²", "demands"]
   
	input = input_data.lower()
	sentences = sent_tokenize(input)
	#print(sentences)
	total = 0

	for sentence in sentences:
		#print("sentence : ", sentence)
		if any(ignore in sentence for ignore in ignore_water_word):
			continue
		else:
			result_water = re.findall(pattern_water, sentence)
			print("result_water : ", result_water)
			for result_d in result_water:
				result_d =  list(result_d)
				print("result_d : ", result_d)
				total = total+gpm_lpm_conversion(result_d)
				print("Each loop total : ", total)
	return total

  
match_AIG_Sprinklers = [
['Building Name', 'Floor Area (1000 sq. m.)', '% AS', '% Adeq. AS', '% ASN', '% Detection', 'Remarks'], 
['Building Name', 'Floor Area(1000 sq. m.)', '% AS', '% Adeq. AS', '% ASN', '% Detection', 'Remarks'],
['Building Name', 'Floor Area (1000 sq.ft.)', '%AS', '% Adeq AS', '% ASN', '% Detection', 'Remarks'],
['Building Name', 'Floor Area (1000 sq.mt.)', '%AS', '% Adeq AS', '% ASN', '% Detection', 'Remarks'],
['Building Name', 'Floor Area (1000 sq.mt.)', '%AS', '% Adeq AS', '% ASN', '% Detection', 'Remarks'],
['Location Group Name', 'Floor Area (1000 sq. m.)', '% AS', '% Adeq. AS', '% ASN', '% Detection']
]
specific_Sprinklers_words = ['% AS', '%AS', '% ASN', '% Detection']
temp_sprinkler = {'% AS': '', '% ASN': '', '% Detection': ''}
sprinkler_spread_flag = False

result_discription = ""

spread_table_fire = []
no_spread_table_fire = []
color_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
color_sprinkler_result = {"Sprinklers":'', 'Sprinklers Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
color_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Detection Rating":''}
color_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''}
color_fire_result_flag = False      # -------------- Add this 


public_keyword = ["public fire department", "public fire brigade", "public fire-brigade", "public fire-department"]
private_keyword = []
municipal_keyword = ["municipal"]
volunteer_keyword = ["volunteer", "low resources", "volunteer fire department", "volunteer fire service"]
paid_keyword = ["paid"]
full_keyword = ["manned 24/7", "full time", "full-time", "24/7", "7/24", "permanently manned"]
fire_only_keyword = ["fire brigade", "fire-brigade", "fire fighters", "fire fighter", "fire station", "fire department", "fire-department", "station", "fire service"]


spread_table_water = []
no_spread_table_water = []
color_water_result_flag = False
match_AIG_Water_Supply = [
['Fire Pump Data', '', '', '', '', ''], 
['Pump Name', 'Pump Driver Type', 'Nameplate Data', '', '', 'Suction Source'],
['Pump Name', 'Pump Driver Type', 'Nameplate Data', '', '', ''],
['Pump Name', 'Pump Driver Type', 'Pump Data Nameplate Data', '', '', ''],
['', '', 'Rated Flow(gpm)', 'Rated Pressure(psi)', 'Rated Speed(rpm)', 'Suction Source'],
['', '', 'Rated Flow(lpm)', 'Rated Pressure(bar)', 'Rated Speed(rpm)', ''],
['', '', 'Rated Flow (L/min)', 'Rated Press. (KPA)', 'Rated Speed (rpm)', '']
]
specific_gpm_words = ["Rated Flow (gpm)", "Rated Flow(gpm)", "Rated Flow (lpm)", "Rated Flow(lpm)", 'Rated Flow (L/min)']

CALCULATED__ = ""   # --------------------- Changes for form recognizer fail

fire_text_data = ""             # -------------- Add this 
water_text_data = ""            # -------------- Add this 
sprinkler_text_data = ""             # -------------- Add this 
detection_text_data = ""            # -------------- Add this

# --------------------- Start Adeq / Not adeq calculation only for AIG-----------------------------


bold_list=['bold','narrow']

def get_row_wise_boxes(bboxes):
    i = 0  # for looping over all the bounding boxes present
    j = 1  # for checking the next box is in same row as the previous box
    box = []  # temp variable for storing the collected boxes of the same row
    row_list = []  # list of list for row wise boxes
    # sorting with x2 and y2 coordinates because we can consider the corner point of bounding box as base
    # line for a row.
    bboxes = sorted(bboxes, key=lambda k: [k[4], k[3]])  # sorting based on ymin coordinate
    while i < len(bboxes) and j < len(bboxes):  # boundary condition for the loop
        # approximating error distance in Y-AXIS direction for boxes
        # this threshold for considering a box in same row I have considered some approximation around 15
        if abs(bboxes[j][4] - bboxes[i][4]) <= 15:
            # present in a row.
            # we go on adding the boxes which are in same row to this lis
            box.append(bboxes[j - 1])
            j += 1
        else:  # if the box is not present in the same row.
            # we append the last box which stored in the box
            box.append(bboxes[j - 1])
            row_list.append(box)  # append to all the row boxes to another list
            # the pointer should go to the box which will come in the next row.
            i += len(box)
            j = j + 1  # increasing the pointer counter of the box
            box = []  # emptying the temporary variable box list.
    # at the end the of loop after exiting every page last combined list will be there in the list
    if j - 1 < len(bboxes):
        box.append(bboxes[j - 1])  # append that last box to the list
    if box:  # checking for if box is not empty then only appending the whole row boxes to row_list variable
        row_list.append(box)
    return row_list


# In[368]:


def get_orderedbox(textnbox):
    raw_boxes=[]
    for block in textnbox['blocks']:
        block_text=''
        if 'lines' in block:
            for span in block['lines']:
                block_text+=span['spans'][0]['text']
        bbox=block['bbox']
        raw_box=[block_text,bbox[0],bbox[1],bbox[2],bbox[3],block]
        raw_boxes.append(raw_box)
    row_wise_boxes=get_row_wise_boxes(raw_boxes)
    final_sorted_list=[]
    for row in row_wise_boxes:
        final_sorted_list.extend(sorted(row, key=lambda x: x[1]))  # sort each individual row based xmin coordinate
    return final_sorted_list




def get_section_heading(ordered_boxes):
    font_sizes=[]
    font_dict={}
    for page_num,value in ordered_boxes.items():
        for item in value:
            block=item[5]
            if 'lines' in block:
                for span in block['lines']:
                    text=span['spans'][0]['text']
                    font_size=span['spans'][0]['size']
                    if 9 < font_size < 25:
                        font_sizes.append(font_size)
                        if font_size in font_dict :
                            font_dict[font_size][0]+=1
                            font_dict[font_size][1].append(page_num)

                        else:
                            font_dict[font_size]=[1,[page_num]]

    for key,value in font_dict.items():
        value[1]=len(list(set(value[1])))
    font_dict_pro={}
    for i in sorted (font_dict):
        font_dict_pro[i]=font_dict[i]
        if font_dict[i][0] > 60:
            font_dict_pro={}


    font_dict_pro= {k:v[1] for (k,v) in font_dict_pro.items()}
    section_heading_size=max(font_dict_pro, key=font_dict_pro.get)
    #print(section_heading_size)
    for page_num,value in ordered_boxes.items():
        for item in value:
            block=item[5]
            if 'lines' in block:
                for span in block['lines']:
                    text=span['spans'][0]['text']
                    font_size=span['spans'][0]['size']
                    if round(section_heading_size)==round(font_size):
                        #print(text)
                        pass
    return section_heading_size


def parse_box(ordered_boxes):

    parsed=OrderedDict()
    scanned_text=''
    heading=''
    for page_num,value in ordered_boxes.items():
        table_areas=line_clusters[page_num]
        for item in value:
            block=item[5]
            block_text=''
            if 'lines' in block:
                for spans in block['lines']:
                    for span in spans['spans']:
                        table_block=False
                        text=span['text']
                        font_size=span['size']
                        ycord=span['bbox'][1]
                        bold=True if any([i in  span['font'].lower() for i in bold_list]) else False
                        for ymin,ymax in table_areas:
                            if ymin< ycord<ymax:
                                table_block=True
                        if table_block:
                            continue
                        scanned_text+=text
                   

                        if bold:
                            parsed[heading]=[0,scanned_text]
                            heading=text
                            scanned_text=''
    return parsed


fire_brigade_bag=["Public Fire Department"]
sprinkler_bag=["Automatic Sprinklers"]
water_supply_bag=["Water Supply, Water Mains, and Fire Hydrants"]
alarm_bag=["Alarm Systems"]

fire_text_data_backup = ""             # -------------- Add this 
water_text_data_backup = ""            # -------------- Add this 
sprinkler_text_data_backup = ""             # -------------- Add this 
detection_text_data_backup = ""            # -------------- Add this 

def get_line_clusters(doc):

    line_clusters={}
    for page_num,page in enumerate(doc):
        page_num+=1
        drawings=page.get_drawings()
        ycords,underlines= get_lines(drawings)
        cluster_master=get_cluster(ycords)
        line_clusters[page_num]=cluster_master
    return line_clusters



def get_lines(drawings):
    ycords=[]
    underlines=[]
    for path in drawings:
        for item in path['items']:
            if item[0]=='l':
                ycord=item[1][1]
                length=item[2][0]-item[1][0]
                if length> 50:
                    ycords.append(ycord)

            elif item[0]=='re':
                ycord1=item[1][1]
                ycord2=item[1][3]
                length=item[1][2]-item[1][0]
                if length >50:
                    ycords.extend([ycord1,ycord2])
                else:
                    underlines.append((ycord1,length))
    return ycords,underlines


def get_cluster(ycords):
    ycords.sort()
    cluster_master=[]
    cluster=[]
    for pos in ycords:
        pos=round(pos)
        if cluster:
            if pos-cluster[-1] <20:
                cluster.append(pos)
            else:
                cluster_master.append(cluster)
                cluster=[]

        else:
            cluster.append(pos)
    cluster_master.append(cluster)
    cluster_master=[list(set(i)) for i in cluster_master]
    cluster_master=[(min(i),max(i)) for i in cluster_master if len(i)>1]
    return cluster_master


# adequate_keyword = ["adequately positioned", "adequately distributed", "adequate"]
# not_adequate_keyword = ["inadequate", "nearly adequate", "not adequate"]

# def adequate_finder(text, adequate_keyword, not_adequate_keyword):

# 	text = text.lower()
# 	#print("text : ",text)

# 	result_adeq_not_adeq = {'adeq/not':""}

# 	# Adequate / Not Adequate Calculation
# 	if any(not_adequate_key in text for not_adequate_key in not_adequate_keyword):
# 		print("Not adequate")
# 		result_adeq_not_adeq['adeq/not'] = "Not Adequate"
# 		return result_adeq_not_adeq
# 	elif any(adequate_key in text for adequate_key in adequate_keyword):
# 		print("Adequate")
# 		result_adeq_not_adeq['adeq/not'] = "Adequate"
# 		return result_adeq_not_adeq
# 	else:
# 		print("No clear information given")
# 	#print("result_adeq_not_adeq, result_adeq_not_adeq_org :", result_adeq_not_adeq, result_adeq_not_adeq_org)
# 	return result_adeq_not_adeq



#files = os.listdir("inputFolder_PDF")
file=path

#for file in files:
if file != "" and carrier_name == 'AIG':
    file_path=file
    print("File : ", file)
    doc = fitz.open(file_path)
    line_clusters=get_line_clusters(doc)
    html=''
    ordered_boxes={}
    for page_num,page in enumerate(doc):
        page_num+=1
        html+=page.getText('html')
        textnbox=page.getText('dict')
        ordered_box=get_orderedbox(textnbox)
        ordered_boxes[page_num]=ordered_box
    section_heqading_size=get_section_heading(ordered_boxes)
    parsed=parse_box(ordered_boxes)


    #star_pattern = '(\.+\s+[a-zA-Z]+[/*\s*]+$)'
    #fire_stoppers = ['WATER SUPPLY', 'Sprinklers', 'Special Extinguishing Protection', 'Page']

    
    for key, value in parsed.items():
        #if any([ bow in key.lower() for bow in fire_brigade_bag]):
        if any([ bow in key for bow in fire_brigade_bag]):
            #data = re.sub("") 
            fire_text_data_backup = value[1]      # -------------- Add this        
            #print(key,'\n\n',fire_text_data)


    for key, value in parsed.items():
        #if any([ bow in key.lower() for bow in sprinkler_bag]):
        if any([ bow in key for bow in sprinkler_bag]):
            sprinkler_text_data_backup = value[1]        
            #print(key,'\n',sprinkler_text_data)      # -------------- Add this 
            #result_adeq_not_adeq = adequate_finder(value[1], adequate_keyword, not_adequate_keyword)
            #print("result_adeq_not_adeq Sprinkler Adeq/Not : ",result_adeq_not_adeq)
            #if result_adeq_not_adeq['adeq/not'] != '':
             #   color_sprinkler_result['Coverage is adequate'] = ""
            #else:
            #    color_sprinkler_result['Coverage is adequate'] = " "

    #print()



    for key, value in parsed.items():
        #if any([ bow in key.lower() for bow in water_supply_bag]): 
        if any([ bow in key for bow in water_supply_bag]):
            water_text_data_backup = value[1]      # -------------- Add this        
           #print(key,'\n',water_text_data)

    #print()

    for key, value in parsed.items():
        #if any([ bow in key.lower() for bow in alarm_bag]):
        if any([ bow in key for bow in alarm_bag]): 
            detection_text_data_backup = value[1]      # -------------- Add this        
            #print(key,'\n\n',detection_text_data)
            #result_adeq_not_adeq = adequate_finder(value[1], adequate_keyword, not_adequate_keyword)
            #print("result_adeq_not_adeq Detection Adeq/Not : ",result_adeq_not_adeq)
            #if result_adeq_not_adeq['adeq/not'] != '':
            #    color_detection_result['Coverage is adequate'] = " "#result_adeq_not_adeq['adeq/not']
            #else:
            #    color_detection_result['Coverage is adequate'] = " "

    

# --------------------- End Adeq / Not adeq calculation only for AIG -----------------------------

#carrier_name = 'AIG'


# if carrier_name == 'AIG':
# 	for file in Images_list:
# 		patch_work(file) 
# 		data_extraction(file)
# 		print("FileName : ",file)
# 		try:
# 			results_list = frame_creation(file)
# 			#print("results_list : ",results_list)
# 			for result in results_list:
# 				total_gpm, unit = find_relevent_table_water(result, match_AIG_Water_Supply[1:], specific_gpm_words)
# 				total_flow = sum(total_gpm)
# 				print("total_flow : ", total_flow)
# 				if total_flow > 0 and color_water_result['Water Supply Flow'] == "":
# 					color_water_result['Water Supplied'] = "Yes"
# 					if ("gpm" in unit.lower()):
# 						print("Total Flow : ", total_flow ," gpm")
# 						total_flow = total_flow * 3.78
# 						print("Total Flow : ", total_flow ," lpm")
# 						color_water_result['Water Supply Flow'] = str(total_flow)+" lpm"
# 					elif ("lpm" in unit.lower()) or ("l/min" in unit.lower()):
# 						print("Total Flow : ", total_flow ," lpm")
# 						color_water_result['Water Supply Flow'] = str(total_flow)+" lpm"
# 					else:
# 						color_water_result['Water Supplied'] = "No"
# 				print("Unit : ", unit)
# 				if unit != "":			
# 					print("Unit : ", unit)

# 				temp_sprinkler = find_relevent_table_sprinkler(result, match_AIG_Sprinklers[1:], specific_Sprinklers_words, sprinkler_spread_flag)      # -------------- Add this 
# 				print("Result Sprinkler first round 1 : ", temp_sprinkler)
# 				if temp_sprinkler['% AS'] !="" or temp_sprinkler['% ASN'] !="" or temp_sprinkler['% Detection'] !="":
# 					color_sprinkler_result['Coverage Provided'] = temp_sprinkler['% AS']
# 					color_sprinkler_result['Extra coverage required'] = temp_sprinkler['% ASN']
# 					color_detection_result['Coverage Provided'] = temp_sprinkler['% Detection']
                    
# 				if temp_sprinkler['% AS'] == 3 or temp_sprinkler['% ASN'] == 5 or temp_sprinkler['% Detection'] == 6:      # -------------- Add this 
# 					sprinkler_spread_flag = True       # -------------- Add this 
                    
# 				score = find_relevent_AIG_FireBrigade(result)
# 				results = [[i for i in row] for row in result.itertuples()]
# 				#print("results : ",results)
# 				#print("len(results) : ", results)
# 				for spread in results:
# 					if (ratio(spread[1], "Water Supply") > 90):
# 						spread_table_water.append(spread)
# 					if (ratio(spread[1], "Automatic Fire Detection") > 90 or ratio(spread[1], "Security") > 90) and len(results) > 0:
# 						spread_table_fire.append(spread)
# 					if (ratio(spread[1], "Sprinklers") > 90 or ratio(spread[1], "Special Extinguishing Protection") > 90 or ratio(spread[1], "Fire Extinguishing Appliances/Hosereels") > 90 or ratio(spread[1], "Public Fire Department") > 90 or ratio(spread[1], "Smoke Extraction") > 90) and len(results) < 5:
# 						spread_table_fire.append(spread)
# 					if (ratio(spread[1], "Sprinklers") > 90 or ratio(spread[1], "Special Extinguishing Protection") > 90 or ratio(spread[1], "Fire Extinguishing Appliances/Hosereels") > 90 or ratio(spread[1], "Public Fire Department") > 90 or ratio(spread[1], "Smoke Extraction") > 90) and len(results) == 5 or len(results) == 6 and score == 0:
# 						#print("Single page result : ",results)
# 						no_spread_table_fire = results
# 						break
# 		except UnboundLocalError as u:
# 			traceback.print_exc()

# 		#print("len(spread_table_fire) : ",len(spread_table_fire))

# 		if (len(spread_table_fire) > 0) and (len(spread_table_fire) < 4) :
# 			top_patch(file)
# 			data_extraction(file)
# 			print("FileName : ",file)
# 			try:
# 				results_list = frame_creation(file)
# 				for result in results_list:
# 					if temp_sprinkler['% AS'] =="" or temp_sprinkler['% ASN'] =="" or temp_sprinkler['% Detection'] =="":
# 						temp_sprinkler = find_relevent_table_sprinkler(result, match_AIG_Sprinklers[1:], specific_Sprinklers_words, sprinkler_spread_flag)      # -------------- Add this 
# 						print("Result Sprinkler first round 2 : ", temp_sprinkler)
# 					if temp_sprinkler['% AS'] !="" and temp_sprinkler['% ASN'] !="" and temp_sprinkler['% Detection'] !="":
# 						color_sprinkler_result['Coverage Provided'] = temp_sprinkler['% AS']
# 						color_sprinkler_result['Extra coverage required'] = temp_sprinkler['% ASN']
# 						color_detection_result['Coverage Provided'] = temp_sprinkler['% Detection']
                        
# 					if temp_sprinkler['% AS'] == 3 or temp_sprinkler['% ASN'] == 5 or temp_sprinkler['% Detection'] == 6:      # -------------- Add this 
# 						sprinkler_spread_flag = True       # -------------- Add this 
                        
# 					score = find_relevent_AIG_FireBrigade(result)
# 					results = [[i for i in row] for row in result.itertuples()]
# 					for spread in results:
# 						if (ratio(spread[1], "Sprinklers") > 90 or ratio(spread[1], "Special Extinguishing Protection") > 90 or ratio(spread[1], "Fire Extinguishing Appliances/Hosereels") > 90 or ratio(spread[1], "Public Fire Department") > 90 or ratio(spread[1], "Smoke Extraction") > 90) and len(results) < 5:	
# 							if spread not in spread_table_fire:
# 								spread_table_fire.append(spread)
# 			except UnboundLocalError as u:
# 				traceback.print_exc()

# 		#print("spread_table_fire : ",spread_table_fire)
# 		if len(spread_table_fire) <= 4 :
# 			#print("Multi page result : ",spread_table_fire)
# 			for color in spread_table_fire:
# 				for color_fire_count,color_fire_data in enumerate(color):
# 					if color_fire_data == "Sprinklers"  or color_fire_data == "S prinklers":
# 						color_result = [x for x in color if x]
# 						color_sprinkler_result['Sprinklers'] = "".join(color_result[2:])
# 					if color_fire_data == "Public Fire Department":
# 						color_result = [x for x in color if x]
# 						color_fire_result['Public Fire Department'] = "".join(color_result[2:])


# 		if len(spread_table_fire) >= 1 :
# 			#print("Multi page result : ",spread_table_fire)
# 			for color in spread_table_fire:
# 				for color_fire_count,color_fire_data in enumerate(color):
# 					if color_fire_data == "Automatic Fire Detection":
# 						color_result = [x for x in color if x]
# 						color_detection_result['Detection'] = "".join(color_result[2:])


# 			print()
# 			spread_table_fire = []

# 		if len(spread_table_water) > 0 :
# 			#print("Multi page result : ",spread_table_water)
# 			for color in spread_table_water:
# 				for color_water_count,color_water_data in enumerate(color):
# 					if color_water_data == "Water Supply":
# 						color_result = [x for x in color if x]
# 						color_water_result['Water Supply'] = "".join(color_result[2:])
# 			spread_table_water = []

# 		#print("no_spread_table_fire : ", no_spread_table_fire)
# 		for color in no_spread_table_fire:
# 			for color_fire_count,color_fire_data in enumerate(color):
# 				if color_fire_data == "Public Fire Department":
# 					color_result = [x for x in color if x]
# 					#print("color_result : ", color_result)
# 					color_fire_report = "".join(color_result[2:])
# 					color_fire_result['Public Fire Department'] = color_fire_report
# 				if color_fire_data == "Sprinklers" or color_fire_data == "S prinklers":
# 					color_result = [x for x in color if x]
# 					#print("color_result : ", color_result)
# 					color_sprinklers_report = "".join(color_result[2:])
# 					color_sprinkler_result['Sprinklers'] = color_sprinklers_report



# 		# Masking process and text extraction

# 		#frame_creation_mask(file)
# 		if temp_sprinkler['% AS'] =="" or temp_sprinkler['% ASN'] =="" or temp_sprinkler['% Detection'] =="":
# 			maskImage(file)
# 			data_extraction(file)
# 			results_list = frame_creation(file)
# 			#print("results_list : ", results_list)
# 			for result in results_list:
# 				temp_sprinkler = find_relevent_table_sprinkler(result, match_AIG_Sprinklers[1:], specific_Sprinklers_words, sprinkler_spread_flag)      # -------------- Add this 
# 				print("Result Sprinkler second round : ",temp_sprinkler)
# 			if temp_sprinkler['% AS'] !="" and temp_sprinkler['% ASN'] !="" and temp_sprinkler['% Detection'] !="":
# 				color_sprinkler_result['Coverage Provided'] = temp_sprinkler['% AS']
# 				color_sprinkler_result['Extra coverage required'] = temp_sprinkler['% ASN']
# 				color_detection_result['Coverage Provided'] = temp_sprinkler['% Detection']
                
# 			if temp_sprinkler['% AS'] == 3 or temp_sprinkler['% ASN'] == 5 or temp_sprinkler['% Detection'] == 6:      # -------------- Add this 
# 				sprinkler_spread_flag = True       # -------------- Add this 

# 		print()

# 		#print("color_detection_result : ", color_detection_result)

# 		# Detection operation
# 		#color_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":""}
# 		if color_detection_result['Coverage Provided'] != '' :
# 			try:
# 				temp_detection_cover = re.sub("\s*%", '', color_detection_result['Coverage Provided'])
# 			except:
# 				color_sprinkler_result['Coverage Provided'] = ""
# 				color_sprinkler_result['Extra coverage required'] = ""
# 				color_detection_result['Coverage Provided'] = ""
# 				pass
# 			try:
# 				detection_coverage = float(temp_detection_cover)
# 				if detection_coverage > 0:
# 					color_detection_result['Detection Installed'] = "Yes"
# 				else:
# 					color_detection_result['Detection Installed'] = "No"
# 			except:
# 				print("Form recognizer failed on detection table")
# 				pass
# 		else:
# 			print("No data captured for detection")

# 		#print("color_detection_result : ", color_detection_result)
# 		#print("color_fire_result : ", color_fire_result)
# 		#print("color_sprinkler_result : ", color_sprinkler_result)


# 		# Sprinkler opereation
# 		# color_sprinkler_result = {"Sprinklers":'', '% Detection':"", 'Sprinklers installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
# 		if color_sprinkler_result['Coverage Provided'] != '' :
# 			try:
# 				temp_sprinkler_cover = re.sub("\s*%", '', color_sprinkler_result['Coverage Provided'])
# 				temp_sprinkler_required_cover = re.sub("\s*%", '', color_sprinkler_result['Extra coverage required'])
# 			except :
# 				color_sprinkler_result['Coverage Provided'] = ""
# 				color_sprinkler_result['Extra coverage required'] = ""
# 				color_detection_result['Coverage Provided'] = ""
# 				pass
# 			try:
# 				sprinkler_coverage = float(temp_sprinkler_cover)
# 				sprinkler_required_cover = float(temp_sprinkler_required_cover)
# 				if sprinkler_coverage > 0:
# 					color_sprinkler_result['Sprinklers Installed'] = "Yes"
# 				else:
# 					color_sprinkler_result['Sprinklers Installed'] = "No"
# 			except:
# 				print("Form recognizer failed on sprinkler table")
# 				pass
# 		else:
# 			print("No data captured for sprinkler")




# 		maskImage(file)
# 		data = text_identification(file)
# 		result, result_list = text_protection(data['line'])
# 		result_discription = result

# 		# New code implemented here
# 		if len(result_list) > 0:
# 			print("result_discription : ",result_discription)
# 			result_dics, result_org = organization_finder(result, public_keyword, private_keyword, volunteer_keyword, paid_keyword, full_keyword)
# 			discription_fire_bridage = discription_fire_bridage_processing(result_discription) # ---- Add this
# 			result_discription = "" # ---- Add this
# 			#print("Result : ", result_dics)
# 			#print()
# 			#print("Result Final Org Type : ", result_org)
# 			if color_fire_result['organization_type'] == "" and len(result_org) > 0 :
# 				color_fire_result['organization_type'] = result_org[0]
# 			else:
# 				color_fire_result['organization_type'] == ""

# 			p = score_calculation_protection_fire(result)	
# 			print("Responce result final : ",p[0])
# 			print("Distance result final : ",p[1])

# 			if color_fire_result['distance'] == "" and p[1] != "":
# 				if isinstance(p[1], list): 
# 					distance_ = convert_To_String(p[1])
# 					color_fire_result['distance'] = distance_
# 				else:
# 					color_fire_result['distance'] = p[1]
# 			if color_fire_result['responce'] == "" and p[0] != "":
# 				if isinstance(p[0], list):
# 					responce_ = convert_To_String(p[0])
# 					color_fire_result['responce'] = responce_
# 				else:
# 					color_fire_result['responce'] = p[0]

# 		#print(color_fire_result)
# 		#print()


# 	color_fire_result = final_rating_calculation(color_fire_result)
# 	#print(color_fire_result)
  
# 	if color_water_result['Water Supply Flow'] == '':      # -------------- Add this
# 		water_flow = water_supply_gpm_calculation(water_text_data)

# 		if water_flow > 0:
# 			color_water_result['Water Supplied'] = "Yes"
# 		else:
# 			color_water_result['Water Supplied'] = "No"
# 		color_water_result['Water Supply Flow'] = str(water_flow) + " lpm"      # -------------- Add this 
    
# 	if color_water_result['Water Supply Flow'] != '':
# 		discription_water_supply = discription_water_supply_processing(water_text_data)      # -------------- Add this
        
# 	discription_fire_sprinkler = discription_fire_sprinkler_processing(sprinkler_text_data)
# 	discription_fire_detection = discription_fire_detection_processing(detection_text_data)
#     #try:
# 		#if (color_fire_result['Public Fire Department'] != "" or color_fire_result['distance'] != "" or color_fire_result['responce'] != "" or color_fire_result['organization_type'] != ""):
# 	#	if (color_fire_result['responce'] != "" or color_fire_result['distance'] != "" or color_fire_result['organization_type'] != "" or color_fire_result['Public Fire Department'] != ""):
# 	#		df_protection_fire_brigade = intermidiet_table_fire_brigade(file, color_fire_result, "output")
# 			#color_fire_result_flag = True
# 	#except IndexError:
# 	#	print("Index Not enough data to create a csv file")
# 	#except KeyError:
# 		#print("Key Not enough data to create a csv file")
# 	#color_fire_result = {}

# 	#intermidiet_table_water_supply, intermidiet_table_sprinkler, intermidiet_table_detection
# 	df_protection_fire_brigade = intermidiet_table_fire_brigade(file, color_fire_result, "output",discription_fire_bridage)
# 	print("color_fire_result : ", color_fire_result)
# 	print(df_protection_fire_brigade)

# 	color_detection_result = final_rating_calculation_fire_detection(color_detection_result)      # -------------- Add this
# 	print("color_detection_result : ", color_detection_result)
# 	df_protection_detection = intermidiet_table_detection(file, color_detection_result, "output",discription_fire_detection)
# 	print(df_protection_detection)

# 	color_sprinkler_result = final_rating_calculation_fire_sprinkler(color_sprinkler_result)      # -------------- Add this
# 	print("color_sprinkler_result : ", color_sprinkler_result)
# 	df_protection_sprinkler = intermidiet_table_sprinkler(file, color_sprinkler_result, "output",discription_fire_sprinkler)
# 	print(df_protection_sprinkler)

# 	print("color_water_result : ", color_water_result)
# 	df_protection_water_supply = intermidiet_table_water_supply(file, color_water_result, "output",discription_water_supply)
# 	print(df_protection_water_supply)
    
#     data = final_UI(discription_fire_bridage, discription_water_supply, discription_fire_detection, discription_fire_sprinkler, chubb_fire_result, chubb_water_result, chubb_sprinkler_result, chubb_detection_result)

#xzxz

File : /dbfs/mnt/aria/AIG_Input/377955 AIG_redacted.pdf
No clear information given
result_adeq_not_adeq Sprinkler Adeq/Not : {'adeq/not': ''}
No clear information given
result_adeq_not_adeq Detection Adeq/Not : {'adeq/not': ''}

In [0]:
def text_fire_chubb(data):
	fire_text_filter_keywords = ['Fire Sprinklers', 'Fire Sprinkler', 'Systems','Other Extinguishing Systems', 'Fire Hydrants', 'Hose Reels & Extinguishers', 'Fire Detection', 'Fire Cut-Offs', 'External Exposures', 'Common Hazards', 'Special Hazards', 'Water Supplies for Fire Fighting', 'Chubb Risk Engineering Services', 'Chubb Virtual Risk Assessment']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Fire Brigade' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Fire Brigade'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in fire_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						return result, result_list
					#print(data[count])
					if ('Business Interrruption' in data[count]):
						water_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

def text_water_chubb(data):
	water_text_filter_keywords = ['Fire Brigade', 'Fire Sprinklers', 'Fire Sprinkler', 'Other Extinguishing Systems', 'Fire Hydrants', 'Hose Reels & Extinguishers', 'Fire Detection', 'Fire Cut-Offs', 'External Exposures', 'Common Hazards', 'Special Hazards', 'Chubb Risk Engineering Services', 'Chubb Virtual Risk Assessment']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Water Supplies for Fire Fighting' in data[line]) or ('Water Supplies' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Water Supplies for Fire Fighting'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in water_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						return result, result_list
					#print(data[count])
					if ('Business Interrruption' in data[count]):
						water_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

def text_sprinkler_chubb(data):
	sprinkler_text_filter_keywords = ['Other Fire', 'Other Extinguishing Systems', 'Fire Hydrants', 'Hose Reels & Extinguishers', 'Fire Detection', 'Fire Cut-Offs', 'External Exposures', 'Common Hazards', 'Special Hazards', 'Water Supplies for Fire Fighting', 'Fire Brigade', 'Chubb Risk Engineering Services', 'Chubb Virtual Risk Assessment']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Fire Sprinklers' in data[line]) or ('Fire Sprinkler' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Fire Sprinklers'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in sprinkler_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						return result, result_list
					#print(data[count])
					if ('Business Interrruption' in data[count]):
						water_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

def text_detection_chubb(data):
	detection_text_filter_keywords = ['Fire Rated', 'Compartments', 'Fire Cut-Offs', 'External Exposures', 'Common Hazards', 'Special Hazards', 'Chubb Risk Engineering Services', 'Chubb Virtual Risk Assessment',  'Water Supplies for Fire Fighting', 'Fire Sprinklers', 'Other Extinguishing Systems', 'Fire Hydrants', 'Hose Reels & Extinguishers', 'Fire Brigade']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Fire Detection' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Fire Detection') or (data[count] == 'Fire Detection /'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in detection_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						return result, result_list
					#print(data[count])
					if ('Business Interrruption' in data[count]):
						water_flag = True
						#print("Result : ", result)
						#print("Result List : ", result_list)
						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		return result, result_list

In [0]:
def text_fire_grc(data):
	fire_text_filter_keywords = ['Natural Hazards and Other Exposures/Perils', 'Security Measures']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Public Protection' in data[line]) or ('Public Fire Service' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Public Protection') or (data[count] == 'Public Fire Service'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in fire_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						result = re.sub("Public Fire Service:", "", result)
						return result, result_list
					#print(data[count])
# 					if ('Business Interrruption' in data[count]):
# 						water_flag = True
# 						#print("Result : ", result)
# 						#print("Result List : ", result_list)
# 						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		result = re.sub("Public Fire Service:", "", result)        
		return result, result_list

def text_water_grc(data):
	water_text_filter_keywords = ['Alarm Systems', 'Other Fixed Fire Protection', 'Fixed Fire Suppression Systems',
                                  'Fixed Fire Protection', 'Pier,Coking Plant', 'Blast Furnace Plant',
                                  'Internal Fire Response', 'Public Fire Service',
                                  'Carbon Dioxide Special Extinguishing System', 'Security Measures',
                                  'Manual Protection', 'Management Programs']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Water Supply, Water Mains, and Fire Hydrants' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Water Supplies for Fire Fighting'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in water_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						result = re.sub("Water Supplies for Fire Fighting:", "", result)
						return result, result_list
					#print(data[count])
# 					if ('Business Interrruption' in data[count]):
# 						water_flag = True
# 						#print("Result : ", result)
# 						#print("Result List : ", result_list)
# 						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		result = re.sub("Water Supplies for Fire Fighting:", "", result)        
		return result, result_list

def text_sprinkler_grc(data):
	sprinkler_text_filter_keywords = ['Water Supply, Water Mains, and Fire Hydrants', 'Alarm Systems', 'Security Measures', 'Manual Protection', 'Management Programs','Plant Emergency Organization', 'Pier', 'Coking Plant']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Automatic Sprinklers' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Automatic Sprinklers'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in sprinkler_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						result = re.sub("Automatic Sprinklers:", "", result)
						return result, result_list
					#print(data[count])
# 					if ('Business Interrruption' in data[count]):
# 						water_flag = True
# 						#print("Result : ", result)
# 						#print("Result List : ", result_list)
# 						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		result = re.sub("Automatic Sprinklers:", "", result)        
		return result, result_list

def text_detection_grc(data):
	detection_text_filter_keywords = ['Security Measures','Manual Protection','Management Programs','Property Surveillance','Portable Fire Equipment','Special Extinguishing Systems','Internal Fire Response',
'Carbon Dioxide Special Extinguishing System','Security Measures','Manual Protection']
	result = ""	
	result_list = []
	water_flag = False
	fire_flag = False
	for line in range(len(data)) :
		#print(data[line])
		#print()
		if ('Alarm Systems' in data[line]):
			#print(data[line])
			fire_flag = True
			count = line
			while count < len(data):
				if (fire_flag == True):
					#print(data[count])
					if (data[count] == 'Alarm Systems'): 
						pass
					else:
						result = result+data[count]
						result = result+" "
						result_list.append(data[count])
				count = count+1
				try:
					
					if (any(fire_key in data[count] for fire_key in detection_text_filter_keywords) and (fire_flag == True)):
						print("No Append")
						result = re.sub("Alarm Systems:", "", result)
						return result, result_list
					#print(data[count])
# 					if ('Business Interrruption' in data[count]):
# 						water_flag = True
# 						#print("Result : ", result)
# 						#print("Result List : ", result_list)
# 						return result, result_list
				except IndexError as e:
					pass
				
	if water_flag == False:
		#print("Result : ", result)
		#print("Result List : ", result_list)
		result = re.sub("Alarm Systems:", "", result)
		return result, result_list
      

In [0]:
def recursion_rating_chubb(rating_buffer):
	#rating_buffer = ['A','SD','SD']
	while len(rating_buffer) != 1:
		try:
			rating_buffer.remove('A')
		except ValueError:
			#print("More than one rating present. Look manually.")
			return rating_buffer
	#print(rating_buffer)
	return rating_buffer


def rating_finder_chubb(data):
	rating = ['AA', 'AAA', 'A', 'NI', 'SD']
	result_rating_buffer = []
	words = word_tokenize(data)
	#print(words)
	#words = recursion_rating_chubb(words)
	#print(words)
	for rate in rating:
		for word in words:
			if rate == word:
				#print("Found : ", rate)
				result_rating_buffer.append(rate)
	else:
		#print("Not found")
		pass
	result_rating_buffer = recursion_rating_chubb(result_rating_buffer)
	if len(result_rating_buffer) > 1:
		print("More than one rating present. Look manually.")
	else:
		return result_rating_buffer[0]
      
#data = """Alexandria Fire Brigade is located approximately 1.2km north east of AAA the site by road. Several alternative fire stations are located within 10km of the site. """      
#result = rating_finder_chubb(data)
#print("result : ",result)

In [0]:
# ----------------------------------- Below is Version 1 code -----------------------------------------------------------------

# def detection_sprinkler_coverage_processing(input_data):
#     pattern_detection_sprinkler = "([0-9]+\.?,?[0-9]*)\s*(%)"
#     detection_sprinkler_coverage_word = ["installed throughout",
#                                          "fully covered", "full coverage provided", "partly covered",
#                                          "coverage is partly coverage", "most areas are protected",
#                                          "no extra coverage required", "no smoke detection", "not applicable"]
#     # detection_sprinkler_installed_word = ["installed", "installations"]
#     # detection_sprinkler_not_installed_word = ["not installed"]
#     # install_flag = False
#     # not_install_flag = False

#     result_coverage = ""
#     detection_sprinkler_result_coverage_sentence = ""

#     input = input_data.lower()
#     sentences = sent_tokenize(input)
#     # print(sentences)

#     for sentence in sentences:
#         # print("sentence : ", sentence)
#         for detection_sprinkler in detection_sprinkler_coverage_word:
#             if detection_sprinkler in sentence:
#                 # print("Match coverage : ", detection_sprinkler)
#                 result_coverage = detection_sprinkler
#                 detection_sprinkler_result_coverage_sentence = detection_sprinkler_result_coverage_sentence + sentence
#             # print("Match coverage sentence: ", detection_sprinkler_result)
#         result_detection_sprinkler = re.findall(pattern_detection_sprinkler, sentence)
#         if bool(result_detection_sprinkler):
#             print("result_detection_sprinkler : ", result_detection_sprinkler)
#         for result_d in result_detection_sprinkler:
#             result_d = list(result_coverage)
#             print("result_d : ", result_coverage)
#     return result_coverage, detection_sprinkler_result_coverage_sentence


# def detection_sprinkler_installation_processing(input_data):
#     detection_sprinkler_installed_word = ["installed", "installations", "automatic deluge systems",
#                                           "automatic fire suppression system", "sprinkler system is provided",
#                                           "sprinklers installed", "sprinkler system installed",
#                                           "fire water distribution & hydrants average", "coverage completely",
#                                           "areas are protected with automatic sprinkle", "protected with sprinklers",
#                                           "automatic fire detection systems", "existing fire detection",
#                                           "detection provided"]
#     detection_sprinkler_not_installed_word = ["not installed", "not reported", "sprinklers none",
#                                               "no sprinkler system provided", "no automatic",
#                                               "no automatic fire suppression", "fire detection none", "detection none",
#                                               "no detection system provided", "no automatic",
#                                               "no automatic fire detection","none"]

#     install_flag = False
#     not_install_flag = False


#     install_check = " "
#     result_installation = ""
#     detection_sprinkler_result_installation_sentence = ""
    
#     input = input_data.lower()
#     sentences = sent_tokenize(input)
#     # print(sentences)

#     for sentence in sentences:
#         # print("sentence : ", sentence)
#         for detection_sprinkler in detection_sprinkler_not_installed_word:
#             if (detection_sprinkler in sentence):
#                 # print("Match : ", detection_sprinkler)
#                 result_installation = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
#                 not_install_flag = True
#             # return detection_sprinkler
#             if (detection_sprinkler in sentence) and (not_install_flag == False):
#                 # print("Match : ", detection_sprinkler)
#                 result_installation = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
#                 not_install_flag = True
#             # return detection_sprinkler
#         for detection_sprinkler in detection_sprinkler_installed_word:
#             if (detection_sprinkler in sentence) and ("sprinkler protection" in sentence) and (not_install_flag == False):
#                 # print("Match : ", detection_sprinkler)
#                 result_installation = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
#                 install_flag = True
#             # return detection_sprinkler
#             if (detection_sprinkler in sentence) and (install_flag == False) and (not_install_flag == False):
#                 # print("Match : ", detection_sprinkler)
#                 result_installation = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
#                 install_flag = True
#             # return detection_sprinkler
#         # print("install_flag : ", install_flag)

#     if install_flag == True:  # ---------------- See this logic in actual implemented code
#         install_check = "Yes"  # ---------------- See this logic in actual implemented code
#     elif not_install_flag == True:  # ---------------- See this logic in actual implemented code
#         install_check = "No"  # ---------------- See this logic in actual implemented code
#     else:  # ---------------- See this logic in actual implemented code
#         print("Can't determine installed or not")  # ---------------- See this logic in actual implemented code   

#     return result_installation, detection_sprinkler_result_installation_sentence, install_check

# ----------------------------------- Below is Version 2 code -----------------------------------------------------------------

def detection_sprinkler_coverage_processing(input_data):
    pattern_detection_sprinkler = "([0-9]+\.?,?[0-9]*)\s*(%)"
    detection_sprinkler_coverage_word = ["installed throughout", "not required in other areas", "fully sprinkler protected", 
                                         "fully covered", "full coverage provided", "partly covered",
                                         "coverage is partly coverage", "most areas are protected",
                                         "no extra coverage required", "no smoke detection", "system is through", "sprinkler supply", 
                                         "installed", "combined sprinkler and hydrant system installed", "no smoke detection", 
                                         "installed throughout the site", "installed", "not installed", "scattered fire alarms", 
                                         "fire detection throughout"]
    detection_sprinkler_extra_coverage_word = ["no extra coverage required", "not applicable"]
    full = ["installed throughout", "not required in other areas","fully covered", "full coverage provided","installed throughout the site", "fire detection throughout"]
    partial = ["partly covered", "coverage is partly coverage", "most areas are protected", "combined sprinkler and hydrant system installed",
    "scattered fire alarms", "installed", "not installed"]
    # detection_sprinkler_installed_word = ["installed", "installations"]
    # detection_sprinkler_not_installed_word = ["not installed"]
    # install_flag = False
    # not_install_flag = False

    result_coverage = ""
    detection_sprinkler_result_coverage_sentence = ""

    input = input_data.lower()
    sentences = sent_tokenize(input)
    # print(sentences)

    for sentence in sentences:
        # print("sentence : ", sentence)
        for detection_sprinkler in detection_sprinkler_coverage_word:
            if detection_sprinkler in sentence:
                # print("Match coverage : ", detection_sprinkler)
                result_coverage = detection_sprinkler
                detection_sprinkler_result_coverage_sentence = detection_sprinkler_result_coverage_sentence + sentence
            # print("Match coverage sentence: ", detection_sprinkler_result)
        for no_detection_sprinkler in detection_sprinkler_extra_coverage_word:
            if no_detection_sprinkler in sentence:
                # print("Match coverage : ", detection_sprinkler)
                result_coverage = no_detection_sprinkler
                detection_sprinkler_result_coverage_sentence = detection_sprinkler_result_coverage_sentence + sentence
            # print("Match coverage sentence: ", detection_sprinkler_result)
        result_detection_sprinkler = re.findall(pattern_detection_sprinkler, sentence)
        if bool(result_detection_sprinkler):
            print("result_detection_sprinkler : ", result_detection_sprinkler)
        for result_d in result_detection_sprinkler:
            result_d = list(result_coverage)
            #print("result_d : ", result_coverage)
    if any(full_key in result_coverage for full_key in full):
        result_coverage = "full coverage provided"
    if any(partial_key in result_coverage for partial_key in partial):
        result_coverage = "partial covered"
    return result_coverage, detection_sprinkler_result_coverage_sentence


def detection_sprinkler_installation_processing(input_data):
    detection_sprinkler_installed_word = ["installed", "installations", "sprinkler protection is provided", "fully sprinkler protected", "coverage completely", 
    "automatic deluge systems","automatic fire suppression system","sprinkler system is provided", "fire alarms are generally installed", "detection is also provided", "fire alarms are checked", "detectors located", "fire alarms and transmit", "monitored by automatic smoke detection systems", "activation of the fire alarm",
    "sprinklers are installed", "there are automatic", "sprinklers installed", "system is installed", "sprinkler system installed",
    "fire water distribution & hydrants average","coverage completely ", "areas are protected with automatic sprinkler",
    "protected with sprinklers","protected by","protection is provided", "have sprinkler protection", "protection is installed",
    "sprinkler systems have been designed", "system is located throughout", "adequate sprinkler", "active protection",
    "sprinkler throughout", "system is through", "sprinkler protection is installed", "throughout","sprinkler system reported",
    "been provided", "provided with automatic", "automatic Fire detection systems", "existing fire detection", "detection provided", 
    "installed throughout", "located throughout", "fire detection throughout", "reported in all", "detectors are present", "detectors are installed", 
    "detection installed", "provided with smoke detectors", "system installed", "systems installed", "systems are fully functional", 
    "detection is installed", "following areas have", "transmit via", "protected by", "fire alarms are generally installed", 
    "fire alarm panel", "approved alarm control panel", "protection provided", "detection is provided", "smoke detection is provided", 
    "smoke detection common", "smoke detectors located", "scattered fire alarms", "detection for this site consists", "combined sprinkler and hydrant system installed"]
    detection_sprinkler_not_installed_word = ["not installed", "not reported", "sprinklers none", "no sprinkler system provided", 
    "no sprinkler","no sprinklers","no sprinkler protection installed",
    "no comprehensive sprinkler","no automatic","not sprinkler protected","protection initially installed",
    "no automatic fire suppression", "sprinklers throughout","sprinkler schematics confirm","none", "fire detection none", "detection none",
    "no detection system provided","no automatic","no automatic fire detection","not provided"]

    install_flag = False
    not_install_flag = False


    install_check = " "
    result_installation = ""
    detection_sprinkler_result_installation_sentence = ""
    
    input = input_data.lower()
    sentences = sent_tokenize(input)
    # print(sentences)

    for sentence in sentences:
        # print("sentence : ", sentence)
        for detection_sprinkler in detection_sprinkler_not_installed_word:
            if (detection_sprinkler in sentence):
                # print("Match : ", detection_sprinkler)
                result_installation = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
                not_install_flag = True
            # return detection_sprinkler
            if (detection_sprinkler in sentence) and (not_install_flag == False):
                # print("Match : ", detection_sprinkler)
                result_installation = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
                not_install_flag = True
            # return detection_sprinkler
        for detection_sprinkler in detection_sprinkler_installed_word:
            if (detection_sprinkler in sentence) and ("sprinkler protection" in sentence) and (not_install_flag == False):
                # print("Match : ", detection_sprinkler)
                result_installation = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
                install_flag = True
            # return detection_sprinkler
            if (detection_sprinkler in sentence) and (install_flag == False) and (not_install_flag == False):
                # print("Match : ", detection_sprinkler)
                result_installation = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_installation_sentence = detection_sprinkler_result_installation_sentence + sentence
                install_flag = True
            # return detection_sprinkler
        # print("install_flag : ", install_flag)

    if install_flag == True:  # ---------------- See this logic in actual implemented code
        install_check = "Yes"  # ---------------- See this logic in actual implemented code
    elif not_install_flag == True:  # ---------------- See this logic in actual implemented code
        install_check = "No"  # ---------------- See this logic in actual implemented code
    else:  # ---------------- See this logic in actual implemented code
        print("Can't determine installed or not")  # ---------------- See this logic in actual implemented code   

    return result_installation, detection_sprinkler_result_installation_sentence, install_check


In [0]:
# ----------------------------------- Below is Version 1 code -----------------------------------------------------------------

# def detection_sprinkler_adequate_processing(input_data):  # ---------------- Add this function

#     detection_sprinkler_adequate_word = ['adequate','adequacy coverage']
#     detection_sprinkler_not_adequate_word = ['not adequate','non adequate','below average','out of standard']

#     adequate_flag = False
#     not_adequate_flag = False
#     adequate_check = ""

#     result_adequate = ""
#     detection_sprinkler_result_adequate_sentence = ""

#     input = input_data.lower()
#     sentences = sent_tokenize(input)
#     # print(sentences)

#     for sentence in sentences:
#         # print("sentence : ", sentence)
#         for detection_sprinkler in detection_sprinkler_not_adequate_word:
#             if (detection_sprinkler in sentence):
#                 # print("Match : ", detection_sprinkler)
#                 result_adequate = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
#                 not_adequate_flag = True
#             # return detection_sprinkler
#             if (detection_sprinkler in sentence) and (not_adequate_flag == False):
#                 # print("Match : ", detection_sprinkler)
#                 result_adequate = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
#                 not_adequate_flag = True
#             # return detection_sprinkler
#         for detection_sprinkler in detection_sprinkler_adequate_word:
#             if (detection_sprinkler in sentence) and (not_adequate_flag == False):
#                 # print("Match : ", detection_sprinkler)
#                 result_adequate = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
#                 adequate_flag = True
#             # return detection_sprinkler
#             if (detection_sprinkler in sentence) and (adequate_flag == False) and (not_adequate_flag == False):
#                 # print("Match : ", detection_sprinkler)
#                 result_adequate = detection_sprinkler
#                 # print("Match sentence : ", sentence)
#                 detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
#                 adequate_flag = True
#             # return detection_sprinkler
#         # print("install_flag : ", install_flag)

#     if adequate_flag == True:
#         adequate_check = "Adequate"
#     elif not_adequate_flag == True:
#         adequate_check = "Not adequate"
#     else:
#         print("Can't determine installed or not")

#     return result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check

# ----------------------------------- Below is Version 2 code -----------------------------------------------------------------

def detection_sprinkler_adequate_processing(input_data):  # ---------------- Add this function

    detection_sprinkler_adequate_word = ['adequate', 'adequacy coverage', 'as adequate', 'satisfactory']
    detection_sprinkler_not_adequate_word = ['not adequate','non adequate','below average','out of standard', 'recommendation to expand','inadequate','in-adequate']

    adequate_flag = False
    not_adequate_flag = False
    adequate_check = ""

    result_adequate = ""
    detection_sprinkler_result_adequate_sentence = ""

    input = input_data.lower()
    sentences = sent_tokenize(input)
    # print(sentences)

    for sentence in sentences:
        # print("sentence : ", sentence)
        for detection_sprinkler in detection_sprinkler_not_adequate_word:
            if (detection_sprinkler in sentence):
                # print("Match : ", detection_sprinkler)
                result_adequate = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
                not_adequate_flag = True
            # return detection_sprinkler
            if (detection_sprinkler in sentence) and (not_adequate_flag == False):
                # print("Match : ", detection_sprinkler)
                result_adequate = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
                not_adequate_flag = True
            # return detection_sprinkler
        for detection_sprinkler in detection_sprinkler_adequate_word:
            if (detection_sprinkler in sentence) and (not_adequate_flag == False):
                # print("Match : ", detection_sprinkler)
                result_adequate = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
                adequate_flag = True
            # return detection_sprinkler
            if (detection_sprinkler in sentence) and (adequate_flag == False) and (not_adequate_flag == False):
                # print("Match : ", detection_sprinkler)
                result_adequate = detection_sprinkler
                # print("Match sentence : ", sentence)
                detection_sprinkler_result_adequate_sentence = detection_sprinkler_result_adequate_sentence + sentence
                adequate_flag = True
            # return detection_sprinkler
        # print("install_flag : ", install_flag)

    if adequate_flag == True:
        adequate_check = "Adequate"
    elif not_adequate_flag == True:
        adequate_check = "Not Adequate"
    else:
        print("Can't determine adequate or not")

    return result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check

In [0]:
# grc_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
# grc_sprinkler_result = {"Sprinklers":'', 'Sprinklers Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
# grc_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Detection Rating":''}
# grc_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''}

# chubb_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
# chubb_sprinkler_result = {"Sprinklers":'', 'Sprinklers Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
# chubb_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Detection Rating":''}
# chubb_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''}


In [0]:
#  ------------------------------------------------------------  Version Code 1 Start -----------------------------------------------------------------------------------------------

# def final_UI(discription_fire_bridage,discription_water_supply,discription_fire_sprinkler,discription_fire_detection,color_fire_result,color_water_result,color_sprinkler_result,color_detection_result):
#     try:
#         if discription_fire_bridage['org'] != "" or discription_fire_bridage['responce'] != "" or \
#                 discription_fire_bridage['distance'] != "":
#             pass
#     except:
#         discription_fire_bridage['org'] = ""
#         discription_fire_bridage['responce'] = ""
#         discription_fire_bridage['distance'] = ""

#     try:
#         if discription_water_supply['install'] != "" or discription_water_supply['flow'] != "":
#             pass
#     except:
#         discription_water_supply['install'] = ""
#         discription_water_supply['flow'] = ""

#     try:
#         if discription_fire_sprinkler['install'] != "":
#             pass
#     except:
#         discription_fire_sprinkler['install'] = ""

#     try:
#         if discription_fire_detection['install'] != "":
#             pass
#     except:
#         discription_fire_detection['install'] = ""
#     print("DISCRIPTION FIRE SPRINKLER",discription_fire_sprinkler)
#     print("DISCRIPTION FIRE DETECTION",discription_fire_detection)
    
#     if color_fire_result['fire_chubb_score'] == ""  or color_fire_result['fire_chubb_score'] == "SD":
#         color_fire_result['fire_chubb_score'] = "SD -- Flagged for manual review"
#     if color_sprinkler_result['Sprinkler Rating'] == "" or color_sprinkler_result['Sprinkler Rating'] == "SD":
#         color_sprinkler_result['Sprinkler Rating'] = "SD -- Flagged for manual review"
#     if color_detection_result['Detection Rating'] == "" or color_detection_result['Detection Rating'] == "SD":
#         color_detection_result['Detection Rating'] = "SD -- Flagged for manual review"
    
#     data = {'protection':
#                 [{'Component_Name': 'Protection', 'Components':
#                     [{'name': 'Fire Brigade',
#                       'Output': [color_fire_result['organization_type'], color_fire_result['responce'],
#                                  color_fire_result['distance'], color_fire_result['Public Fire Department']],
#                       'Chubb_Rating': [color_fire_result['fire_chubb_score'], "", "", ""],
#                       'Description': [discription_fire_bridage['org'], discription_fire_bridage['responce'],
#                                       discription_fire_bridage['distance'], ""]},
#                      {'name': 'Water Supply',
#                       'Output': [color_water_result['Class for NAICS'], color_water_result['Water Supplied'],
#                                  color_water_result['Water Supply Flow'], color_water_result['Water Supply']],
#                       'Chubb_Rating': [color_water_result['Water Supply Rating'], "", "", ""],
#                       'Description': ["", discription_water_supply['install'], discription_water_supply['flow'], ""]},

#                      {'name': 'Sprinklers', 'Output': [color_sprinkler_result['Sprinklers Installed'],
#                                                        color_sprinkler_result['Coverage Provided'],
#                                                        color_sprinkler_result['Extra coverage required'],
#                                                        color_sprinkler_result['Coverage is adequate'],
#                                                        color_sprinkler_result['Sprinklers']],
#                       'Chubb_Rating': [color_sprinkler_result['Sprinkler Rating'], "", "", "", ""],
#                       'Description': [discription_fire_sprinkler['install'], "", "", discription_fire_sprinkler['adequate'], ""]},

#                      {'name': 'Detection', 'Output': [color_detection_result['Detection Installed'],
#                                                       color_detection_result['Coverage Provided'],
#                                                       color_detection_result['Extra coverage required'],
#                                                       color_detection_result['Coverage is adequate'],
#                                                       color_detection_result['Detection']],
#                       'Chubb_Rating': [color_detection_result['Detection Rating'], "", "", "", ""],
#                       'Description': [discription_fire_detection['install'], "", "", discription_fire_detection['adequate'], ""]}]}]}

#     return data
  
#  ------------------------------------------------------------  Version Code 1 End -----------------------------------------------------------------------------------------------

#  ------------------------------------------------------------  Version Code 2 Start ---------------------------------------------------------------------------------------------

def final_UI(discription_fire_bridage,discription_water_supply,discription_fire_sprinkler,discription_fire_detection,color_fire_result,color_water_result,color_sprinkler_result,color_detection_result):
    try:
        if discription_fire_bridage['org'] != "" or discription_fire_bridage['responce'] != "" or \
                discription_fire_bridage['distance'] != "":
            pass
    except:
        discription_fire_bridage['org'] = ""
        discription_fire_bridage['responce'] = ""
        discription_fire_bridage['distance'] = ""

    try:
        if discription_water_supply['install'] != "" or discription_water_supply['flow'] != "":
            pass
    except:
        discription_water_supply['install'] = ""
        discription_water_supply['flow'] = ""

    try:
        if discription_fire_sprinkler['install'] != "":
            pass
    except:
        discription_fire_sprinkler['install'] = ""

    try:
        if discription_fire_detection['install'] != "":
            pass
    except:
        discription_fire_detection['install'] = ""
    print("DISCRIPTION FIRE SPRINKLER",discription_fire_sprinkler)
    print("DISCRIPTION FIRE DETECTION",discription_fire_detection)
    
    if color_fire_result['fire_chubb_score'] == "" or color_fire_result['fire_chubb_score'] == "SD": # Changes
        color_fire_result['fire_chubb_score'] = "SD -- Flagged for manual review"
    if color_sprinkler_result['Sprinkler Rating'] == "" or color_sprinkler_result['Sprinkler Rating'] == "SD": # Changes
        color_sprinkler_result['Sprinkler Rating'] = "SD -- Flagged for manual review"
    if color_detection_result['Detection Rating'] == "" or color_detection_result['Detection Rating'] == "SD": # Changes
        color_detection_result['Detection Rating'] = "SD -- Flagged for manual review"
    
    data = {'protection':
                [{'Component_Name': 'Protection', 'Components':
                    [{'name': 'Fire Brigade',
                      'Output': [color_fire_result['organization_type'], color_fire_result['responce'],
                                 color_fire_result['distance'], color_fire_result['Public Fire Department']],
                      'Chubb_Rating': [color_fire_result['fire_chubb_score'], "", "", ""],
                      'Description': [discription_fire_bridage['org'], discription_fire_bridage['responce'],
                                      discription_fire_bridage['distance'], ""]},
                     {'name': 'Water Supply',
                      'Output': [color_water_result['Class for NAICS'], color_water_result['Water Supplied'],
                                 color_water_result['Water Supply Flow'], color_water_result['Water Supply']],
                      'Chubb_Rating': [color_water_result['Water Supply Rating'], "", "", ""],
                      'Description': ["", discription_water_supply['install'], discription_water_supply['flow'], ""]},

                     {'name': 'Sprinklers', 'Output': [color_sprinkler_result['Sprinklers Installed'],
                                                       color_sprinkler_result['Coverage Provided'],
                                                       color_sprinkler_result['Extra coverage required'],
                                                       color_sprinkler_result['Coverage is adequate'],
                                                       color_sprinkler_result['Sprinklers']],
                      'Chubb_Rating': [color_sprinkler_result['Sprinkler Rating'], "", "", "", ""],
                      'Description': [discription_fire_sprinkler['install'], discription_fire_sprinkler['provided'], discription_fire_sprinkler['required'], discription_fire_sprinkler['adequate'], ""]}, # Changes

                     {'name': 'Detection', 'Output': [color_detection_result['Detection Installed'],
                                                      color_detection_result['Coverage Provided'],
                                                      color_detection_result['Extra coverage required'],
                                                      color_detection_result['Coverage is adequate'],
                                                      color_detection_result['Detection']],
                      'Chubb_Rating': [color_detection_result['Detection Rating'], "", "", "", ""],
                  'Description': [discription_fire_detection['install'], discription_fire_detection['provided'], discription_fire_detection['required'], discription_fire_detection['adequate'], ""]}]}]} # Changes

    return data

#  ------------------------------------------------------------  Version Code 2 End -----------------------------------------------------------------------------------------------


In [0]:
# grc_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
# grc_sprinkler_result = {"Sprinklers":'', 'Sprinklers Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
# grc_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Detection Rating":''}
# grc_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''}

# chubb_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
# chubb_sprinkler_result = {"Sprinklers":'', 'Sprinklers Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
# chubb_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Detection Rating":''}
# chubb_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''}

df_fire_brigade_grc = pd.DataFrame()
df_water_supply_grc = pd.DataFrame()
df_fire_brigade_chubb = pd.DataFrame()
df_water_supply_chubb = pd.DataFrame()
df_fire_sprinkler_chubb = pd.DataFrame()
df_fire_detection_chubb = pd.DataFrame()

discription_fire_bridage = {'distance':'', 'responce':'', 'org':''}
discription_water_supply = {'flow':'', 'install':''}
#discription_fire_detection = {'install':'', 'adequate':''}
#discription_fire_sprinkler = {'install':'', 'adequate': ''}
discription_fire_detection = {'install':'', 'adequate':'', 'provided': '', 'required': ''} # Changes
discription_fire_sprinkler = {'install':'', 'adequate':'', 'provided': '', 'required': ''}


if carrier_name == 'CHUBB':
    print("Inside Chubb Block")
    chubb_fire_result = {'Public Fire Department': '', 'fire_chubb_score': '', 'distance': '', 'responce': '',
                         'organization_type': ''}
    chubb_sprinkler_result = {"Sprinklers": '', 'Sprinklers Installed': "", 'Coverage Provided': "",
                              'Extra coverage required': "", "Coverage is adequate": "", "Sprinkler Rating": ''}
    chubb_detection_result = {"Detection": '', 'Detection Installed': "", 'Coverage Provided': "",
                              'Extra coverage required': "", "Coverage is adequate": "", "Detection Rating": ''}
    chubb_water_result = {"Water Supply": '', "Class for NAICS": '', "Water Supplied": '', "Water Supply Flow": '',
                          "Water Supply Rating": ''}
    for file in Images_list:
        # data_extraction(file)
        #data = text_identification(file)
        try:
            data = text_identification(file)  # ------ Http Change
        except:
            dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file again !!!!"
     }))
        result_water_supply_chubb, result_list_water_chubb = text_water_chubb(data['line'])
        result_sprinkler_chubb, result_list_sprinkler_chubb = text_sprinkler_chubb(data['line'])
        result_fire_chubb, result_list_fire_chubb = text_fire_chubb(data['line'])
        result_detection_chubb, result_list_detection_chubb = text_detection_chubb(data['line'])
        #print(11, result_water_supply_chubb)
        #print(22, result_sprinkler_chubb)
        #print(33, result_fire_chubb)
        #print(44, result_detection_chubb)
        if len(result_water_supply_chubb) > 10:
            # print("\n", "result water : ",result, "\n")
            # print("result_list water : ",result_list, "\n")
            chubb_direct_rating = rating_finder_chubb(result_water_supply_chubb)
            # print(chubb_direct_rating)
            result_flow_chubb = water_supply_gpm_calculation(result_water_supply_chubb)
            # print("result gpm : ", result_flow_chubb)
            chubb_water_result['Water Supply'] = chubb_direct_rating
            chubb_water_result['Water Supply Rating'] = chubb_direct_rating
            chubb_water_result['Class for NAICS'] = ''
            if result_flow_chubb > 0:
                chubb_water_result['Water Supplied'] = 'Yes'
            chubb_water_result['Water Supply Flow'] = str(result_flow_chubb) + ' lpm'
            discription_water_supply = discription_water_supply_processing(result_water_supply_chubb)
            print("discription_water_supply : ", discription_water_supply)

        if len(result_fire_chubb) > 10:
            
            # print("\n", "result brigade : ",result_fire, "\n")
            # print("result_list brigade : ",result_list_fire, "\n")
            chubb_direct_rating = rating_finder_chubb(result_fire_chubb)
            # print(chubb_direct_rating)
            responce, distance = score_calculation_protection_fire(result_fire_chubb)
            # print("responce : ", responce)
            # print("distance : ", distance)
            result_dics, result_org = organization_finder(result_fire_chubb, public_keyword, private_keyword,volunteer_keyword, paid_keyword, full_keyword)
            # print("Result : ", result_dics)
            # print()
            print("Result Final111: ", result_org)
            # print()
            chubb_fire_result['Public Fire Department'] = chubb_direct_rating
            chubb_fire_result['fire_chubb_score'] = chubb_direct_rating
            chubb_fire_result['distance'] = distance
            chubb_fire_result['responce'] = responce
            chubb_fire_result['organization_type'] = result_org[0]
            # print("chubb_fire_result : ", chubb_fire_result)
            discription_fire_bridage = discription_fire_bridage_processing(result_fire_chubb)
            print("discription_fire_bridage : ", discription_fire_bridage)

            # rating_fire_GRC = final_rating_calculation(chubb_fire_result)

        if len(result_sprinkler_chubb) > 10:
            # print("\n","result sprinkler : ",result_sprinkler, "\n")
            # print("result_list sprinkler : ",result_list_sprinkler, "\n")
            chubb_direct_rating = rating_finder_chubb(result_sprinkler_chubb)
            # print(chubb_direct_rating)
            result_coverage, detection_sprinkler_result_coverage_sentence = detection_sprinkler_coverage_processing(
                result_sprinkler_chubb)
            # print("result_sprinkler : ", resultd)
            result_installation, detection_sprinkler_result_installation_sentence, install_check = detection_sprinkler_installation_processing(
                result_sprinkler_chubb)
            chubb_sprinkler_result['Sprinklers'] = chubb_direct_rating
            chubb_sprinkler_result['Sprinkler Rating'] = chubb_direct_rating
            if result_coverage != "":
                chubb_sprinkler_result['Coverage Provided'] = result_coverage
            if result_installation != "":
                chubb_sprinkler_result['Sprinklers Installed'] = install_check
            chubb_sprinkler_result['Extra coverage required'] = ""
            result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check = detection_sprinkler_adequate_processing(
                result_sprinkler_chubb)
            chubb_sprinkler_result['Coverage is adequate'] = adequate_check
            discription_fire_sprinkler = discription_fire_sprinkler_processing(result_sprinkler_chubb)
            discription_fire_sprinkler['coverage'] = detection_sprinkler_result_coverage_sentence
            print("discription_fire_sprinkler : ", discription_fire_sprinkler)

        if len(result_detection_chubb) > 10:
            # print("result detection : ",result_detection, "\n")
            # print("result_list detection : ",result_list_detection, "\n")
            chubb_direct_rating = rating_finder_chubb(result_detection_chubb)
            # print(chubb_direct_rating)
            result_coverage, detection_sprinkler_result_coverage_sentence = detection_sprinkler_coverage_processing(
                result_detection_chubb)
            # print("result_detection : ", resultd)
            result_installation, detection_sprinkler_result_installation_sentence, install_check = detection_sprinkler_installation_processing(
                result_detection_chubb)
            chubb_detection_result['Detection'] = chubb_direct_rating
            chubb_detection_result['Detection Rating'] = chubb_direct_rating
            if result_installation != "":
                chubb_detection_result['Detection Installed'] = install_check
            if result_coverage != "":
                chubb_detection_result['Coverage Provided'] = result_coverage
            chubb_detection_result['Extra coverage required'] = ""
            result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check = detection_sprinkler_adequate_processing(
                result_detection_chubb)
            chubb_detection_result['Coverage is adequate'] = adequate_check
            discription_fire_detection = discription_fire_detection_processing(result_detection_chubb)
            discription_fire_detection['coverage'] = detection_sprinkler_result_coverage_sentence
            print("discription_fire_detection : ", discription_fire_detection)

        output_full_path = "/dbfs/mnt/aria/Batch_Output/Batch_Chubb/"

        # print("chubb_fire_result : ", chubb_fire_result)
        df_fire_brigade_chubb = intermidiet_table_fire_brigade(file, chubb_fire_result, "output",
                                                               discription_fire_bridage)
        # print("df_fire_brigade_chubb : ", df_fire_brigade_chubb)
        final_file_name_fire_brigade = fileName.split(".")[0] + "fire_brigade" + ".csv"
        final_output_path = output_full_path + final_file_name_fire_brigade
        #df_fire_brigade_chubb.to_csv(final_output_path, index=False)

        # print("chubb_water_result : ", chubb_water_result)
        df_water_supply_chubb = intermidiet_table_water_supply(file, chubb_water_result, "output",
                                                               discription_water_supply)

        final_file_name_water_supply = fileName.split(".")[0] + "water_supply" + ".csv"
        final_output_path = output_full_path + final_file_name_water_supply
        #df_water_supply_chubb.to_csv(final_output_path, index=False)

        # print("df_water_supply_chubb : ", df_water_supply_chubb)
        # final_ui_water = df_to_json_Conversion_water_supply(df_water_supply_chubb)
        # print("final_ui_water : ", final_ui_water)
        print()
        # print("chubb_sprinkler_result : ", chubb_sprinkler_result)
        df_fire_sprinkler_chubb = intermidiet_table_sprinkler(file, chubb_sprinkler_result, "output",
                                                              discription_fire_sprinkler)
        # print("df_fire_sprinkler_chubb : ", df_fire_sprinkler_chubb)
        # final_ui_sprinkler = df_to_json_Conversion_sprinkler(df_fire_sprinkler_chubb)
        # print("final_ui_sprinkler : ", final_ui_sprinkler)
        final_file_name_sprinkler = fileName.split(".")[0] + "sprinklers" + ".csv"
        final_output_path = output_full_path + final_file_name_sprinkler
        #df_fire_sprinkler_chubb.to_csv(final_output_path, index=False)

        # print("chubb_detection_result : ", chubb_detection_result)
        try:
            df_fire_detection_chubb = intermidiet_table_detection(file, chubb_detection_result, "output",
                                                                  discription_fire_detection)

            final_file_name_fire_hydrant = fileName.split(".")[0] + "fire_hydrant" + ".csv"
            final_output_path = output_full_path + final_file_name_fire_hydrant
            #df_fire_detection_chubb.to_csv(final_output_path, index=False)
        except:
            print("Detection not on this page")
            pass

        # print("df_fire_detection_chubb : ", df_fire_detection_chubb)
        # final_ui_detection = df_to_json_Conversion_detection(df_fire_detection_chubb)
        # print("final_ui_detection : ", final_ui_detection)
        data = final_UI(discription_fire_bridage, discription_water_supply, discription_fire_sprinkler, discription_fire_detection,
                         chubb_fire_result, chubb_water_result, chubb_sprinkler_result,
                        chubb_detection_result)
  
      
elif carrier_name == 'GRC':
    print("Inside GRC Block")
    #print("Images_list : ", Images_list)
    grc_fire_result = {'Public Fire Department': '', 'fire_chubb_score': '', 'distance': '', 'responce': '',
                       'organization_type': ''}
    grc_sprinkler_result = {"Sprinklers": '', 'Sprinklers Installed': "", 'Coverage Provided': "",
                            'Extra coverage required': "", "Coverage is adequate": "", "Sprinkler Rating": ''}
    grc_detection_result = {"Detection": '', 'Detection Installed': "", 'Coverage Provided': "",
                            'Extra coverage required': "", "Coverage is adequate": "", "Detection Rating": ''}
    grc_water_result = {"Water Supply": '', "Class for NAICS": '', "Water Supplied": '', "Water Supply Flow": '',
                        "Water Supply Rating": ''}
    for file in Images_list:
        # flag=None
        # data_extraction(file)
        #data = text_identification(file)
        try:
            data = text_identification(file)  # ------ Http Change
        except:
            dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file again !!!!"
     }))
        result_water_grc, result_list_grc = text_water_grc(data['line'])
        result_sprinkler_grc, result_list_sprinkler_grc = text_sprinkler_grc(data['line'])
        result_fire_grc, result_list_fire_grc = text_fire_grc(data['line'])
        result_detection_grc, result_list_detection_grc = text_detection_grc(data['line'])
        print("result_fire_grc : ", result_fire_grc)
        print("COMING BEFORE FIRST IF CONDITION")
        print("result_water_grc : ", result_water_grc)
        if len(result_water_grc) >= 2:
            print("COMING INSIDE THE FIRST IF CONDITION")
            # flag="Water Supply"
            result_flow_grc = water_supply_gpm_calculation(result_water_grc)
            grc_water_result['Water Supply'] = ''
            grc_water_result['Water Supply Rating'] = ''
            grc_water_result['Class for NAICS'] = ''
            if result_flow_grc > 0:
                grc_water_result['Water Supplied'] = 'Yes'
            grc_water_result['Water Supply Flow'] = str(result_flow_grc) + ' lpm'
            discription_water_supply = discription_water_supply_processing(result_water_grc)
            print("discription_water_supply : ", discription_water_supply)
            df_water_supply_grc = intermidiet_table_water_supply(file, grc_water_result, "output", discription_water_supply)
            print("df_water_supply_grc : ", df_water_supply_grc)
    
            output_full_path = "/dbfs/mnt/aria/Batch_Output/Batch_GRC/"
    
            final_file_name_water_supply = fileName.split(".")[0] + "water_supply" + ".csv"
            final_output_path = output_full_path + final_file_name_water_supply
            print('final_output_path : ', final_output_path)
            #df_water_supply_grc.to_csv(final_output_path, index=False)
    
            # final_ui_water_grc = df_to_json_Conversion_water_supply(df_water_supply_grc)
        if len(result_fire_grc) >= 2:
            print("COMING INSIDE SECOND IF CONDITION")
            # flag="Fire Brigade"
            # print("\n", "result brigade : ",result_fire, "\n")
            # print("result_list brigade : ",result_list_fire, "\n")
            # grc_direct_rating = rating_finder_chubb(result_fire_chubb)
            # print(chubb_direct_rating)
            responce, distance = score_calculation_protection_fire(result_fire_grc)
            # print("responce : ", responce)
            # print("distance : ", distance)
            result_dics, result_org = organization_finder(result_fire_grc, public_keyword, private_keyword,
                                                          volunteer_keyword, paid_keyword, full_keyword)
            # print("Result : ", result_dics)
            # print("Result Final: ", result_org)
            grc_fire_result['Public Fire Department'] = ''
            grc_fire_result['fire_chubb_score'] = ''
            grc_fire_result['distance'] = distance
            grc_fire_result['responce'] = responce
            grc_fire_result['organization_type'] = result_org[0]
            # print("chubb_fire_result : ", chubb_fire_result)
            grc_fire_result = final_rating_calculation(grc_fire_result)
            # print("grc_fire_result : ", grc_fire_result)
            discription_fire_bridage = discription_fire_bridage_processing(result_fire_grc)
            print("discription_fire_bridage : ", discription_fire_bridage)
            df_fire_brigade_grc = intermidiet_table_fire_brigade(file, grc_fire_result, "output", discription_fire_bridage)
            print("df_fire_brigade_grc : ", df_fire_brigade_grc)
            # rating_fire_GRC = final_rating_calculation(chubb_fire_result)
    
            output_full_path = "/dbfs/mnt/aria/Batch_Output/Batch_GRC/"
    
            final_file_name_fire_brigade = fileName.split(".")[0] + "fire_brigade" + ".csv"
    
            final_output_path = output_full_path + final_file_name_fire_brigade
            print("final_output_path : ", final_output_path)
            #df_fire_brigade_grc.to_csv(final_output_path, index=False)
    
        if len(result_sprinkler_grc) >= 2:
            # print("\n","result sprinkler : ",result_sprinkler, "\n")
            # print("result_list sprinkler : ",result_list_sprinkler, "\n")
            result_coverage, detection_sprinkler_result_coverage_sentence = detection_sprinkler_coverage_processing(result_sprinkler_grc)
            # print("result_sprinkler : ", resultd)
            print("result_coverage, detection_sprinkler_result_coverage_sentence : ", result_coverage, detection_sprinkler_result_coverage_sentence)
            result_installation, detection_sprinkler_result_installation_sentence, install_check = detection_sprinkler_installation_processing(
                result_sprinkler_grc)
            print("result_installation, detection_sprinkler_result_installation_sentence, install_check : ", result_installation, detection_sprinkler_result_installation_sentence, install_check)
            grc_sprinkler_result['Sprinklers'] = ""
            grc_sprinkler_result['Sprinkler Rating'] = ""
            if result_coverage != "":
                grc_sprinkler_result['Coverage Provided'] = result_coverage
            if install_check != "":
                grc_sprinkler_result['Sprinklers Installed'] = install_check
            grc_sprinkler_result['Extra coverage required'] = ""
            result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check = detection_sprinkler_adequate_processing(result_sprinkler_grc)
            grc_sprinkler_result['Coverage is adequate'] = adequate_check
            discription_fire_sprinkler = discription_fire_sprinkler_processing(result_sprinkler_grc)
            #discription_fire_sprinkler['coverage'] = detection_sprinkler_result_coverage_sentence    # Remove comment if this if backfire
            print("discription_fire_sprinkler 1 : ", discription_fire_sprinkler)
            print("grc_sprinkler_result : ", grc_sprinkler_result)
            grc_sprinkler_result = final_rating_calculation_fire_sprinkler(grc_sprinkler_result, carrier_name)    # Remove this if backfire    
            df_fire_sprinkler_grc = intermidiet_table_sprinkler(file, grc_sprinkler_result, "output",discription_fire_sprinkler)
            print("grc_sprinkler_result 2 : ", grc_sprinkler_result)
    
            output_full_path = "/dbfs/mnt/aria/Batch_Output/Batch_GRC/"
    
            final_file_name_fire_sprinkler = fileName.split(".")[0] + "fire_sprinkler" + ".csv"
    
            final_output_path = output_full_path + final_file_name_fire_sprinkler
            print("final_output_path : ", final_output_path)
            #df_fire_sprinkler_grc.to_csv(final_output_path, index=False)
        print("RESULT DETECITON GRC:",result_detection_grc) 
        if len(result_detection_grc) >= 2:
            # print("result detection : ",result_detection, "\n")
            # print("result_list detection : ",result_list_detection, "\n")
            result_coverage, detection_sprinkler_result_coverage_sentence = detection_sprinkler_coverage_processing(
                result_detection_grc)
            # print("result_detection : ", resultd)
            result_installation, detection_sprinkler_result_installation_sentence, install_check = detection_sprinkler_installation_processing(
                result_detection_grc)
            grc_detection_result['Detection'] = ""
            grc_detection_result['Detection Rating'] = ""
            if install_check != "":
                grc_detection_result['Detection Installed'] = install_check
            if result_coverage != "":
                grc_detection_result['Coverage Provided'] = result_coverage
            grc_detection_result['Extra coverage required'] = ""
            result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check = detection_sprinkler_adequate_processing(result_detection_grc)
            grc_detection_result['Coverage is adequate'] = adequate_check
                  
            discription_fire_detection = discription_fire_detection_processing(result_detection_grc)
            #discription_fire_detection['coverage'] = detection_sprinkler_result_coverage_sentence    # Remove comment if this if backfire
            print("discription_fire_detection : ", discription_fire_detection)
    
            output_full_path = "/dbfs/mnt/aria/Batch_Output/Batch_Chubb/"
            grc_detection_result = final_rating_calculation_fire_detection(grc_detection_result, carrier_name)    # Remove this if backfire
            print("grc_detection_result : ", grc_detection_result)
            df_fire_detection_grc = intermidiet_table_detection(file, grc_detection_result, "output", discription_fire_detection)
            # print("df_fire_brigade_chubb : ", df_fire_brigade_chubb)
            final_file_name_fire_detection = fileName.split(".")[0] + "fire_detection" + ".csv"
            final_output_path = output_full_path + final_file_name_fire_detection
            #df_fire_detection_grc.to_csv(final_output_path, index=False)
    
        data = final_UI(discription_fire_bridage, discription_water_supply, discription_fire_sprinkler, discription_fire_detection, grc_fire_result, grc_water_result, grc_sprinkler_result, grc_detection_result)
    
        # output_full_path = "/dbfs/mnt/aria/Batch_Output/Batch_Protection/TodaysOutPut2/"
    
        # final_file_name_fire_brigade = fileName.split(".")[0] + "fire_brigade" + ".csv"
        # final_output_path = output_full_path + final_file_name_fire_brigade
        # df_protection_fire_brigade.to_csv(final_output_path, index=False)
    
        # final_file_name_fire_hydrant = fileName.split(".")[0] + "fire_hydrant" + ".csv"
        # final_output_path = output_full_path + final_file_name_fire_hydrant
        # df_protection_detection.to_csv(final_output_path, index=False)
    
        # final_file_name_sprinkler = fileName.split(".")[0] + "sprinklers" + ".csv"
        # final_output_path = output_full_path + final_file_name_sprinkler
        # df_protection_sprinkler.to_csv(final_output_path, index=False)
    
        # final_file_name_water_supply = fileName.split(".")[0] + "water_supply" + ".csv"
        # final_output_path = output_full_path + final_file_name_water_supply
        # df_protection_water_supply.to_csv(final_output_path, index=False)
if carrier_name == 'AIG':
    for file in Images_list:
        patch_work(file)
        #data_extraction(file)
        try:
            data_extraction(file)  # ------ Http Change
        except:
            dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file again !!!!"
  })) 
        print("FileName : ", file)
        try:
            results_list = frame_creation(file)
            # print("results_list : ",results_list)
            for result in results_list:
                total_gpm, unit = find_relevent_table_water(result, match_AIG_Water_Supply[1:], specific_gpm_words)
                total_flow = sum(total_gpm)
                #print("total_flow : ", total_flow)
                if total_flow > 0 and color_water_result['Water Supply Flow'] == "":
                    color_water_result['Water Supplied'] = "Yes"
                    if ("gpm" in unit.lower()):
                        #print("Total Flow : ", total_flow, " gpm")
                        total_flow = total_flow * 3.78
                        #print("Total Flow : ", total_flow, " lpm")
                        color_water_result['Water Supply Flow'] = str(total_flow) + " lpm"
                    elif ("lpm" in unit.lower()) or ("l/min" in unit.lower()):
                        #print("Total Flow : ", total_flow, " lpm")
                        color_water_result['Water Supply Flow'] = str(total_flow) + " lpm"
                    else:
                        color_water_result['Water Supplied'] = "No"
                #print("Unit : ", unit)
                if unit != "":
                    pass #print("Unit : ", unit)

                temp_sprinkler, sprinkler_detection = find_relevent_table_sprinkler(result, match_AIG_Sprinklers[1:],
                                                               specific_Sprinklers_words,
                                                               sprinkler_spread_flag)  # -------------- Add this 
                #print("Result Sprinkler first round 1 : ", temp_sprinkler)
                if temp_sprinkler['% AS'] != "" or temp_sprinkler['% ASN'] != "" or temp_sprinkler['% Detection'] != "":
                    color_sprinkler_result['Coverage Provided'] = temp_sprinkler['% AS']
                    color_sprinkler_result['Extra coverage required'] = temp_sprinkler['% ASN']
                    color_detection_result['Coverage Provided'] = temp_sprinkler['% Detection']

                if temp_sprinkler['% AS'] == 3 or temp_sprinkler['% ASN'] == 5 or temp_sprinkler['% Detection'] == 6:  # -------------- Add this 
                    sprinkler_spread_flag = True  # -------------- Add this 
                try:    
                    if sprinkler_detection: # --------------------- Changes for form recognizer fail
                        perc_sprinkler,perc_sprinkler_required,perc_detection=calculate_total(sprinkler_detection)   # --------------------- Changes for form recognizer fail
                        #print("sprinkler_detection : ", sprinkler_detection)   # --------------------- Changes for form recognizer fail
                        print('CALCULATED__',perc_sprinkler,perc_detection,perc_sprinkler_required)   # --------------------- Changes for form recognizer fail
                        CALCULATED__ = "Got Value"
                except:
                    pass
                score = find_relevent_AIG_FireBrigade(result)
                results = [[i for i in row] for row in result.itertuples()]
                # print("results : ",results)
                # print("len(results) : ", results)
                for spread in results:
                    if (ratio(spread[1], "Water Supply") > 90):
                        spread_table_water.append(spread)
                    if (ratio(spread[1], "Automatic Fire Detection") > 90 or ratio(spread[1], "Security") > 90) and len(
                            results) > 0:
                        spread_table_fire.append(spread)
                    if (ratio(spread[1], "Sprinklers") > 90 or ratio(spread[1],
                                                                     "Special Extinguishing Protection") > 90 or ratio(
                            spread[1], "Fire Extinguishing Appliances/Hosereels") > 90 or ratio(spread[1],
                                                                                                "Public Fire Department") > 90 or ratio(
                            spread[1], "Smoke Extraction") > 90) and len(results) < 5:
                        spread_table_fire.append(spread)
                    if (ratio(spread[1], "Sprinklers") > 90 or ratio(spread[1],
                                                                     "Special Extinguishing Protection") > 90 or ratio(
                            spread[1], "Fire Extinguishing Appliances/Hosereels") > 90 or ratio(spread[1],
                                                                                                "Public Fire Department") > 90 or ratio(
                            spread[1], "Smoke Extraction") > 90) and len(results) == 5 or len(
                            results) == 6 and score == 0:
                        # print("Single page result : ",results)
                        no_spread_table_fire = results
                        break
        except UnboundLocalError as u:
            traceback.print_exc()

        # print("len(spread_table_fire) : ",len(spread_table_fire))

        if (len(spread_table_fire) > 0) and (len(spread_table_fire) < 4):
            top_patch(file)
            #data_extraction(file)
            try:
                data_extraction(file)  # ------ Http Change
            except:
                dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file again !!!!"
  })) 
            print("FileName : ", file)
            try:
                results_list = frame_creation(file)
                for result in results_list:
                    if temp_sprinkler['% AS'] == "" or temp_sprinkler['% ASN'] == "" or temp_sprinkler[
                        '% Detection'] == "":
                        temp_sprinkler, _ = find_relevent_table_sprinkler(result, match_AIG_Sprinklers[1:],
                                                                       specific_Sprinklers_words,
                                                                       sprinkler_spread_flag)  # -------------- Add this 
                        #print("Result Sprinkler first round 2 : ", temp_sprinkler)
                    if temp_sprinkler['% AS'] != "" and temp_sprinkler['% ASN'] != "" and temp_sprinkler[
                        '% Detection'] != "":
                        color_sprinkler_result['Coverage Provided'] = temp_sprinkler['% AS']
                        color_sprinkler_result['Extra coverage required'] = temp_sprinkler['% ASN']
                        color_detection_result['Coverage Provided'] = temp_sprinkler['% Detection']

                    if temp_sprinkler['% AS'] == 3 or temp_sprinkler['% ASN'] == 5 or temp_sprinkler[
                        '% Detection'] == 6:  # -------------- Add this 
                        sprinkler_spread_flag = True  # -------------- Add this 

                    score = find_relevent_AIG_FireBrigade(result)
                    results = [[i for i in row] for row in result.itertuples()]
                    for spread in results:
                        if (ratio(spread[1], "Sprinklers") > 90 or ratio(spread[1],
                                                                         "Special Extinguishing Protection") > 90 or ratio(
                                spread[1], "Fire Extinguishing Appliances/Hosereels") > 90 or ratio(spread[1],
                                                                                                    "Public Fire Department") > 90 or ratio(
                                spread[1], "Smoke Extraction") > 90) and len(results) < 5:
                            if spread not in spread_table_fire:
                                spread_table_fire.append(spread)
            except UnboundLocalError as u:
                traceback.print_exc()

        # print("spread_table_fire : ",spread_table_fire)
        if len(spread_table_fire) <= 4:
            # print("Multi page result : ",spread_table_fire)
            for color in spread_table_fire:
                for color_fire_count, color_fire_data in enumerate(color):
                    if color_fire_data == "Sprinklers" or color_fire_data == "S prinklers":
                        color_result = [x for x in color if x]
                        color_sprinkler_result['Sprinklers'] = "".join(color_result[2:])
                    if color_fire_data == "Public Fire Department":
                        color_result = [x for x in color if x]
                        color_fire_result['Public Fire Department'] = "".join(color_result[2:])

        if len(spread_table_fire) >= 1:
            # print("Multi page result : ",spread_table_fire)
            for color in spread_table_fire:
                for color_fire_count, color_fire_data in enumerate(color):
                    if color_fire_data == "Automatic Fire Detection":
                        color_result = [x for x in color if x]
                        color_detection_result['Detection'] = "".join(color_result[2:])

            print()
            spread_table_fire = []

        if len(spread_table_water) > 0:
            # print("Multi page result : ",spread_table_water)
            for color in spread_table_water:
                for color_water_count, color_water_data in enumerate(color):
                    if color_water_data == "Water Supply":
                        color_result = [x for x in color if x]
                        color_water_result['Water Supply'] = "".join(color_result[2:])
            spread_table_water = []

        # print("no_spread_table_fire : ", no_spread_table_fire)
        for color in no_spread_table_fire:
            for color_fire_count, color_fire_data in enumerate(color):
                if color_fire_data == "Public Fire Department":
                    color_result = [x for x in color if x]
                    # print("color_result : ", color_result)
                    color_fire_report = "".join(color_result[2:])
                    color_fire_result['Public Fire Department'] = color_fire_report
                if color_fire_data == "Sprinklers" or color_fire_data == "S prinklers":
                    color_result = [x for x in color if x]
                    # print("color_result : ", color_result)
                    color_sprinklers_report = "".join(color_result[2:])
                    color_sprinkler_result['Sprinklers'] = color_sprinklers_report

        # Masking process and text extraction

        # frame_creation_mask(file)
        if temp_sprinkler['% AS'] == "" or temp_sprinkler['% ASN'] == "" or temp_sprinkler['% Detection'] == "":
            maskImage(file)
            #data_extraction(file)
            try:
                data_extraction(file)  # ------ Http Change
            except:
                dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file again !!!!"
  })) 
            results_list = frame_creation(file)
            # print("results_list : ", results_list)
            for result in results_list:
                temp_sprinkler, _ = find_relevent_table_sprinkler(result, match_AIG_Sprinklers[1:],
                                                               specific_Sprinklers_words,
                                                               sprinkler_spread_flag)  # -------------- Add this 
                #print("Result Sprinkler second round : ", temp_sprinkler)
            if temp_sprinkler['% AS'] != "" and temp_sprinkler['% ASN'] != "" and temp_sprinkler['% Detection'] != "":
                color_sprinkler_result['Coverage Provided'] = temp_sprinkler['% AS']
                color_sprinkler_result['Extra coverage required'] = temp_sprinkler['% ASN']
                color_detection_result['Coverage Provided'] = temp_sprinkler['% Detection']

            if temp_sprinkler['% AS'] == 3 or temp_sprinkler['% ASN'] == 5 or temp_sprinkler['% Detection'] == 6:  # -------------- Add this 
                sprinkler_spread_flag = True  # -------------- Add this

        print()

        # print("color_detection_result : ", color_detection_result)

        # Detection operation
        # color_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":""}
        if color_detection_result['Coverage Provided'] != '':
            try:
                temp_detection_cover = re.sub("\s*%", '', color_detection_result['Coverage Provided'])
            except:
                color_sprinkler_result['Coverage Provided'] = ""
                color_sprinkler_result['Extra coverage required'] = ""
                color_detection_result['Coverage Provided'] = ""
                pass
            try:
                detection_coverage = float(temp_detection_cover)
                if detection_coverage > 0:
                    color_detection_result['Detection Installed'] = "Yes"
                else:
                    color_detection_result['Detection Installed'] = "No"
            except:
                print("Form recognizer failed on detection table")
                pass
        else:
            print("No data captured for detection")

        # print("color_detection_result : ", color_detection_result)
        # print("color_fire_result : ", color_fire_result)
        # print("color_sprinkler_result : ", color_sprinkler_result)

        # Sprinkler opereation
        # color_sprinkler_result = {"Sprinklers":'', '% Detection':"", 'Sprinklers installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
        if color_sprinkler_result['Coverage Provided'] != '':
            try:
                temp_sprinkler_cover = re.sub("\s*%", '', color_sprinkler_result['Coverage Provided'])
                temp_sprinkler_required_cover = re.sub("\s*%", '', color_sprinkler_result['Extra coverage required'])
            except:
                color_sprinkler_result['Coverage Provided'] = ""
                color_sprinkler_result['Extra coverage required'] = ""
                color_detection_result['Coverage Provided'] = ""
                pass
            try:
                sprinkler_coverage = float(temp_sprinkler_cover)
                sprinkler_required_cover = float(temp_sprinkler_required_cover)
                if sprinkler_coverage > 0:
                    color_sprinkler_result['Sprinklers Installed'] = "Yes"
                else:
                    color_sprinkler_result['Sprinklers Installed'] = "No"
            except:
                print("Form recognizer failed on sprinkler table")
                pass
        else:
            print("No data captured for sprinkler")

        maskImage(file)
        #data = text_identification(file)
        try:
            data = text_identification(file)  # ------ Http Change
        except:
            dbutils.notebook.exit(json.dumps({
    "isToast": "True",
    "fileName":fileName,
    "Error_message": "Protection failed because of Form-Recognizer. Please re-run the file again !!!!"
     }))
        #result, result_list = text_protection(data['line'])
        result, result_list = text_protection(data['line'])    #-----------------------  Change Made
        water_text_data, _ = text_water_AIG(data['line'])          #-----------------------  Change Made
        detection_text_data, _ = text_detection_AIG(data['line'])  #-----------------------  Change Made
        sprinkler_text_data, _ = text_sprinkler_AIG(data['line'])  #-----------------------  Change Made
        result_discription = result
        print("detection_text_data : ", detection_text_data)
        # New code implemented here
        if len(result_list) > 0:
            print("result_discription : ", result_discription)
            result_dics, result_org = organization_finder(result, public_keyword, private_keyword, volunteer_keyword,
                                                          paid_keyword, full_keyword)
            discription_fire_bridage = discription_fire_bridage_processing(result_discription)  # ---- Add this
            result_discription = ""  # ---- Add this
            print("discription_fire_bridage : ", discription_fire_bridage)
            # print()
            # print("Result Final Org Type : ", result_org)
            if color_fire_result['organization_type'] == "" and len(result_org) > 0:
                color_fire_result['organization_type'] = result_org[0]
            else:
                color_fire_result['organization_type'] == ""

            p = score_calculation_protection_fire(result)
            #print("Responce result final : ", p[0])
            #print("Distance result final : ", p[1])

            if color_fire_result['distance'] == "" and p[1] != "":
                if isinstance(p[1], list):
                    distance_ = convert_To_String(p[1])
                    color_fire_result['distance'] = distance_
                else:
                    color_fire_result['distance'] = p[1]
            if color_fire_result['responce'] == "" and p[0] != "":
                if isinstance(p[0], list):
                    responce_ = convert_To_String(p[0])
                    color_fire_result['responce'] = responce_
                else:
                    color_fire_result['responce'] = p[0]

    # print(color_fire_result)
    # print()
    
        if color_water_result['Water Supply Flow'] == "0 lpm" or color_water_result['Water Supply Flow'] == "":   # ------------------------ Change Made
            #print("--------------- water_text_data : ", water_text_data)
            water_flow = water_supply_gpm_calculation(water_text_data)
            #print("water_flow : ", water_flow)
            if water_flow > 0:
                color_water_result['Water Supplied'] = "Yes"
            else:
                color_water_result['Water Supplied'] = "No"
            color_water_result['Water Supply Flow'] = str(water_flow) + " lpm"  # -------------- Add this 

        discription_water_supply = discription_water_supply_processing(water_text_data)  # -------------- Add this
        #print("detection_text_data : ", detection_text_data)
        if len(sprinkler_text_data) > 0:  # ------------------------ Change Made
            result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check = detection_sprinkler_adequate_processing(sprinkler_text_data)   # ------------------------ Change Made
            color_sprinkler_result['Coverage is adequate'] = adequate_check   # ------------------------ Change Made
            discription_fire_sprinkler = discription_fire_sprinkler_processing(sprinkler_text_data)   # ------------------------ Change Made
        if len(detection_text_data) > 0:  # ------------------------ Change Made      
            result_adequate, detection_sprinkler_result_adequate_sentence, adequate_check = detection_sprinkler_adequate_processing(detection_text_data)   # ------------------------ Change Made
            color_detection_result['Coverage is adequate'] = adequate_check   # ------------------------ Change Made
            discription_fire_detection = discription_fire_detection_processing(detection_text_data)   # ------------------------ Change Made

    color_fire_result = final_rating_calculation(color_fire_result)
    
    if discription_fire_detection['install'] == '' and discription_fire_detection['adequate'] == "":
        discription_fire_detection = discription_fire_detection_processing(detection_text_data_backup)
    if discription_fire_sprinkler['install'] == '' and discription_fire_sprinkler['adequate'] == "":
        discription_fire_sprinkler = discription_fire_sprinkler_processing(sprinkler_text_data_backup)
        
    
    # print(color_fire_result)

#     if color_water_result['Water Supply Flow'] == '':  # -------------- Add this
#         water_flow = water_supply_gpm_calculation(water_text_data)

#         if water_flow > 0:
#             color_water_result['Water Supplied'] = "Yes"
#         else:
#             color_water_result['Water Supplied'] = "No"
#         color_water_result['Water Supply Flow'] = str(water_flow) + " lpm"  # -------------- Add this 

#     if color_water_result['Water Supply Flow'] != '':
#         discription_water_supply = discription_water_supply_processing(water_text_data)  # -------------- Add this

    #discription_fire_sprinkler = discription_fire_sprinkler_processing(sprinkler_text_data)
    #discription_fire_detection = discription_fire_detection_processing(detection_text_data)
    # try:
    # if (color_fire_result['Public Fire Department'] != "" or color_fire_result['distance'] != "" or color_fire_result['responce'] != "" or color_fire_result['organization_type'] != ""):
    #	if (color_fire_result['responce'] != "" or color_fire_result['distance'] != "" or color_fire_result['organization_type'] != "" or color_fire_result['Public Fire Department'] != ""):
    #		df_protection_fire_brigade = intermidiet_table_fire_brigade(file, color_fire_result, "output")
    # color_fire_result_flag = True
    # except IndexError:
    #	print("Index Not enough data to create a csv file")
    # except KeyError:
    # print("Key Not enough data to create a csv file")
    # color_fire_result = {}

    
    if	(color_sprinkler_result['Coverage Provided'] == "" or color_sprinkler_result['Extra coverage required'] == "" or color_detection_result['Coverage Provided'] == "") and (CALCULATED__ == "Got Value"):   # --------------------- Changes for form recognizer fail    
        print("Form recognizer fail handled : ",  perc_sprinkler, perc_sprinkler_required, perc_detection )   # --------------------- Changes for form  recognizer fail
        color_sprinkler_result['Coverage Provided'] = perc_sprinkler   # --------------------- Changes for form  recognizer fail
        color_sprinkler_result['Extra coverage required'] = perc_sprinkler_required   # --------------------- Changes for form  recognizer fail
        if perc_sprinkler == "0%" or perc_sprinkler == "0 %":   # --------------------- Changes for form  recognizer fail
            color_sprinkler_result['Sprinklers Installed'] = "No"   # --------------------- Changes for form  recognizer fail
        else:   # --------------------- Changes for form  recognizer fail
            color_sprinkler_result['Sprinklers Installed'] = "Yes"   # --------------------- Changes for form  recognizer fail
        color_detection_result['Coverage Provided'] = perc_detection   # --------------------- Changes for form  recognizer fail
        if perc_detection == "0%" or perc_detection == "0 %":   # --------------------- Changes for form  recognizer fail
            color_detection_result['Detection Installed'] = "No"   # --------------------- Changes for form  recognizer fail
        else:   # --------------------- Changes for form  recognizer fail
            color_detection_result['Detection Installed'] = "Yes"   # --------------------- Changes for form  recognizer fail
    
    
    # intermidiet_table_water_supply, intermidiet_table_sprinkler, intermidiet_table_detection
    df_protection_fire_brigade = intermidiet_table_fire_brigade(file, color_fire_result, "output",
                                                                discription_fire_bridage)
    #print("color_fire_result : ", color_fire_result)
    #print(df_protection_fire_brigade)

    color_detection_result = final_rating_calculation_fire_detection(color_detection_result, carrier_name)  # -------------- Add this
    #print("color_detection_result : ", color_detection_result)
    df_protection_detection = intermidiet_table_detection(file, color_detection_result, "output",
                                                          discription_fire_detection)
    #print(df_protection_detection)

    color_sprinkler_result = final_rating_calculation_fire_sprinkler(color_sprinkler_result, carrier_name)  # -------------- Add this
    #print("color_sprinkler_result : ", color_sprinkler_result)
    df_protection_sprinkler = intermidiet_table_sprinkler(file, color_sprinkler_result, "output",
                                                          discription_fire_sprinkler)
    #print(df_protection_sprinkler)

    #print("color_water_result : ", color_water_result)
    df_protection_water_supply = intermidiet_table_water_supply(file, color_water_result, "output",
                                                                discription_water_supply)
    #print(df_protection_water_supply)

    data = final_UI(discription_fire_bridage, discription_water_supply,discription_fire_sprinkler ,discription_fire_detection,
                    color_fire_result, color_water_result, color_sprinkler_result,
                    color_detection_result)

    
    
    
#print(55,df_water_supply_grc)
#print(66,final_ui_water_grc)  
# print("final_ui_water_grc : ", final_ui_water_grc)
# print()
# print("df_fire_brigade_grc : ", df_fire_brigade_grc)
# print("discription_fire_bridage : ", discription_fire_bridage_grc)
# print("grc_fire_result : ", grc_fire_result)
# print()
# print("chubb_fire_result : ", chubb_fire_result)
# print("df_fire_brigade_chubb : ", df_fire_brigade_chubb)
# print()
# print("chubb_water_result : ", chubb_water_result)
# print("df_water_supply_chubb : ", df_water_supply_chubb)
# print("final_ui_water : ", final_ui_water)
# print()
# print("chubb_sprinkler_result : ", chubb_sprinkler_result)
# print("df_fire_sprinkler_chubb : ", df_fire_sprinkler_chubb)
# print("final_ui_sprinkler : ", final_ui_sprinkler)
# print()
# print("chubb_detection_result : ", chubb_detection_result)
# print("df_fire_detection_chubb : ", df_fire_detection_chubb)
# print("final_ui_detection : ", final_ui_detection)
# print()
#print("Data",data)

#xzxz

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-928412721240104> in <module> 
 9 # chubb_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''} 
 10 
 ---> 11 df_fire_brigade_grc = pd . DataFrame ( ) 
 12 df_water_supply_grc = pd . DataFrame ( ) 
 13 df_fire_brigade_chubb = pd . DataFrame ( ) 

 NameError : name 'pd' is not defined

In [0]:
try:
  print("Data : ",data)
except:
  print("Please check the format of the file")
  dbutils.notebook.exit(json.dumps({
        "isToast": "True",
        "Error_message": "Protection - The file format or the carrier is not handled. Please check the file !!!!!",
        "filename": fileName
  }))  

# pdf2image_path_protection = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection/" + "protection" + "_" + first_file + res + '/'
# shutil.rmtree(pdf2image_path_protection)
# logo_detection_path = "/dbfs/FileStore/tables/riskreport/FinalCode/pdf2images_Protection_logodetection/" + "Logo" + "_" + first_file + res + '/'
# shutil.rmtree(logo_detection_path)

shutil.rmtree(final_protection_images_path)
os.remove(toc_name+"final_risk_protection.toml")
shutil.rmtree(final_logo_detection)
#json_string = json.dumps(data)
#json_df=pd.read_json(json_string)
#filename=fileName.split('.')[0] if '.' in fileName else fileName
#csv_file_name='/dbfs/mnt/aria/test_final_Protection/'+filename+'.csv'
#json_df.to_csv(csv_file_name,index=False,header=True)
dbutils.notebook.exit(json.dumps(data))
#xzxz

{"protection": [{"Component_Name": "Protection", "Components": [{"name": "Fire Brigade", "Output": ["Public fire brigade Full-time", "10.0 Minute", "3.5 Mile", "Excellent"], "Chubb_Rating": ["AA", "", "", ""], "Description": ["a municipal paid full time (24/7) fire department is located within 3.5 miles of the location with a response time of 10 minutes with no road and/or traffic issues. there is an additional municipal paid full time fire department located within 10 miles. fire department visits the facility once a year. the facility is provided with several fire department connections at the main buildings entrance. this fire department is considered to be iso 6. ", "a municipal paid full time (24/7) fire department is located within 3.5 miles of the location with a response time of 10 minutes with no road and/or traffic issues. ", "a municipal paid full time (24/7) fire department is located within 3.5 miles of the location with a response time of 10 minutes with no road and/or tr

In [0]:
# display(df_protection_fire_brigade)
# save_file = "/dbfs/mnt/aria/csv_output/"+filename+"_fire_brigade.csv"
# df_protection_fire_brigade.to_csv(save_file, index=False)

In [0]:
# display(df_protection_detection)
# save_file = "/dbfs/mnt/aria/csv_output/"+filename+"_detection.csv"
# df_protection_detection.to_csv(save_file, index=False)

In [0]:
# display(df_protection_sprinkler)
# save_file = "/dbfs/mnt/aria/csv_output/"+filename+"_sprinklers.csv"
# df_protection_sprinkler.to_csv(save_file, index=False)

In [0]:
# display(df_protection_water_supply)
# save_file = "/dbfs/mnt/aria/csv_output/"+filename+"_water_supply.csv"
# df_protection_water_supply.to_csv(save_file, index=False)

In [0]:
# #print(result_discription)
# #discription_fire_bridage = discription_fire_bridage(result)
# #print("discription_fire_bridage : ",discription_fire_bridage) # ---- Add this

# #print("discription_fire_bridage distance : ",discription_fire_bridage['distance'])
# #print("discription_fire_bridage responce : ",discription_fire_bridage['responce'])
# #print("discription_fire_bridage org : ",discription_fire_bridage['org'])
# #print("discription_water_supply : ",discription_water_supply) # ---- Add this

# try:
# 	if discription_fire_bridage['org'] !="" or discription_fire_bridage['responce']!="" or discription_fire_bridage['distance'] !="":
# 		pass
# except:
# 	discription_fire_bridage['org'] = ""
# 	discription_fire_bridage['responce'] = ""
# 	discription_fire_bridage['distance'] = ""

    
# try:
# 	if discription_water_supply['install'] != "" or discription_water_supply['flow'] != "":
# 		pass
# except:
# 	discription_water_supply['install'] = ""
# 	discription_water_supply['flow'] = ""
    
    
# try:
# 	if discription_fire_sprinkler['install'] !=  "":
# 		pass
# except:
# 	discription_fire_sprinkler['install'] = ""
    
    
# try:
# 	if  discription_fire_detection['install'] != "":
# 		pass
# except:
# 	discription_fire_detection['install'] = ""
    
    
    
# data = {'protection': 
# 	[{'Component_Name':'Protection', 'Components': 
# 		[{'name': 'Fire Brigade', 'Output':[color_fire_result['organization_type'],color_fire_result['responce'],color_fire_result['distance'],color_fire_result['Public Fire Department']],'Chubb_Rating':[color_fire_result['fire_chubb_score'],"","",""], 'Description':[discription_fire_bridage['org'],discription_fire_bridage['responce'],discription_fire_bridage['distance'],""]},
# 	{'name': 'Water Supply', 'Output':[color_water_result['Class for NAICS'],color_water_result['Water Supplied'],color_water_result['Water Supply Flow'],color_water_result['Water Supply']],'Chubb_Rating':[color_water_result['Water Supply Rating'],"","",""], 'Description':["",discription_water_supply['install'],discription_water_supply['flow'],""]},

# 	{'name': 'Sprinklers', 'Output':[color_sprinkler_result['Sprinklers Installed'],color_sprinkler_result['Coverage Provided'],color_sprinkler_result['Extra coverage required'],color_sprinkler_result['Coverage is adequate'],color_sprinkler_result['Sprinklers']],'Chubb_Rating':[color_sprinkler_result['Sprinkler Rating'],"","","",""], 'Description':[discription_fire_sprinkler['install'],"","","",""]},

# 	{'name': 'Detection', 'Output':[color_detection_result['Detection Installed'],color_detection_result['Coverage Provided'],color_detection_result['Extra coverage required'],color_detection_result['Coverage is adequate'],color_detection_result['Detection']],'Chubb_Rating':[color_detection_result['Detection Rating'],"","","",""], 'Description':[discription_fire_detection['install'],"","","",""]}]}]}

# print(data)

In [0]:
# grc_fire_result = {'Public Fire Department':'', 'fire_chubb_score':'', 'distance':'', 'responce':'', 'organization_type':''}
# grc_sprinkler_result = {"Sprinklers":'', 'Sprinklers Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Sprinkler Rating":''}
# grc_detection_result = {"Detection":'', 'Detection Installed':"", 'Coverage Provided':"", 'Extra coverage required':"","Coverage is adequate":"", "Detection Rating":''}
# grc_water_result = {"Water Supply":'', "Class for NAICS":'', "Water Supplied":'', "Water Supply Flow":'', "Water Supply Rating":''}





In [0]:

# if carrier_name == 'CHUBB':
#   data = final_UI(discription_fire_bridage, discription_water_supply, discription_fire_detection, discription_fire_sprinkler, chubb_fire_result, chubb_water_result, chubb_sprinkler_result, chubb_detection_result)
#   print(data)
# elif carrier_name == 'GRC':
#   data = final_UI(discription_fire_bridage_grc, discription_water_supply, discription_detection, discription_sprinkler, grc_fire_result, grc_water_result, grc_sprinkler_result, grc_detection_result)
#   print(data)